In [ ]:
# DANTE Rosenbrock 优化任务 - Kaggle版本

## 项目概述
基于CNN代理模型的DANTE优化算法，用于解决20维Rosenbrock函数优化问题。
主要目标是调试和优化NTE搜索器中的动态探索因子参数。

## 当前配置 (已恢复到原项目设置)
- **维度**: 20D Rosenbrock函数
- **初始样本**: 800个
- **迭代次数**: 400次主动学习 (原项目完整设置)
- **每次迭代样本**: 20个新样本
- **CNN训练轮数**: 200 epochs (原项目设置)
- **重点调试**: `dynamic_exploration_factor = 0.2 + 1 * min(20, 2 ** no_improvement_streak)` (原项目验证的最佳公式)

## P100显卡优化
- 专门针对Kaggle P100显卡进行了性能优化
- 虚拟GPU内存限制: 15GB
- 混合精度训练: mixed_float16
- XLA编译优化: 已启用


In [ ]:
# Cell 0: 环境设置与GPU配置
import os
import sys
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import logging
import torch
import random
from scipy.stats import pearsonr
import traceback
from datetime import datetime
import gc
import tensorflow as tf
from tensorflow.python.framework import ops
import psutil

# GPU配置优化
print("=== GPU环境配置 ===")

# TensorFlow GPU设置
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 启用内存增长
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"TensorFlow检测到{len(gpus)}个GPU设备")
        for i, gpu in enumerate(gpus):
            print(f"  GPU {i}: {gpu}")
    except RuntimeError as e:
        print(f"GPU设置错误: {e}")
else:
    print("未检测到GPU设备，将使用CPU")

# PyTorch GPU设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print(f"PyTorch将使用设备: {device}")
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"GPU设备名称: {torch.cuda.get_device_name(0)}")
    print(f"GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print(f"PyTorch将使用CPU")

# 可视化设置
plt.rcParams['font.size'] = 12
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['font.family'] = 'DejaVu Sans'  # 使用默认字体避免中文问题

# 配置日志记录器
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.info("环境配置完成")

# Kaggle环境路径设置 - 更新为实际数据集路径
DATA_PATH = "/kaggle/input/rosenbrock-data-20d-800/rosenbrock_data_raw"
RESULTS_PATH = "/kaggle/working/results"

# 创建结果目录
os.makedirs(RESULTS_PATH, exist_ok=True)
logger.info(f"数据路径: {DATA_PATH}")
logger.info(f"结果路径: {RESULTS_PATH}")

# 验证数据文件是否存在
data_files = [
    "Rosenbrock_x_train.npy",
    "Rosenbrock_y_train.npy", 
    "Rosenbrock_x_test.npy",
    "Rosenbrock_y_test.npy"
]

print("\n=== 数据文件检查 ===")
for file in data_files:
    file_path = os.path.join(DATA_PATH, file)
    if os.path.exists(file_path):
        print(f"✅ {file} - 找到")
        if file.endswith('.npy'):
            data_shape = np.load(file_path).shape
            print(f"   数据形状: {data_shape}")
    else:
        print(f"❌ {file} - 未找到")
        logger.warning(f"数据文件未找到: {file_path}")

print("\n=== 环境配置完成 ===")


In [ ]:
# Cell 1: 核心工具函数 (utils.py)
import numpy as np

def rosenbrock_function(x):
    """
    Calculates the Rosenbrock function value for a given input vector x.
    The function is typically evaluated on the hypercube xi ∈ [-5, 10] for all i, 
    although it may be evaluated for xi ∈ [-2.048, 2.048] as well.
    For our DANTE optimization, we usually normalize inputs to [0,1] or work within specific bounds.
    Args:
        x (np.ndarray): Input vector of shape (n,) or (batch_size, n).
    Returns:
        float or np.ndarray: Rosenbrock function value(s).
    """
    if not isinstance(x, np.ndarray):
        x = np.array(x)

    if x.ndim == 1:
        # Single input vector
        n = len(x)
        if n < 2:
            raise ValueError("Rosenbrock function requires at least 2 dimensions.")
        sum_val = 0
        for i in range(n - 1):
            sum_val += 100 * (x[i+1] - x[i]**2)**2 + (x[i] - 1)**2
        return sum_val
    elif x.ndim == 2:
        # Batch of input vectors
        n = x.shape[1]
        if n < 2:
            raise ValueError("Rosenbrock function requires at least 2 dimensions for each vector in the batch.")
        results = [] 
        for i in range(x.shape[0]):
            sum_val = 0
            for j in range(n - 1):
                sum_val += 100 * (x[i, j+1] - x[i, j]**2)**2 + (x[i, j] - 1)**2
            results.append(sum_val)
        return np.array(results)
    else:
        raise ValueError("Input x must be a 1D or 2D NumPy array.")


def get_rosenbrock_bounds(dimension):
    """
    Returns the typical bounds for the Rosenbrock function for a given dimension.
    Often [-5, 10] or [-2.048, 2.048]. Let's use [-5, 10] for now.
    """
    return [(-5.0, 10.0)] * dimension

# 外部评估器
def rosenbrock_evaluator(x_vector):
    """计算给定向量x的Rosenbrock函数值。"""
    return rosenbrock_function(x_vector)

# 定义LOG_EPSILON常量
LOG_EPSILON = 1.0  # 使用10为底的对数变换: log10(y + 1)

# NTE相关常量
def safe_power10(log_values):
    """安全地计算10的幂，避免溢出"""
    # 限制输入范围以避免溢出
    log_values_clipped = np.clip(log_values, -10, 10)
    return 10.0 ** log_values_clipped - LOG_EPSILON

NTE_C0 = 1
NTE_ROLLOUT_ROUNDS = 100

logger.info("核心工具函数加载完成")
print("测试Rosenbrock函数...")
test_x = np.array([1, 1, 1])
test_result = rosenbrock_function(test_x)
print(f"Rosenbrock([1,1,1]) = {test_result} (期望值: 0)")


In [ ]:
# Cell 2: 数据管理模块 (data_manager.py)
import numpy as np
import torch
import logging
import os
import hashlib

class DataManager:
    def __init__(self, initial_X=None, initial_y=None, dimension=None, data_file_path=None):
        """
        Initializes the DataManager.
        Args:
            initial_X (np.ndarray, optional): Initial X data. Defaults to None.
            initial_y (np.ndarray, optional): Initial y data (original scale). Defaults to None.
            dimension (int, optional): Dimension of the X data.
            data_file_path (str, optional): Path to an .npz file to load data from.
        """
        self.X_data = np.array([])
        self.y_data_orig = np.array([])  # Stores original y values
        self.y_data_log = np.array([])   # Stores log-transformed y values
        self.dimension = dimension
        self.mean_X = None
        self.std_X = None
        
        # 添加域范围属性，用于NTE搜索
        self.x_min = -2.048  # Rosenbrock函数的默认下限
        self.x_max = 2.048   # Rosenbrock函数的默认上限
        
        # 添加样本权重属性
        self.last_batch_weights = None
        
        # 添加数据缓存
        self._normalized_cache = {}
        self._cache_hash = None

        if data_file_path and os.path.exists(data_file_path):
            logger.info(f"Loading data from {data_file_path}")
            try:
                data = np.load(data_file_path)
                self.X_data = data.get('X_data', np.array([]))
                loaded_y_orig = data.get('y_data', np.array([])) 
                
                if self.X_data.ndim == 1 and self.X_data.shape[0] > 0:
                    self.X_data = self.X_data.reshape(1, -1)
                if loaded_y_orig.ndim == 1 and loaded_y_orig.shape[0] > 0:
                     loaded_y_orig = loaded_y_orig.reshape(-1,1)

                self.y_data_orig = loaded_y_orig
                if self.y_data_orig.shape[0] > 0:
                    self.y_data_log = np.log10(self.y_data_orig + LOG_EPSILON)
                else:
                    self.y_data_log = np.empty((0,1))

                self.mean_X = data.get('mean_X', None)
                self.std_X = data.get('std_X', None)
                
                if self.X_data.shape[0] > 0:
                    self.dimension = self.X_data.shape[1]
                    logger.info(f"Loaded {self.X_data.shape[0]} samples.")
                    if self.mean_X is not None and self.std_X is not None:
                        logger.info("Loaded existing normalization parameters for X.")
                    else:
                        logger.info("Normalization parameters not found, will compute them.")
                        self.normalize_X()
                else:
                    logger.warning(f"No data found in {data_file_path} or data is empty.")
                    if initial_X is not None and initial_y is not None:
                        logger.info("Falling back to provided initial_X and initial_y.")
                        self.X_data = initial_X.copy()
                        self.y_data_orig = initial_y.copy()
                        if self.y_data_orig.ndim == 1: 
                            self.y_data_orig = self.y_data_orig.reshape(-1, 1)
                        self.y_data_log = np.log10(self.y_data_orig + LOG_EPSILON)
                        if self.dimension is None and self.X_data.shape[0] > 0:
                            self.dimension = self.X_data.shape[1]
                        self.normalize_X()
                    elif self.dimension is None:
                        raise ValueError("Dimension must be provided if not loading from file and no initial data.")

            except Exception as e:
                logger.error(f"Error loading data from {data_file_path}: {e}")
                if initial_X is not None and initial_y is not None:
                    logger.info("Falling back to provided initial_X and initial_y due to loading error.")
                    self.X_data = initial_X.copy()
                    self.y_data_orig = initial_y.copy()
                    if self.y_data_orig.ndim == 1: 
                        self.y_data_orig = self.y_data_orig.reshape(-1, 1)
                    self.y_data_log = np.log10(self.y_data_orig + LOG_EPSILON)
                    if self.dimension is None and self.X_data.shape[0] > 0:
                        self.dimension = self.X_data.shape[1]
                    self.normalize_X()
                elif self.dimension is None:
                    raise ValueError("Dimension must be provided if loading fails and no initial data.")
        
        elif initial_X is not None and initial_y is not None:
            logger.info("Initializing with provided initial_X and initial_y.")
            self.X_data = initial_X.copy()
            self.y_data_orig = initial_y.copy()
            if self.X_data.ndim == 1: 
                self.X_data = self.X_data.reshape(1, -1)
            if self.y_data_orig.ndim == 1: 
                self.y_data_orig = self.y_data_orig.reshape(-1, 1)
            self.y_data_log = np.log10(self.y_data_orig + LOG_EPSILON)
            if self.dimension is None and self.X_data.shape[0] > 0:
                 self.dimension = self.X_data.shape[1]
            self.normalize_X()
        elif self.dimension is None:
             raise ValueError("Dimension must be provided if no data source is specified.")
        else:
            logger.info(f"DataManager initialized empty. Dimension set to {self.dimension}.")
            self.X_data = np.empty((0, self.dimension if self.dimension else 0))
            self.y_data_orig = np.empty((0, 1))
            self.y_data_log = np.empty((0, 1))
            
        self._update_best()

    def normalize_X(self):
        """改进的规范化方法，使用鲁棒性缩放以处理异常值"""
        if self.X_data.shape[0] > 1:
            self.mean_X = np.mean(self.X_data, axis=0)
            self.std_X = np.std(self.X_data, axis=0)
            
            # 处理近零方差的特征
            epsilon = 1e-6
            self.std_X = np.where(self.std_X < epsilon, 1.0, self.std_X)
            
            # 检查是否有需要鲁棒性处理的极端值
            X_normalized = (self.X_data - self.mean_X) / self.std_X
            extremes = np.abs(X_normalized) > 5.0
            
            if np.any(extremes):
                # 使用分位数方法处理极端值
                q1 = np.percentile(self.X_data, 25, axis=0)
                q3 = np.percentile(self.X_data, 75, axis=0)
                iqr = q3 - q1
                
                # 将IQR为0的维度设为1，防止除零
                iqr = np.where(iqr < epsilon, 1.0, iqr)
                
                # 基于IQR的规范化参数
                self.mean_X = np.median(self.X_data, axis=0)
                self.std_X = iqr / 1.35
                
                logger.info("使用鲁棒性规范化 (基于分位数) 处理极端值")
            else:
                logger.info("使用标准规范化 (均值和标准差)")
        elif self.X_data.shape[0] == 1:
            self.mean_X = self.X_data[0]
            self.std_X = np.ones_like(self.X_data[0])
            logger.info("X normalization parameters set for single data point.")
        else:
            self.mean_X = None
            self.std_X = None
            logger.info("No data to calculate X normalization parameters.")
            
    def get_normalized_X(self, X_samples):
        """使用存储的参数对X样本进行规范化，支持缓存优化"""
        # 生成数据哈希用于缓存
        data_hash = hashlib.md5(X_samples.tobytes()).hexdigest()
        
        # 检查缓存
        if data_hash in self._normalized_cache:
            logger.debug("使用缓存的规范化数据")
            return self._normalized_cache[data_hash]
        
        if self.mean_X is None or self.std_X is None:
            if self.X_data.shape[0] > 0:
                 logger.warning("Normalization params not available, attempting to calculate them now.")
                 self.normalize_X()
                 if self.mean_X is None or self.std_X is None:
                    raise ValueError("Normalization parameters could not be calculated.")
            else:
                logger.warning("No stored normalization parameters. Normalizing based on input samples.")
                if X_samples.shape[0] > 1:
                    mean_temp = np.mean(X_samples, axis=0)
                    std_temp = np.std(X_samples, axis=0)
                    std_temp[std_temp < 1e-6] = 1.0
                    normalized = (X_samples - mean_temp) / std_temp
                else:
                    normalized = np.zeros_like(X_samples)
                
                # 缓存结果（限制缓存大小）
                if len(self._normalized_cache) < 10:
                    self._normalized_cache[data_hash] = normalized.copy()
                return normalized

        # 应用规范化，并截断极端值到合理范围
        normalized = (X_samples - self.mean_X) / self.std_X
        normalized = np.clip(normalized, -10.0, 10.0)
        
        # 缓存结果（限制缓存大小）
        if len(self._normalized_cache) < 10:
            self._normalized_cache[data_hash] = normalized.copy()
        
        return normalized

    def get_train_data_for_cnn(self):
        if self.X_data.shape[0] == 0:
            return torch.empty(0), torch.empty(0)
        
        normalized_X = self.get_normalized_X(self.X_data)
        
        if self.dimension != 20:
            raise ValueError(f"DataManager expected dimension 20 for CNN reshape, got {self.dimension}")
        
        reshaped_X = normalized_X.reshape(-1, 1, 4, 5)
        
        # Return log-transformed y for training
        return torch.tensor(reshaped_X, dtype=torch.float32), torch.tensor(self.y_data_log, dtype=torch.float32)

    def add_samples(self, new_X, new_y_orig, re_normalize=False, shuffle=True):
        """添加新样本到数据集中"""
        if new_X.ndim == 1: 
            new_X = new_X.reshape(1, -1)
        if new_y_orig.ndim == 1: 
            new_y_orig = new_y_orig.reshape(-1, 1)
        
        new_y_log = np.log10(new_y_orig + LOG_EPSILON)

        if self.X_data.shape[0] == 0:
            self.X_data = new_X.copy()
            self.y_data_orig = new_y_orig.copy()
            self.y_data_log = new_y_log.copy()
            if self.dimension is None: 
                self.dimension = new_X.shape[1]
        else:
            if new_X.shape[1] != self.dimension:
                raise ValueError(f"New X samples have dimension {new_X.shape[1]}, expected {self.dimension}")
            
            # 先合并数据
            self.X_data = np.vstack((self.X_data, new_X))
            self.y_data_orig = np.vstack((self.y_data_orig, new_y_orig))
            self.y_data_log = np.vstack((self.y_data_log, new_y_log))
            
            # 如果需要打乱，则生成随机排列
            if shuffle:
                n_samples = self.X_data.shape[0]
                permutation = np.random.permutation(n_samples)
                self.X_data = self.X_data[permutation]
                self.y_data_orig = self.y_data_orig[permutation]
                self.y_data_log = self.y_data_log[permutation]
                logger.info(f"数据集已打乱，确保新样本随机分布")
        
        if re_normalize or self.mean_X is None:
            self.normalize_X()
            
        self._update_best()
        logger.info(f"Added {new_X.shape[0]} new samples. Total samples: {self.X_data.shape[0]}")

    def _update_best(self):
        if self.y_data_orig.shape[0] > 0:
            min_idx = np.argmin(self.y_data_orig)
            self.best_y = self.y_data_orig[min_idx, 0]
            self.best_x = self.X_data[min_idx]
        else:
            self.best_y = float('inf')
            self.best_x = None

    def get_current_best(self):
        return self.best_x, self.best_y

    def save_data(self, file_path):
        """保存当前数据和规范化参数到.npz文件"""
        try:
            save_dict = {
                'X_data': self.X_data,
                'y_data': self.y_data_orig
            }
            if self.mean_X is not None:
                save_dict['mean_X'] = self.mean_X
            if self.std_X is not None:
                save_dict['std_X'] = self.std_X
            
            np.savez(file_path, **save_dict)
            logger.info(f"DataManager state saved to {file_path}")
        except Exception as e:
            logger.error(f"Error saving DataManager state to {file_path}: {e}")

    @property
    def num_samples(self):
        return self.X_data.shape[0]

    def get_sorted_data_by_y(self, top_n=None):
        """获取按y值排序的数据（升序，即最优先）"""
        if self.X_data.shape[0] == 0:
            return np.array([]), np.array([])
            
        sorted_indices = np.argsort(self.y_data_orig, axis=0).flatten()
        
        sorted_X = self.X_data[sorted_indices]
        sorted_y_orig = self.y_data_orig[sorted_indices]
        
        if top_n is not None:
            return sorted_X[:top_n], sorted_y_orig[:top_n]
        else:
            return sorted_X, sorted_y_orig
    
    def get_best_samples_range(self, n_best=60):
        """获取最优的N个样本的原始Y值的范围"""
        if self.num_samples == 0:
            return float('nan'), float('nan')
        
        _, sorted_y_orig = self.get_sorted_data_by_y(top_n=n_best)
        
        if sorted_y_orig.shape[0] == 0:
            return float('nan'), float('nan')
            
        min_y = np.min(sorted_y_orig)
        max_y = np.max(sorted_y_orig)
        return min_y, max_y

    def get_all_y_orig(self):
        """Returns all original y data."""
        return self.y_data_orig.copy()

    def get_all_y_log(self):
        """Returns all log-transformed y data."""
        return self.y_data_log.copy()

    def get_all_x_orig(self):
        """Returns all original X data."""
        return self.X_data.copy()
    
    def clear_cache(self):
        """清理数据缓存以释放内存"""
        self._normalized_cache.clear()
        self._cache_hash = None
        logger.info("已清理DataManager缓存")

logger.info("DataManager类加载完成")
print("DataManager模块已准备就绪")


In [ ]:
# Cell 3: 性能优化模块 (简化版)
import os
import gc
import psutil
import numpy as np
import tensorflow as tf
from tensorflow.keras import mixed_precision

class PerformanceOptimizer:
    """性能优化器，负责优化TensorFlow训练性能"""
    
    def __init__(self):
        self.cpu_count = psutil.cpu_count(logical=True)
        self.physical_cpu_count = psutil.cpu_count(logical=False)
        self.memory_cache = {}
        self.is_configured = False
        logger.info(f"检测到CPU核心数: 物理={self.physical_cpu_count}, 逻辑={self.cpu_count}")
    
    def configure_tensorflow_performance(self):
        """配置TensorFlow性能设置"""
        if self.is_configured:
            return
            
        logger.info("开始配置TensorFlow性能优化...")
        
        # 配置GPU内存增长
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logger.info(f"已配置GPU内存增长，检测到{len(gpus)}个GPU")
            except RuntimeError as e:
                logger.warning(f"GPU内存配置失败: {e}")
        
        # 配置CPU并行度
        tf.config.threading.set_inter_op_parallelism_threads(self.cpu_count)
        tf.config.threading.set_intra_op_parallelism_threads(self.physical_cpu_count)
        
        # 配置环境变量
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
        os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
        
        self.is_configured = True
        logger.info("TensorFlow性能优化配置完成")
    
    def create_optimized_dataset(self, X_data, y_data, batch_size=32, shuffle=True, cache=True):
        """创建优化的tf.data.Dataset"""
        logger.info(f"创建优化的数据集，样本数: {len(X_data)}, 批次大小: {batch_size}")
        
        dataset = tf.data.Dataset.from_tensor_slices((X_data, y_data))
        
        if cache and len(X_data) < 50000:
            dataset = dataset.cache()
            logger.info("已启用数据集内存缓存")
        
        if shuffle:
            buffer_size = min(len(X_data), 10000)
            dataset = dataset.shuffle(buffer_size=buffer_size)
        
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
        return dataset
    
    def clear_memory_cache(self):
        """清理内存缓存"""
        self.memory_cache.clear()
        gc.collect()
        logger.info("已清理内存缓存")
    
    def cache_data(self, key, data):
        """缓存数据到内存"""
        self.memory_cache[key] = data
    
    def get_cached_data(self, key):
        """从缓存获取数据"""
        return self.memory_cache.get(key, None)
    
    def optimize_memory_usage(self):
        """优化内存使用"""
        gc.collect()
        if tf.config.experimental.list_physical_devices('GPU'):
            try:
                tf.keras.backend.clear_session()
            except:
                pass
        logger.info("已执行内存优化清理")
    
    def monitor_performance(self, stage_name=""):
        """监控性能指标"""
        try:
            process = psutil.Process()
            memory_info = process.memory_info()
            memory_mb = memory_info.rss / 1024 / 1024
            logger.info(f"[{stage_name}] 内存使用: {memory_mb:.1f}MB")
        except Exception as e:
            logger.warning(f"性能监控失败: {e}")

# 创建全局性能优化器实例
performance_optimizer = PerformanceOptimizer()

logger.info("性能优化模块加载完成")
print("性能优化模块已准备就绪")


In [ ]:
# Cell 5: NTE Searcher (完整版) - 关键调试模块

import math
import random
import logging
from typing import List, Tuple, Dict, Any, Union, Callable
from dataclasses import dataclass
import numpy as np

logger = logging.getLogger(__name__)

# NTE搜索器常量
NTE_C0 = 1
NTE_ROLLOUT_ROUNDS = 100
VALIDATION_BATCH_SIZE_NTE = 20

def safe_power10(log_values, clip_min=-30, clip_max=25):
    """安全地计算10的幂，避免数值溢出"""
    clipped_values = np.clip(log_values, clip_min, clip_max)
    return 10.0 ** clipped_values

@dataclass
class Node:
    """NTE搜索树的节点"""
    state: Tuple[float, ...]  # 状态向量（元组用于哈希）
    value_pred: float = float('inf')  # 替代模型预测值
    visit_count: int = 0  # 访问次数

class NTESearcher:
    """Neural Tree Expansion搜索器 - 完整实现，包含所有原始功能"""
    
    def __init__(self, dimension, domain=None, domain_mins=None, domain_maxs=None, 
                 c0=NTE_C0, rollout_rounds=NTE_ROLLOUT_ROUNDS, 
                 validation_batch_size=VALIDATION_BATCH_SIZE_NTE,
                 rho_scaling_factor_heuristic=lambda min_y: 1.0):
        """初始化NTE搜索器"""
        self.dimension = dimension
        self.base_c0 = c0
        self.rollout_rounds = rollout_rounds
        self.validation_batch_size = validation_batch_size
        self.rho_scaling_factor_heuristic = rho_scaling_factor_heuristic
        
        # 处理域边界 - 保持向后兼容
        if domain_mins is not None and domain_maxs is not None:
            self.domain_mins = np.array(domain_mins)
            self.domain_maxs = np.array(domain_maxs)
        elif domain is not None:
            domain_array = np.array(domain)
            self.domain_mins = domain_array[:, 0]
            self.domain_maxs = domain_array[:, 1]
        else:
            self.domain_mins = np.array([-2.048] * dimension)
            self.domain_maxs = np.array([2.048] * dimension)
            
        # 状态追踪变量
        self.last_move_successful = False
        self.last_move_deterministic = False
        self.last_move_type = None
        self.last_move_direction = None
        self.last_move_dim_index = -1

print("✓ NTE Searcher (完整版第1部分) 已加载")


In [ ]:
# Cell 6: NTE Searcher 完整实现 - 混合扩展策略

def _calculate_dynamic_search_space(self, all_states=None, all_values=None, iteration=1):
    """计算动态搜索空间，考虑已知样本分布和迭代进度"""
    if all_states is None or len(all_states) == 0:
        return self.domain_mins.copy(), self.domain_maxs.copy()
        
    # 基于当前数据集动态调整搜索范围
    data_mins = np.min(all_states, axis=0)
    data_maxs = np.max(all_states, axis=0)
    data_ranges = data_maxs - data_mins
    
    # 计算扩展因子：早期迭代更广泛探索，后期更集中
    base_expansion = 0.2
    iteration_factor = max(0.1, 1.0 - 0.01 * iteration)
    expansion_factor = base_expansion * iteration_factor
    
    # 基于函数值分布调整搜索区域
    if all_values is not None and len(all_values) > 0:
        # 找到最优10%的样本，重点关注这些区域
        sorted_indices = np.argsort(all_values.flatten())
        top_10_percent = max(1, len(sorted_indices) // 10)
        best_samples = all_states[sorted_indices[:top_10_percent]]
        
        if len(best_samples) > 0:
            best_mins = np.min(best_samples, axis=0)
            best_maxs = np.max(best_samples, axis=0)
            best_ranges = best_maxs - best_mins
            
            # 在最优区域周围扩展
            best_expansion = 0.5  # 在最优区域周围扩展50%
            extended_best_mins = best_mins - best_expansion * best_ranges
            extended_best_maxs = best_maxs + best_expansion * best_ranges
            
            # 合并全局搜索区域和最优区域
            extended_mins = np.minimum(data_mins - expansion_factor * data_ranges, extended_best_mins)
            extended_maxs = np.maximum(data_maxs + expansion_factor * data_ranges, extended_best_maxs)
        else:
            extended_mins = data_mins - expansion_factor * data_ranges
            extended_maxs = data_maxs + expansion_factor * data_ranges
    else:
        extended_mins = data_mins - expansion_factor * data_ranges
        extended_maxs = data_maxs + expansion_factor * data_ranges
    
    # 确保在原始域内
    dynamic_mins = np.maximum(extended_mins, self.domain_mins)
    dynamic_maxs = np.minimum(extended_maxs, self.domain_maxs)
    
    # 确保搜索区域不会退化为点
    for i in range(self.dimension):
        if dynamic_maxs[i] - dynamic_mins[i] < 0.01:
            center = (dynamic_mins[i] + dynamic_maxs[i]) / 2
            dynamic_mins[i] = max(self.domain_mins[i], center - 0.005)
            dynamic_maxs[i] = min(self.domain_maxs[i], center + 0.005)
    
    return dynamic_mins, dynamic_maxs

def _mixed_expansion(self, state_vector, dynamic_factor=1.0, all_states=None, 
                    all_values=None, iteration=1, pre_computed_bounds=None):
    """完整的混合扩展策略，包括随机移动和确定性移动"""
    leaf_states = []
    current_state = state_vector.copy()

    if self.dimension <= 0:
        return []

    # 计算动态搜索区间
    if pre_computed_bounds is not None:
        dynamic_mins, dynamic_maxs = pre_computed_bounds
    else:
        dynamic_mins, dynamic_maxs = self._calculate_dynamic_search_space(all_states, all_values, iteration)
    
    # 计算自适应步长
    base_step = 0.05
    adaptive_step = base_step * (0.5 + 0.5 * (dynamic_factor - 1.0))
    
    # 调整探索概率
    stochastic_prob = 0.67  # 默认67%概率使用随机移动
    
    if self.last_move_successful and self.last_move_deterministic:
        stochastic_prob = 0.0  # 如果上次确定性移动成功，继续使用确定性移动
        logger.debug("上次确定性移动成功，继续使用确定性移动(100%)")
    else:
        logger.debug(f"使用标准概率: {stochastic_prob*100:.1f}%随机移动, {(1-stochastic_prob)*100:.1f}%确定性移动")
    
    # 混合扩展策略：生成维度数量的叶节点
    for _ in range(self.dimension): 
        new_state = current_state.copy()
        
        use_stochastic = random.random() < stochastic_prob
        
        if use_stochastic: 
            # 随机移动
            move_type = random.choices(
                ['single_var', 'd_div_5_vars', 'd_div_10_vars'],
                weights=[0.5, 0.25, 0.25],
                k=1
            )[0]
            
            self.last_move_deterministic = False
            
            if move_type == 'single_var':
                # 随机改变1个变量
                idx = random.randint(0, self.dimension - 1)
                new_state[idx] = random.uniform(dynamic_mins[idx], dynamic_maxs[idx])
                
            elif move_type == 'd_div_5_vars':
                # 随机改变d/5个变量
                num_vars_to_change = max(1, int(self.dimension / 5))
                indices = random.sample(range(self.dimension), num_vars_to_change)
                for idx in indices:
                    new_state[idx] = random.uniform(dynamic_mins[idx], dynamic_maxs[idx])
                    
            elif move_type == 'd_div_10_vars':
                # 随机改变d/10个变量
                num_vars_to_change = max(1, int(self.dimension / 10))
                indices = random.sample(range(self.dimension), num_vars_to_change)
                for idx in indices:
                    new_state[idx] = random.uniform(dynamic_mins[idx], dynamic_maxs[idx])
        else:
            # 确定性移动
            self.last_move_deterministic = True
            self.last_move_type = "multi_dim_step"
            
            # 选择修改的维度数量
            dim_choice = random.choices(
                ['single_dim', 'd_div_5_dims', 'd_div_10_dims', 'd_div_2_dims'],
                weights=[0.91, 0.03, 0.03, 0.03],
                k=1
            )[0]
            
            # 确定要修改的维度索引
            if dim_choice == 'single_dim':
                if self.last_move_successful and self.last_move_dim_index >= 0:
                    indices = [self.last_move_dim_index]
                else:
                    indices = [random.randint(0, self.dimension - 1)]
                logger.debug(f"确定性移动: 修改单个维度")
                
            elif dim_choice == 'd_div_5_dims':
                num_dims = max(4, int(self.dimension / 5))
                if self.last_move_successful and self.last_move_dim_index >= 0:
                    remaining_dims = [i for i in range(self.dimension) if i != self.last_move_dim_index]
                    additional_dims = random.sample(remaining_dims, min(num_dims - 1, len(remaining_dims)))
                    indices = [self.last_move_dim_index] + additional_dims
                else:
                    indices = random.sample(range(self.dimension), num_dims)
                logger.debug(f"确定性移动: 修改d/5={num_dims}个维度")
                
            elif dim_choice == 'd_div_10_dims':
                num_dims = max(20, int(self.dimension / 10))
                if self.last_move_successful and self.last_move_dim_index >= 0:
                    remaining_dims = [i for i in range(self.dimension) if i != self.last_move_dim_index]
                    additional_dims = random.sample(remaining_dims, min(num_dims - 1, len(remaining_dims)))
                    indices = [self.last_move_dim_index] + additional_dims
                else:
                    indices = random.sample(range(self.dimension), num_dims)
                logger.debug(f"确定性移动: 修改d/10={num_dims}个维度")
            
            elif dim_choice == 'd_div_2_dims':
                num_dims = 2
                if self.last_move_successful and self.last_move_dim_index >= 0:
                    remaining_dims = [i for i in range(self.dimension) if i != self.last_move_dim_index]
                    additional_dims = random.sample(remaining_dims, min(num_dims - 1, len(remaining_dims)))
                    indices = [self.last_move_dim_index] + additional_dims
                else:
                    indices = random.sample(range(self.dimension), num_dims)
                logger.debug(f"确定性移动: 修改2个维度")
            
            # 记录第一个维度
            if indices:
                self.last_move_dim_index = indices[0]
            
            # 对选定的维度进行小步调整
            for idx in indices:
                if self.last_move_successful and idx == self.last_move_dim_index:
                    direction = self.last_move_direction
                else:
                    direction = random.choice([-1, 1])
                
                if idx == indices[0]:
                    self.last_move_direction = direction
                
                step_size = 0.1 
                new_state[idx] = current_state[idx] + direction * step_size
        
        # 确保状态在动态搜索区间内
        np.clip(new_state, dynamic_mins, dynamic_maxs, out=new_state)
        leaf_states.append(new_state)

    # 确保生成足够的叶节点
    if not leaf_states: 
        rand_state = np.array([random.uniform(dynamic_mins[i], dynamic_maxs[i]) for i in range(self.dimension)])
        return [rand_state] * self.dimension if self.dimension > 0 else []
    
    while len(leaf_states) < self.dimension and self.dimension > 0:
        leaf_states.append(leaf_states[-1].copy())

    return leaf_states[:self.dimension]

# 将方法添加到NTESearcher类中
NTESearcher._calculate_dynamic_search_space = _calculate_dynamic_search_space
NTESearcher._mixed_expansion = _mixed_expansion

print("✓ NTE Searcher 核心扩展方法已添加")


In [ ]:
# Cell 7: 环境配置验证与基本功能测试
import traceback

def comprehensive_environment_test():
    """执行全面的环境配置验证和基本功能测试"""
    print("🚀 开始执行DANTE环境配置验证...")
    print("=" * 60)
    
    test_results = {
        'gpu_config': False,
        'data_loading': False,
        'model_creation': False,
        'model_training': False,
        'nte_search': False,
        'visualization': False
    }
    
    # 1. GPU配置验证
    print("📊 1. GPU配置验证")
    try:
        print(f"   PyTorch CUDA可用: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"   GPU设备数量: {torch.cuda.device_count()}")
            print(f"   当前GPU: {torch.cuda.get_device_name()}")
            print(f"   GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
        
        print(f"   TensorFlow GPU可用: {len(tf.config.list_physical_devices('GPU')) > 0}")
        if tf.config.list_physical_devices('GPU'):
            print(f"   TensorFlow GPU数量: {len(tf.config.list_physical_devices('GPU'))}")
        
        test_results['gpu_config'] = True
        print("   ✅ GPU配置验证通过")
    except Exception as e:
        print(f"   ❌ GPU配置验证失败: {e}")
    
    print()
    
    # 2. 数据加载与管理验证
    print("📁 2. 数据加载与管理验证")
    try:
        # 生成测试数据
        test_dimension = 20
        test_samples = 100
        
        print(f"   生成测试数据: {test_samples}个样本, {test_dimension}维")
        X_test = np.random.uniform(-2.048, 2.048, (test_samples, test_dimension))
        y_test = np.array([rosenbrock_function(x) for x in X_test]).reshape(-1, 1)
        
        print(f"   Rosenbrock函数值范围: {y_test.min():.2e} - {y_test.max():.2e}")
        
        # 测试DataManager
        data_manager = DataManager(
            initial_X=X_test[:80], 
            initial_y=y_test[:80], 
            dimension=test_dimension
        )
        
        print(f"   DataManager初始化成功: {data_manager.num_samples}个样本")
        print(f"   数据标准化测试: X范围{data_manager.X_normalized.min():.3f}-{data_manager.X_normalized.max():.3f}")
        print(f"   对数变换测试: y_log范围{data_manager.y_log.min():.3f}-{data_manager.y_log.max():.3f}")
        
        test_results['data_loading'] = True
        print("   ✅ 数据加载与管理验证通过")
    except Exception as e:
        print(f"   ❌ 数据加载与管理验证失败: {e}")
        traceback.print_exc()
    
    print()
    
    # 3. CNN模型创建验证
    print("🧠 3. CNN模型创建验证")
    try:
        cnn_model = CNN1DSurrogate(input_dim=test_dimension)
        print(f"   CNN1D模型创建成功")
        print(f"   模型参数数量: {sum(p.numel() for p in cnn_model.model.parameters() if p.requires_grad):,}")
        
        # 测试预测功能
        test_input = X_test[:5]
        predictions = cnn_model.predict(test_input)
        print(f"   模型预测测试: 输入{test_input.shape} -> 输出{predictions.shape}")
        print(f"   预测值范围: {predictions.min():.3f} - {predictions.max():.3f}")
        
        test_results['model_creation'] = True
        print("   ✅ CNN模型创建验证通过")
    except Exception as e:
        print(f"   ❌ CNN模型创建验证失败: {e}")
        traceback.print_exc()
    
    print()
    
    # 4. 模型训练验证（快速测试）
    print("🏃 4. 模型训练验证（快速测试）")
    try:
        # 快速训练测试（仅5个epoch）
        print("   开始快速训练测试（5个epochs）...")
        training_history = cnn_model.train(
            data_manager=data_manager,
            epochs=5,
            batch_size=16,
            validation_split=0.2,
            patience=10,
            verbose=1
        )
        
        final_loss = training_history['loss'][-1]
        print(f"   训练完成，最终损失: {final_loss:.4f}")
        
        # 测试训练后的预测
        post_train_predictions = cnn_model.predict(test_input)
        print(f"   训练后预测值范围: {post_train_predictions.min():.3f} - {post_train_predictions.max():.3f}")
        
        test_results['model_training'] = True
        print("   ✅ 模型训练验证通过")
    except Exception as e:
        print(f"   ❌ 模型训练验证失败: {e}")
        traceback.print_exc()
    
    print()
    
    # 5. NTE搜索验证
    print("🔍 5. NTE搜索验证")
    try:
        # 创建NTE搜索器
        nte_searcher = NTESearcher(
            dimension=test_dimension,
            domain=[(-2.048, 2.048) for _ in range(test_dimension)],
            c0=1,
            rollout_rounds=10,  # 减少轮次以加快测试
            validation_batch_size=5   # 减少批次大小
        )
        
        print(f"   NTESearcher创建成功")
        
        # 创建简化的surrogate wrapper用于测试
        class TestSurrogateWrapper:
            def __init__(self, model):
                self.model = model
            
            def predict(self, X_batch):
                if X_batch.shape[0] == 0:
                    return np.array([])
                return self.model.predict(X_batch)
        
        wrapper = TestSurrogateWrapper(cnn_model)
        current_best_x, current_best_y = data_manager.get_current_best()
        
        print(f"   开始NTE搜索测试（小规模）...")
        print(f"   当前最优值: {current_best_y:.4e}")
        
        # 执行简化的NTE搜索
        search_result = nte_searcher.search(
            surrogate_model=wrapper,
            current_best_state=current_best_x,
            min_y_observed=current_best_y,
            dynamic_exploration_factor=1.0,
            all_states=data_manager.X_data,
            all_values=data_manager.get_all_y_orig(),
            previous_iteration_samples=None,
            current_iteration=0,
            return_ducb_trends=True
        )
        
        candidates, ducb_trends = search_result
        print(f"   NTE搜索完成: 找到{len(candidates)}个候选点")
        
        if len(candidates) > 0:
            # 评估候选点
            candidate_values = np.array([rosenbrock_function(x) for x in candidates])
            print(f"   候选点真值范围: {candidate_values.min():.4e} - {candidate_values.max():.4e}")
            print(f"   最佳候选点值: {candidate_values.min():.4e}")
        
        test_results['nte_search'] = True
        print("   ✅ NTE搜索验证通过")
    except Exception as e:
        print(f"   ❌ NTE搜索验证失败: {e}")
        traceback.print_exc()
    
    print()
    
    # 6. 可视化功能验证
    print("📈 6. 可视化功能验证")
    try:
        # 测试基本可视化功能
        import matplotlib.pyplot as plt
        
        # 创建测试图形
        plt.figure(figsize=(8, 6))
        test_x = np.linspace(0, 10, 50)
        test_y = np.sin(test_x)
        plt.plot(test_x, test_y, 'b-', label='Test Plot')
        plt.xlabel('X Values')
        plt.ylabel('Y Values')
        plt.title('Visualization Test')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 在Kaggle环境中显示图形
        plt.tight_layout()
        plt.show()
        plt.close()
        
        print("   ✅ 基本可视化功能正常")
        
        # 测试plot_utils中的函数
        if len(candidates) > 0:
            try:
                # 测试预测vs真值图
                pred_vs_true_metrics = plot_prediction_vs_truth(
                    cnn_model.predict(candidates), 
                    candidate_values, 
                    "/kaggle/working", 
                    iteration=0,
                    filename="test_pred_vs_true.png"
                )
                print(f"   预测vs真值图测试成功: RMSE={pred_vs_true_metrics[1]:.4e}")
            except Exception as e:
                print(f"   高级可视化测试警告: {e}")
        
        test_results['visualization'] = True
        print("   ✅ 可视化功能验证通过")
    except Exception as e:
        print(f"   ❌ 可视化功能验证失败: {e}")
        traceback.print_exc()
    
    print()
    print("🔧 环境配置验证完成!")
    print("=" * 60)
    
    # 输出测试总结
    passed_tests = sum(test_results.values())
    total_tests = len(test_results)
    
    print(f"📊 测试结果汇总: {passed_tests}/{total_tests} 项测试通过")
    print()
    
    for test_name, passed in test_results.items():
        status = "✅" if passed else "❌"
        print(f"   {status} {test_name.replace('_', ' ').title()}")
    
    print()
    
    if passed_tests == total_tests:
        print("🎉 所有环境配置验证通过！DANTE已准备好进行完整训练。")
        return True
    else:
        print(f"⚠️  有 {total_tests - passed_tests} 项测试未通过，请检查相关配置后再进行完整训练。")
        return False

# 执行环境验证
print("开始DANTE环境配置验证...")
validation_success = comprehensive_environment_test()

if validation_success:
    print("\n🚀 环境验证成功！可以开始完整的DANTE训练流程。")
else:
    print("\n⚠️  环境验证未完全通过，建议先解决问题后再进行训练。")


In [ ]:
# Cell 8: 主要DANTE训练执行逻辑
import logging
from datetime import datetime
import gc
from scipy.stats import pearsonr

# 配置日志记录器
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def calculate_pearson_correlation(model, data_manager):
    """计算模型在所有数据上的皮尔逊相关系数"""
    if data_manager.num_samples < 2:
        return 0.0
    
    try:
        X_data = data_manager.get_all_x_orig()
        y_data_orig = data_manager.get_all_y_orig().flatten()
        
        predictions_log = model.predict(X_data)
        predictions_orig = safe_power10(predictions_log)
        
        if len(predictions_orig) < 2 or len(y_data_orig) < 2:
            logger.warning("数据点不足，无法计算皮尔逊相关系数")
            return 0.0
            
        correlation, _ = pearsonr(predictions_orig, y_data_orig)
        return correlation if not np.isnan(correlation) else 0.0
    
    except Exception as e:
        logger.error(f"计算皮尔逊相关系数时出错: {e}")
        return 0.0

def calculate_pearson_on_best_samples(model, data_manager, n_best=60):
    """计算模型在最佳样本上的皮尔逊相关系数"""
    all_X = data_manager.get_all_x_orig()
    all_y_orig = data_manager.get_all_y_orig()
    
    if all_X.shape[0] < 2:
        return 0.0, float('nan'), float('nan')
    
    # 按y值排序（升序）取最佳样本
    sorted_indices = np.argsort(all_y_orig.flatten())
    n_best = min(n_best, len(sorted_indices))
    sorted_X = all_X[sorted_indices[:n_best]]
    sorted_y_orig = all_y_orig[sorted_indices[:n_best]]
    
    min_y_orig_for_range = np.min(sorted_y_orig)
    max_y_orig_for_range = np.max(sorted_y_orig)
    
    try:
        predictions_log = model.predict(sorted_X)
        predictions_orig = safe_power10(predictions_log)
        
        if np.any(np.isinf(predictions_orig)) or np.any(np.isnan(predictions_orig)):
            logger.warning("预测值中存在无穷大或NaN值")
            return 0.0, min_y_orig_for_range, max_y_orig_for_range
            
        true_values_orig = sorted_y_orig.flatten()
        
        if len(predictions_orig) < 2 or len(true_values_orig) < 2:
            return 0.0, min_y_orig_for_range, max_y_orig_for_range
        
        if np.var(predictions_orig) == 0 or np.var(true_values_orig) == 0:
            logger.warning("预测值或真实值没有变化，无法计算相关系数")
            return 0.0, min_y_orig_for_range, max_y_orig_for_range
            
        correlation, _ = pearsonr(predictions_orig, true_values_orig)
        if np.isnan(correlation) or np.isinf(correlation):
            logger.warning("Pearson相关系数计算结果为NaN或无穷大")
            return 0.0, min_y_orig_for_range, max_y_orig_for_range
            
        return (correlation if not np.isnan(correlation) else 0.0), min_y_orig_for_range, max_y_orig_for_range
        
    except Exception as e:
        logger.warning(f"计算Pearson相关系数时出错: {e}")
        return 0.0, min_y_orig_for_range, max_y_orig_for_range

def create_weighted_samples(num_total_samples, y_data_orig_for_weighting, 
                          nte_samples_X=None, nte_samples_y_orig=None, 
                          initial_lowest_indices=None, initial_highest_indices=None):
    """为数据集创建样本权重"""
    sample_weights = np.ones(num_total_samples)  # 默认权重为1

    if initial_lowest_indices is not None:
        sample_weights[initial_lowest_indices] = 2.0
        logger.info(f"为初始数据集中 {len(initial_lowest_indices)} 个最低y值样本设置权重为2.0")

    if initial_highest_indices is not None:
        sample_weights[initial_highest_indices] = 3.0
        logger.info(f"为初始数据集中 {len(initial_highest_indices)} 个最高y值样本设置权重为3.0")

    # 为NTE样本设置权重
    if nte_samples_X is not None and nte_samples_y_orig is not None:
        num_nte_samples = nte_samples_X.shape[0]
        start_index_for_nte = num_total_samples - num_nte_samples
        
        if num_nte_samples > 0:
            nte_specific_weights = np.ones(num_nte_samples) * 4.0
            sample_weights[start_index_for_nte:] = nte_specific_weights
            logger.info(f"为新加入的 {num_nte_samples} 个NTE样本设置权重为4.0")

    final_weights = torch.tensor(sample_weights, dtype=torch.float32)
    logger.info(f"最终样本权重统计: 最小值={final_weights.min().item():.1f}, 最大值={final_weights.max().item():.1f}, 平均值={final_weights.mean().item():.1f}")
    
    return final_weights


In [ ]:
# Cell 9: 主训练函数和启动脚本
def save_iteration_samples_to_csv(X_batch, y_batch, results_dir, iteration):
    """将每次迭代的样本保存到CSV文件"""
    try:
        csv_filename = os.path.join(results_dir, f"iteration_{iteration}_samples.csv")
        
        data_dict = {}
        for i in range(X_batch.shape[1]):
            data_dict[f'x{i+1}'] = X_batch[:, i]
        data_dict['y_true'] = y_batch.flatten()
        
        df = pd.DataFrame(data_dict)
        df.to_csv(csv_filename, index=False)
        
        logger.info(f"第{iteration}轮迭代的{len(y_batch)}个样本已保存到: {csv_filename}")
        
    except Exception as e:
        logger.error(f"保存样本到CSV失败: {e}")

def main_dante_training():
    """DANTE主训练流程"""
    logger.info("开始DANTE Rosenbrock函数优化...")
    
    # === 参数配置 ===
    ROSENBROCK_DIMENSION = 20
    domain_min, domain_max = -2.048, 2.048 
    ROSENBROCK_DOMAIN = [(domain_min, domain_max)] * ROSENBROCK_DIMENSION
    
    INITIAL_DATA_SIZE = 800
    N_AL_ITERATIONS = 100  # 在Kaggle上使用较少的迭代次数进行测试
    CNN_TRAINING_EPOCHS = 200
    CNN_EARLY_STOPPING_PATIENCE = 30
    CNN_VALIDATION_SPLIT = 0.2
    CNN_BATCH_SIZE = 32
    NTE_C0 = 1
    NTE_ROLLOUT_ROUNDS = 100
    VALIDATION_BATCH_SIZE = 20
    
    logger.info(f"DANTE配置:")
    logger.info(f"  维度: {ROSENBROCK_DIMENSION}")
    logger.info(f"  初始数据大小: {INITIAL_DATA_SIZE}")
    logger.info(f"  主动学习迭代次数: {N_AL_ITERATIONS}")
    logger.info(f"  CNN训练轮数: {CNN_TRAINING_EPOCHS}")
    
    # 检测GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"使用设备: {device}")
    
    # === 创建结果目录 ===
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = f"/kaggle/working/dante_results_{timestamp}"
    os.makedirs(results_dir, exist_ok=True)
    logger.info(f"结果将保存到: {results_dir}")
    
    # === 初始化历史记录变量 ===
    cnn_performance_history = []
    history_pearson_all = []
    history_pearson_best_samples = []
    history_iterations = []
    history_best_y = []
    history_nte_candidates_count = []
    
    no_improvement_streak = 0
    previous_best_y = float('inf')
    previous_iteration_samples_x = None
    
    viz_interval = 20  # 每20次迭代可视化一次
    logger.info(f"可视化间隔设置为: 每{viz_interval}次迭代输出一次结果")
    
    # === 1. 数据初始化 ===
    logger.info("生成初始数据...")
    initial_X = np.random.uniform(low=domain_min, high=domain_max, size=(INITIAL_DATA_SIZE, ROSENBROCK_DIMENSION))
    initial_y = np.array([rosenbrock_function(x) for x in initial_X]).reshape(-1, 1)
    logger.info("初始数据生成完成")
    
    # === 2. DataManager初始化 ===
    data_manager = DataManager(
        initial_X=initial_X, 
        initial_y=initial_y, 
        dimension=ROSENBROCK_DIMENSION,
        data_file_path=None
    )
    logger.info("DataManager初始化完成")
    
    # === 3. 初始样本权重设定 ===
    if data_manager.num_samples > 0:
        all_y_orig = data_manager.get_all_y_orig().flatten()
        sorted_indices = np.argsort(all_y_orig)
        
        lowest_20_indices = sorted_indices[:20] if len(sorted_indices) >= 20 else sorted_indices
        highest_20_indices = sorted_indices[-20:] if len(sorted_indices) >= 20 else np.array([])

        initial_weights = create_weighted_samples(
            num_total_samples=data_manager.num_samples,
            y_data_orig_for_weighting=all_y_orig,
            initial_lowest_indices=lowest_20_indices,
            initial_highest_indices=highest_20_indices
        )
        data_manager.last_batch_weights = initial_weights
    else:
        data_manager.last_batch_weights = torch.empty(0, dtype=torch.float32)
    
    # === 4. CNN模型初始化 ===
    cnn_model = CNN1DSurrogate(input_dim=ROSENBROCK_DIMENSION)
    logger.info("CNN1DSurrogate模型初始化完成")
    
    # === 5. 主动学习主循环 ===
    for al_iteration in range(N_AL_ITERATIONS):
        should_generate_plots = ((al_iteration + 1) % viz_interval == 0) or (al_iteration == N_AL_ITERATIONS - 1)
        current_iteration_number = al_iteration + 1
        history_iterations.append(current_iteration_number)
        
        logger.info(f"\n--- 训练CNN 1D模型 (迭代 {current_iteration_number}/{N_AL_ITERATIONS}) ---")
        
        # === 5a. 训练CNN模型 ===
        training_history = cnn_model.train(
            data_manager=data_manager,
            epochs=CNN_TRAINING_EPOCHS,
            batch_size=CNN_BATCH_SIZE,
            validation_split=CNN_VALIDATION_SPLIT,
            patience=CNN_EARLY_STOPPING_PATIENCE,
            verbose=0
        )
        
        # === 5b. 评估模型性能 ===
        if should_generate_plots:
            cnn_model.visualize_training_history(
                training_history,
                save_path=os.path.join(results_dir, f"cnn_1d_training_history_iter_{current_iteration_number}.png")
            )
        
        # 计算皮尔逊相关系数
        current_pearson = calculate_pearson_correlation(cnn_model, data_manager)
        history_pearson_all.append(current_pearson)
        logger.info(f"模型皮尔逊相关系数: {current_pearson:.4f}")
        
        current_pearson_best_samples, min_y_best, max_y_best = calculate_pearson_on_best_samples(cnn_model, data_manager)
        history_pearson_best_samples.append(current_pearson_best_samples)
        logger.info(f"最佳{min(60, data_manager.num_samples)}个样本皮尔逊相关系数: {current_pearson_best_samples:.4f}")
        
        # === 5c. NTE搜索新候选点 ===
        logger.info("使用NTE搜索新候选样本...")
        
        # NTE Surrogate Wrapper
        class NTESurrogateWrapper:
            def __init__(self, actual_surrogate_model, data_manager_instance):
                self.surrogate_model = actual_surrogate_model
                self.dm = data_manager_instance
                self.logger = logging.getLogger(self.__class__.__name__)

            def predict(self, X_batch_orig: np.ndarray) -> np.ndarray:
                if X_batch_orig.shape[0] == 0:
                    return np.array([])
                if X_batch_orig.ndim == 1:
                     X_batch_orig = X_batch_orig.reshape(1, -1)

                try:
                    pred_log = self.surrogate_model.predict(X_batch_orig)
                    
                    if X_batch_orig.shape[0] == 1:
                        return pred_log[0] if isinstance(pred_log, np.ndarray) else pred_log
                    else:
                        return pred_log
                except Exception as e:
                    self.logger.error(f"NTESurrogateWrapper预测错误: {e}")
                    
                    if X_batch_orig.shape[0] == 1:
                        return np.log10(1e6 + 1.0)
                    else:
                        return np.ones(X_batch_orig.shape[0]) * np.log10(1e6 + 1.0)
        
        nte_model_wrapper = NTESurrogateWrapper(cnn_model, data_manager)
        
        # 初始化NTE搜索器
        nte_searcher = NTESearcher(
            dimension=ROSENBROCK_DIMENSION, 
            domain=[(domain_min, domain_max) for _ in range(ROSENBROCK_DIMENSION)],
            c0=NTE_C0,
            rollout_rounds=NTE_ROLLOUT_ROUNDS,
            validation_batch_size=VALIDATION_BATCH_SIZE
        )
        
        current_best_x_orig, current_best_y_overall = data_manager.get_current_best()
        
        # 🔧 关键调试参数：计算动态探索因子
        dynamic_exploration_factor = 0.8 + 1 * min(20, 2 ** no_improvement_streak)
        logger.info(f"🔧 使用动态探索因子: {dynamic_exploration_factor:.2f} (基于连续无改进次数: {no_improvement_streak})")
        
        try:
            logger.info("=" * 50)
            logger.info(f"开始第 {current_iteration_number} 轮NTE搜索...")
            logger.info("=" * 50)
            
            # 执行NTE搜索
            nte_search_result = nte_searcher.search(
                surrogate_model=nte_model_wrapper,
                current_best_state=current_best_x_orig if current_best_x_orig is not None else None,
                min_y_observed=current_best_y_overall,
                dynamic_exploration_factor=dynamic_exploration_factor,
                all_states=data_manager.X_data if data_manager.num_samples > 0 else None,
                all_values=data_manager.get_all_y_orig() if data_manager.num_samples > 0 else None,
                previous_iteration_samples=previous_iteration_samples_x,
                current_iteration=al_iteration,
                return_ducb_trends=True
            )
            
            nte_selected_samples_x, ducb_trends_data = nte_search_result
            
            if isinstance(nte_selected_samples_x, list):
                nte_selected_samples_x = np.array(nte_selected_samples_x)
            
            logger.info(f"NTE搜索完成，返回 {len(nte_selected_samples_x)} 个候选点")
            
            if nte_selected_samples_x.shape[0] > 0:
                logger.info(f"评估 {nte_selected_samples_x.shape[0]} 个新候选样本...")
                nte_selected_samples_y_true = np.array([rosenbrock_function(x) for x in nte_selected_samples_x]).reshape(-1, 1)
                
                num_samples_before_adding = data_manager.num_samples
                data_manager.add_samples(nte_selected_samples_x, nte_selected_samples_y_true, re_normalize=True, shuffle=True)
                logger.info(f"添加了 {nte_selected_samples_x.shape[0]} 个新样本。DataManager现在有 {data_manager.num_samples} 个样本。")
                
                # 更新样本权重
                if data_manager.num_samples > num_samples_before_adding:
                    new_batch_weights = np.ones(data_manager.num_samples) 
                    if data_manager.last_batch_weights is not None and len(data_manager.last_batch_weights) == num_samples_before_adding:
                        new_batch_weights[:num_samples_before_adding] = data_manager.last_batch_weights.numpy() 
                    
                    new_nte_sample_weights = np.ones(nte_selected_samples_x.shape[0]) * 4.0
                    new_batch_weights[num_samples_before_adding:] = new_nte_sample_weights
                    data_manager.last_batch_weights = torch.tensor(new_batch_weights, dtype=torch.float32)
                
                # 保存迭代结果
                if should_generate_plots:
                    save_iteration_samples_to_csv(nte_selected_samples_x, nte_selected_samples_y_true, results_dir, current_iteration_number)
                
                previous_iteration_samples_x = nte_selected_samples_x.copy()
                
                # 更新最佳值
                current_best_x_orig, current_best_y_overall = data_manager.get_current_best()
                history_best_y.append(current_best_y_overall)
                logger.info(f"更新后全局最小函数值: {current_best_y_overall:.6e} @ 迭代 {current_iteration_number}")
                
                history_nte_candidates_count.append(nte_selected_samples_x.shape[0])
                
                # 检查是否有改进
                new_global_best_found = current_best_y_overall < previous_best_y
                if new_global_best_found:
                    logger.info(f"🎉 发现新的全局最优解: {current_best_y_overall:.6e}")
                    
            else:
                logger.info("NTE未返回有效候选点")
                history_nte_candidates_count.append(0)
                if history_best_y:
                    history_best_y.append(history_best_y[-1])
                else:
                    history_best_y.append(current_best_y_overall)
                    
        except Exception as e:
            logger.error(f"NTE搜索过程中出错: {e}")
            import traceback
            traceback.print_exc()
            nte_selected_samples_x = np.array([])
        
        # === 5d. 更新改进统计 ===
        iteration_improved = False
        if 'new_global_best_found' in locals() and new_global_best_found:
            iteration_improved = True
        elif len(nte_selected_samples_x) == 0:
            iteration_improved = False
        else:
            iteration_improved = current_best_y_overall < previous_best_y
            
        if iteration_improved:
            no_improvement_streak = 0
            logger.info(f"本次迭代有改进，重置 no_improvement_streak = 0")
        else:
            no_improvement_streak += 1
            logger.info(f"本次迭代无改进，no_improvement_streak 增加到 {no_improvement_streak}")
            
        previous_best_y = current_best_y_overall
        
        # === 5e. 生成可视化图表 ===
        if should_generate_plots:
            logger.info(f"在迭代 {current_iteration_number} 生成可视化图表...")
            
            # 全局最小值趋势图
            if history_best_y:
                try:
                    plot_global_min_value_trend(history_iterations[:len(history_best_y)], history_best_y, results_dir, 
                                              filename=f"global_min_iter_{current_iteration_number}.png")
                except Exception as e:
                    logger.error(f"绘制全局最小值趋势图失败: {e}")
            
            # 皮尔逊相关系数趋势图
            if history_pearson_all:
                try:
                    plot_pearson_correlation_trend(history_iterations, history_pearson_all, results_dir,
                                                 filename=f"pearson_corr_iter_{current_iteration_number}.png",
                                                 title="Pearson Correlation Trend")
                except Exception as e:
                    logger.error(f"绘制皮尔逊相关系数趋势图失败: {e}")
            
            # 最佳样本皮尔逊相关系数趋势图
            if history_pearson_best_samples:
                try:
                    plot_best_samples_pearson_trend(history_iterations, history_pearson_best_samples, results_dir,
                                                  filename=f"best_samples_pearson_iter_{current_iteration_number}.png")
                except Exception as e:
                    logger.error(f"绘制最佳样本皮尔逊相关系数趋势图失败: {e}")
            
            # NTE性能图
            if history_nte_candidates_count:
                try:
                    plot_nte_performance(history_iterations, history_nte_candidates_count, results_dir,
                                       filename=f"nte_perf_iter_{current_iteration_number}.png")
                except Exception as e:
                    logger.error(f"绘制NTE性能图失败: {e}")
        
        # === 5f. 内存清理 ===
        if current_iteration_number % 10 == 0:
            data_manager.clear_cache()
            gc.collect()
            logger.info(f"第{current_iteration_number}轮迭代: 已清理缓存")
    
    # === 6. 生成最终结果 ===
    logger.info("生成最终图表...")
    
    # 最终全局最小值趋势图
    if history_best_y:
        try:
            plot_global_min_value_trend(history_iterations[:len(history_best_y)], history_best_y, results_dir, 
                                      filename="global_min_value_trend_final.png")
        except Exception as e:
            logger.error(f"绘制最终全局最小值趋势图失败: {e}")
    
    # 保存最终DataManager状态
    final_data_path = os.path.join(results_dir, "final_data_manager_state.npz")
    data_manager.save_data(final_data_path)
    logger.info(f"最终DataManager状态已保存到 {final_data_path}")
    
    # 保存性能历史
    try:
        perf_data = {
            'iterations': history_iterations,
            'best_y_values': history_best_y,
            'pearson_all': history_pearson_all,
            'pearson_best_samples': history_pearson_best_samples,
            'nte_candidates_count': history_nte_candidates_count
        }
        
        # 确保所有列表长度一致
        min_length = min(len(v) if isinstance(v, list) else 0 for v in perf_data.values())
        for k, v in perf_data.items():
            if isinstance(v, list) and len(v) > min_length:
                perf_data[k] = v[:min_length]
        
        perf_csv_path = os.path.join(results_dir, "performance_history.csv")
        pd.DataFrame(perf_data).to_csv(perf_csv_path, index=False)
        logger.info(f"性能历史已保存至: {perf_csv_path}")
    except Exception as e:
        logger.error(f"保存性能历史失败: {e}")
    
    logger.info(f"🎉 DANTE训练完成！")
    logger.info(f"最终最佳解: y = {current_best_y_overall:.4e}")
    logger.info(f"所有结果已保存在: {results_dir}")
    
    return results_dir, current_best_y_overall

# 提供训练启动选项
def start_dante_training():
    """启动DANTE训练的便捷函数"""
    print("🚀 准备启动DANTE训练...")
    print("⚠️  注意：完整训练可能需要几个小时时间")
    print("📊 建议先运行环境验证确保一切正常")
    
    return main_dante_training()

# 显示使用说明
print("📋 DANTE训练使用说明:")
print("1. 首先运行 Cell 7 进行环境验证")
print("2. 环境验证通过后，运行以下代码开始训练:")
print("   results_dir, best_value = start_dante_training()")
print("3. 训练过程中的所有结果将保存到 /kaggle/working/dante_results_[timestamp]/ 目录")
print("4. 可以通过修改 main_dante_training() 函数中的参数来调整训练配置")
print("\n🔧 关键调试参数:")
print("   - dynamic_exploration_factor: 0.8 + 1 * min(20, 2 ** no_improvement_streak)")
print("   - N_AL_ITERATIONS: 当前设置为100次迭代（可根据需要调整）")
print("   - NTE_ROLLOUT_ROUNDS: NTE内部搜索轮次")
print("   - VALIDATION_BATCH_SIZE: 每次迭代选择的样本数量")
print("\n✅ 准备就绪！运行环境验证后即可开始训练。")


In [ ]:
# Cell 7: 环境配置验证与功能测试
print("="*60)
print("DANTE Kaggle环境配置验证与功能测试")
print("="*60)

# 1. 系统环境检查
print("\n1. 系统环境检查:")
print(f"   Python版本: {sys.version}")
print(f"   TensorFlow版本: {tf.__version__}")
print(f"   PyTorch版本: {torch.__version__}")
print(f"   Numpy版本: {np.__version__}")
print(f"   Pandas版本: {pd.__version__}")

# 2. GPU/计算资源检查
print("\n2. 计算资源检查:")
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"   TensorFlow检测到 {len(gpus)} 个GPU:")
        for i, gpu in enumerate(gpus):
            print(f"     GPU {i}: {gpu}")
            # 配置GPU内存增长
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        print("   未检测到GPU，将使用CPU训练")
    
    print(f"   PyTorch CUDA可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   PyTorch GPU设备: {torch.cuda.get_device_name(0)}")
        print(f"   CUDA版本: {torch.version.cuda}")
    
except Exception as e:
    print(f"   GPU检查时出错: {e}")

# 3. 数据路径检查
print("\n3. 数据路径检查:")
input_paths = [
    "/kaggle/input/rosenbrock-dante-data",
    "/kaggle/input/rosenbrock-dante-data/Rosenbrock_x_train.npy",
    "/kaggle/input/rosenbrock-dante-data/Rosenbrock_y_train.npy",
    "/kaggle/input/rosenbrock-dante-data/Rosenbrock_x_test.npy", 
    "/kaggle/input/rosenbrock-dante-data/Rosenbrock_y_test.npy"
]

for path in input_paths:
    exists = os.path.exists(path)
    print(f"   {path}: {'✓存在' if exists else '✗缺失'}")
    
# 4. 核心组件实例化测试
print("\n4. 核心组件实例化测试:")

try:
    # 测试Rosenbrock函数
    test_x = np.array([1.0] * 20)  # 全局最优解
    test_result = rosenbrock_function(test_x)
    print(f"   Rosenbrock函数测试: f([1]*20) = {test_result:.2e} {'✓正确' if abs(test_result) < 1e-10 else '✗错误'}")
    
    # 生成测试数据
    print("   生成测试数据集...")
    test_X = np.random.uniform(-2.048, 2.048, size=(100, 20))
    test_y = np.array([rosenbrock_function(x) for x in test_X]).reshape(-1, 1)
    print(f"   测试数据: X.shape={test_X.shape}, y.shape={test_y.shape}")
    print(f"   y值范围: [{test_y.min():.2e}, {test_y.max():.2e}]")
    
    # 测试DataManager
    print("   测试DataManager...")
    test_dm = DataManager(test_X[:50], test_y[:50], dimension=20)
    print(f"   DataManager初始化成功: {test_dm.num_samples}个样本")
    
    # 测试性能优化器
    print("   测试性能优化器...")
    test_optimizer = PerformanceOptimizer()
    test_optimizer.configure_tensorflow_performance()
    print("   性能优化器配置完成 ✓")
    
    # 测试CNN1DSurrogate
    print("   测试CNN1DSurrogate...")
    test_cnn = CNN1DSurrogate(input_dim=20)
    print("   CNN模型初始化成功 ✓")
    
    # 快速训练测试（1个epoch）
    print("   快速训练测试（1个epoch）...")
    training_history = test_cnn.train(
        data_manager=test_dm,
        epochs=1,
        batch_size=16,
        validation_split=0.2,
        patience=10,
        verbose=0
    )
    print("   快速训练完成 ✓")
    
    # 测试预测
    test_pred = test_cnn.predict(test_X[:5])
    print(f"   预测测试: 输入5个样本，输出shape={test_pred.shape}")
    print(f"   预测值范围: [{test_pred.min():.2e}, {test_pred.max():.2e}]")
    
    # 测试NTE搜索器初始化
    print("   测试NTE搜索器...")
    test_nte = NTESearcher(
        dimension=20,
        domain=[(-2.048, 2.048)] * 20,
        c0=1.0,
        rollout_rounds=10,  # 减少轮数用于测试
        validation_batch_size=5
    )
    print("   NTE搜索器初始化成功 ✓")
    
    print("\n✓ 所有核心组件测试通过！")
    
except Exception as e:
    print(f"✗ 组件测试失败: {e}")
    import traceback
    traceback.print_exc()

# 5. 内存和存储检查
print("\n5. 资源检查:")
try:
    # 检查可用内存
    import psutil
    memory = psutil.virtual_memory()
    print(f"   可用内存: {memory.available / (1024**3):.1f} GB / {memory.total / (1024**3):.1f} GB")
    
    # 检查磁盘空间
    disk = psutil.disk_usage('/kaggle/working')
    print(f"   可用磁盘: {disk.free / (1024**3):.1f} GB / {disk.total / (1024**3):.1f} GB")
    
    # 检查CPU
    print(f"   CPU核心数: {psutil.cpu_count()}")
    
except Exception as e:
    print(f"   资源检查出错: {e}")

# 6. 生成配置摘要
print("\n6. 运行配置摘要:")
print("   DANTE优化配置:")
print("   - 目标函数: 20维Rosenbrock函数")
print("   - 初始样本数: 800")
print("   - 主动学习迭代数: 400")  
print("   - 每次迭代批量大小: 20")
print("   - CNN训练epochs: 200")
print("   - NTE rollout轮数: 100")
print("   - 动态探索因子: 0.8 + 1 * min(20, 2^no_improvement_streak)")

print("\n" + "="*60)
print("环境验证完成！准备开始DANTE训练...")
print("="*60)


In [ ]:
# Cell 8: DANTE主训练循环
def run_dante_optimization():
    """运行DANTE优化主循环"""
    print("开始DANTE优化...")
    
    # ===== 配置参数 =====
    ROSENBROCK_DIMENSION = 20
    DOMAIN_MIN, DOMAIN_MAX = -2.048, 2.048
    ROSENBROCK_DOMAIN = [(DOMAIN_MIN, DOMAIN_MAX)] * ROSENBROCK_DIMENSION
    
    # 训练参数 - 可调整用于调试动态探索因子
    INITIAL_DATA_SIZE = 800
    N_AL_ITERATIONS = 400
    CNN_TRAINING_EPOCHS = 200
    CNN_EARLY_STOPPING_PATIENCE = 30
    CNN_VALIDATION_SPLIT = 0.2
    CNN_BATCH_SIZE = 32
    NTE_C0 = 1
    NTE_ROLLOUT_ROUNDS = 100
    VALIDATION_BATCH_SIZE = 20
    
    # 可视化间隔（减少输出量）
    viz_interval = 10
    
    print(f"配置参数:")
    print(f"- 维度: {ROSENBROCK_DIMENSION}D")
    print(f"- 初始样本: {INITIAL_DATA_SIZE}")
    print(f"- AL迭代: {N_AL_ITERATIONS}")
    print(f"- 每批样本: {VALIDATION_BATCH_SIZE}")
    print(f"- CNN训练epochs: {CNN_TRAINING_EPOCHS}")
    print(f"- NTE rollout轮数: {NTE_ROLLOUT_ROUNDS}")
    
    # ===== 加载数据 =====
    print("\n加载数据...")
    try:
        # 尝试从Kaggle数据集加载
        X_train_path = "/kaggle/input/rosenbrock-dante-data/Rosenbrock_x_train.npy"
        y_train_path = "/kaggle/input/rosenbrock-dante-data/Rosenbrock_y_train.npy"
        X_test_path = "/kaggle/input/rosenbrock-dante-data/Rosenbrock_x_test.npy"
        y_test_path = "/kaggle/input/rosenbrock-dante-data/Rosenbrock_y_test.npy"
        
        if os.path.exists(X_train_path) and os.path.exists(y_train_path):
            initial_X = np.load(X_train_path)
            initial_y = np.load(y_train_path)
            if initial_y.ndim == 1:
                initial_y = initial_y.reshape(-1, 1)
            print(f"从Kaggle数据集加载训练数据: {initial_X.shape}")
        else:
            raise FileNotFoundError("训练数据不存在")
            
        if os.path.exists(X_test_path) and os.path.exists(y_test_path):
            X_test = np.load(X_test_path)
            y_test = np.load(y_test_path)
            if y_test.ndim == 1:
                y_test = y_test.reshape(-1, 1)
            print(f"从Kaggle数据集加载测试数据: {X_test.shape}")
        else:
            X_test, y_test = None, None
            print("测试数据不存在，将使用训练数据的一部分")
            
    except Exception as e:
        print(f"数据加载失败: {e}")
        print("生成随机数据...")
        initial_X = np.random.uniform(DOMAIN_MIN, DOMAIN_MAX, size=(INITIAL_DATA_SIZE, ROSENBROCK_DIMENSION))
        initial_y = np.array([rosenbrock_function(x) for x in initial_X]).reshape(-1, 1)
        
        # 生成测试集
        test_size = 200
        X_test = np.random.uniform(DOMAIN_MIN, DOMAIN_MAX, size=(test_size, ROSENBROCK_DIMENSION))
        y_test = np.array([rosenbrock_function(x) for x in X_test]).reshape(-1, 1)
        print(f"生成随机数据: 训练{initial_X.shape}, 测试{X_test.shape}")
    
    # 确保有足够的初始样本
    if initial_X.shape[0] < INITIAL_DATA_SIZE:
        additional_needed = INITIAL_DATA_SIZE - initial_X.shape[0]
        additional_X = np.random.uniform(DOMAIN_MIN, DOMAIN_MAX, size=(additional_needed, ROSENBROCK_DIMENSION))
        additional_y = np.array([rosenbrock_function(x) for x in additional_X]).reshape(-1, 1)
        initial_X = np.vstack((initial_X, additional_X))
        initial_y = np.vstack((initial_y, additional_y))
        print(f"补充{additional_needed}个随机样本，总计{initial_X.shape[0]}个")
    
    # ===== 初始化核心组件 =====
    print("\n初始化组件...")
    
    # 数据管理器
    data_manager = DataManager(initial_X, initial_y, dimension=ROSENBROCK_DIMENSION)
    print(f"DataManager初始化: {data_manager.num_samples}个样本")
    
    # 性能优化器
    performance_optimizer = PerformanceOptimizer()
    performance_optimizer.configure_tensorflow_performance()
    
    # CNN替代模型
    cnn_model = CNN1DSurrogate(input_dim=ROSENBROCK_DIMENSION)
    print("CNN1DSurrogate模型初始化完成")
    
    # NTE搜索器
    nte_searcher = NTESearcher(
        dimension=ROSENBROCK_DIMENSION,
        domain=ROSENBROCK_DOMAIN,
        c0=NTE_C0,
        rollout_rounds=NTE_ROLLOUT_ROUNDS,
        validation_batch_size=VALIDATION_BATCH_SIZE
    )
    print("NTESearcher初始化完成")
    
    # ===== 历史记录变量 =====
    history_iterations = []
    history_best_y = []
    history_pearson_all = []
    history_pearson_best_samples = []
    history_nte_candidates_count = []
    cnn_performance_history = []
    
    # 优化相关变量
    no_improvement_streak = 0
    previous_best_y = float('inf')
    previous_iteration_samples_x = None
    
    # 获取初始最优值
    current_best_x_orig, current_best_y_overall = data_manager.get_current_best()
    print(f"初始最优值: {current_best_y_overall:.6e}")
    
    # ===== 主训练循环 =====
    print(f"\n开始{N_AL_ITERATIONS}次主动学习迭代...")
    print("="*60)
    
    for al_iteration in range(N_AL_ITERATIONS):
        current_iteration_number = al_iteration + 1
        history_iterations.append(current_iteration_number)
        
        # 决定是否生成详细可视化
        should_generate_plots = ((current_iteration_number % viz_interval == 0) or 
                                (current_iteration_number == N_AL_ITERATIONS))
        
        if current_iteration_number % 50 == 0 or current_iteration_number <= 5:
            print(f"\n--- 迭代 {current_iteration_number}/{N_AL_ITERATIONS} ---")
        
        # === 1. 训练CNN替代模型 ===
        training_history = cnn_model.train(
            data_manager=data_manager,
            epochs=CNN_TRAINING_EPOCHS,
            batch_size=CNN_BATCH_SIZE,
            validation_split=CNN_VALIDATION_SPLIT,
            patience=CNN_EARLY_STOPPING_PATIENCE,
            verbose=0
        )
        
        # === 2. 评估模型性能 ===
        if X_test is not None:
            current_pearson = calculate_pearson_on_test_set(cnn_model, X_test, y_test, data_manager)
        else:
            current_pearson = calculate_pearson_correlation(cnn_model, data_manager)
        
        history_pearson_all.append(current_pearson)
        
        # 计算最佳样本上的皮尔逊相关系数
        current_pearson_best_samples, _, _ = calculate_pearson_on_best_samples(cnn_model, data_manager)
        history_pearson_best_samples.append(current_pearson_best_samples)
        
        # === 3. NTE搜索新候选点 ===
        # 计算动态探索因子（关键调试参数）
        dynamic_exploration_factor = 0.8 + 1 * min(20, 2 ** no_improvement_streak)
        
        if current_iteration_number % 50 == 0 or current_iteration_number <= 5:
            print(f"动态探索因子: {dynamic_exploration_factor:.2f} (无改进连续次数: {no_improvement_streak})")
        
        # NTE搜索包装器
        class NTESurrogateWrapper:
            def __init__(self, model, dm):
                self.model = model
                self.dm = dm
            
            def predict(self, X_batch):
                if X_batch.shape[0] == 0:
                    return np.array([])
                if X_batch.ndim == 1:
                    X_batch = X_batch.reshape(1, -1)
                
                try:
                    pred_log = self.model.predict(X_batch)
                    return pred_log[0] if X_batch.shape[0] == 1 else pred_log
                except Exception as e:
                    print(f"预测错误: {e}")
                    return np.log10(1e6 + 1.0) if X_batch.shape[0] == 1 else np.ones(X_batch.shape[0]) * np.log10(1e6 + 1.0)
        
        nte_model_wrapper = NTESurrogateWrapper(cnn_model, data_manager)
        
        try:
            # 执行NTE搜索
            nte_search_result = nte_searcher.search(
                surrogate_model=nte_model_wrapper,
                current_best_state=current_best_x_orig,
                min_y_observed=current_best_y_overall,
                dynamic_exploration_factor=dynamic_exploration_factor,
                all_states=data_manager.X_data,
                all_values=data_manager.get_all_y_orig(),
                previous_iteration_samples=previous_iteration_samples_x,
                current_iteration=al_iteration,
                return_ducb_trends=True
            )
            
            nte_selected_samples_x, ducb_trends_data = nte_search_result
            
            if isinstance(nte_selected_samples_x, list):
                nte_selected_samples_x = np.array(nte_selected_samples_x)
            
            if nte_selected_samples_x.shape[0] > 0:
                # 评估新样本
                nte_selected_samples_y_true = np.array([rosenbrock_function(x) for x in nte_selected_samples_x]).reshape(-1, 1)
                
                # 添加到数据管理器
                data_manager.add_samples(nte_selected_samples_x, nte_selected_samples_y_true, re_normalize=True, shuffle=True)
                
                # 更新权重（为新样本设置较高权重）
                num_new_samples = nte_selected_samples_x.shape[0]
                new_weights = np.ones(data_manager.num_samples)
                new_weights[-num_new_samples:] = 4.0  # 新样本权重为4.0
                data_manager.last_batch_weights = torch.tensor(new_weights, dtype=torch.float32)
                
                history_nte_candidates_count.append(num_new_samples)
                previous_iteration_samples_x = nte_selected_samples_x.copy()
                
                # 更新最优值
                current_best_x_orig, current_best_y_overall = data_manager.get_current_best()
                history_best_y.append(current_best_y_overall)
                
                # 检查是否有改进
                iteration_improved = current_best_y_overall < previous_best_y
                if iteration_improved:
                    no_improvement_streak = 0
                    if current_iteration_number % 10 == 0 or current_iteration_number <= 10:
                        print(f"发现更优解: {current_best_y_overall:.6e}")
                else:
                    no_improvement_streak += 1
                
                previous_best_y = current_best_y_overall
                
            else:
                history_nte_candidates_count.append(0)
                history_best_y.append(history_best_y[-1] if history_best_y else current_best_y_overall)
                no_improvement_streak += 1
                
        except Exception as e:
            print(f"NTE搜索失败: {e}")
            history_nte_candidates_count.append(0)
            history_best_y.append(history_best_y[-1] if history_best_y else current_best_y_overall)
            no_improvement_streak += 1
        
        # === 4. 性能记录和可视化 ===
        perf_metrics = {
            'iteration': current_iteration_number,
            'best_y': current_best_y_overall,
            'pearson_test': current_pearson,
            'pearson_best_samples': current_pearson_best_samples,
            'num_samples': data_manager.num_samples,
            'no_improvement_streak': no_improvement_streak,
            'dynamic_exploration_factor': dynamic_exploration_factor
        }
        cnn_performance_history.append(perf_metrics)
        
        # 定期可视化
        if should_generate_plots:
            print(f"生成第{current_iteration_number}轮可视化图表...")
            
            # 全局最小值趋势
            plot_global_min_value_trend(
                history_iterations[:len(history_best_y)], 
                history_best_y, 
                "/kaggle/working", 
                filename=f"global_min_iter_{current_iteration_number}.png"
            )
            
            # 皮尔逊相关系数趋势
            plot_pearson_correlation_trend(
                history_iterations, 
                history_pearson_all, 
                "/kaggle/working",
                filename=f"pearson_trend_iter_{current_iteration_number}.png",
                title="Test Set Pearson Correlation"
            )
            
            # 最佳样本皮尔逊趋势
            plot_best_samples_pearson_trend(
                history_iterations, 
                history_pearson_best_samples, 
                "/kaggle/working",
                filename=f"best_samples_pearson_iter_{current_iteration_number}.png"
            )
            
            # NTE性能
            plot_nte_performance(
                history_iterations, 
                history_nte_candidates_count, 
                "/kaggle/working",
                filename=f"nte_performance_iter_{current_iteration_number}.png"
            )
        
        # 内存清理
        if current_iteration_number % 10 == 0:
            data_manager.clear_cache()
            performance_optimizer.clear_memory_cache()
            import gc
            gc.collect()
        
        # 进度报告
        if current_iteration_number % 50 == 0:
            print(f"进度: {current_iteration_number}/{N_AL_ITERATIONS} ({100*current_iteration_number/N_AL_ITERATIONS:.1f}%)")
            print(f"当前最优: {current_best_y_overall:.6e}")
            print(f"样本总数: {data_manager.num_samples}")
            print(f"无改进连续次数: {no_improvement_streak}")
    
    # ===== 训练完成 =====
    print("\n" + "="*60)
    print("DANTE训练完成!")
    print(f"最终最优值: {current_best_y_overall:.6e}")
    print(f"最优解位置: {current_best_x_orig}")
    print(f"总样本数: {data_manager.num_samples}")
    print(f"训练迭代数: {N_AL_ITERATIONS}")
    
    # 保存最终结果
    results = {
        'final_best_value': current_best_y_overall,
        'final_best_x': current_best_x_orig,
        'total_samples': data_manager.num_samples,
        'iterations': N_AL_ITERATIONS,
        'history_best_y': history_best_y,
        'history_pearson': history_pearson_all,
        'cnn_performance_history': cnn_performance_history
    }
    
    # 保存为numpy文件
    np.savez("/kaggle/working/dante_results.npz", **results)
    print("结果已保存到 /kaggle/working/dante_results.npz")
    
    # 生成最终可视化
    print("\n生成最终可视化图表...")
    plot_global_min_value_trend(
        history_iterations[:len(history_best_y)], 
        history_best_y, 
        "/kaggle/working", 
        filename="final_global_min_trend.png"
    )
    
    plot_pearson_correlation_trend(
        history_iterations, 
        history_pearson_all, 
        "/kaggle/working",
        filename="final_pearson_trend.png",
        title="Final Pearson Correlation Trend"
    )
    
    return results

# 定义辅助函数
def calculate_pearson_on_test_set(model, X_test, y_test_orig, data_manager):
    """计算测试集上的皮尔逊相关系数"""
    if X_test.shape[0] < 2:
        return 0.0
    
    try:
        predictions_log = model.predict(X_test)
        predictions_orig = safe_power10(predictions_log)
        
        if len(predictions_orig) < 2:
            return 0.0
            
        from scipy.stats import pearsonr
        correlation, _ = pearsonr(predictions_orig, y_test_orig.flatten())
        return correlation if not np.isnan(correlation) else 0.0
    except Exception as e:
        print(f"计算测试集皮尔逊相关系数失败: {e}")
        return 0.0

def calculate_pearson_on_best_samples(model, data_manager, n_best=60):
    """计算最佳样本上的皮尔逊相关系数"""
    all_X = data_manager.get_all_x_orig()
    all_y_orig = data_manager.get_all_y_orig()
    
    if all_X.shape[0] < 2:
        return 0.0, float('nan'), float('nan')
    
    # 按y值排序，取最小的n_best个
    sorted_indices = np.argsort(all_y_orig.flatten())
    best_indices = sorted_indices[:n_best]
    
    sorted_X = all_X[best_indices]
    sorted_y_orig = all_y_orig[best_indices]
    
    if sorted_X.shape[0] < 2:
        return 0.0, float('nan'), float('nan')
    
    min_y = np.min(sorted_y_orig)
    max_y = np.max(sorted_y_orig)
    
    try:
        predictions_log = model.predict(sorted_X)
        predictions_orig = safe_power10(predictions_log)
        
        if np.any(np.isinf(predictions_orig)) or np.any(np.isnan(predictions_orig)):
            return 0.0, min_y, max_y
        
        from scipy.stats import pearsonr
        correlation, _ = pearsonr(predictions_orig, sorted_y_orig.flatten())
        return (correlation if not np.isnan(correlation) else 0.0), min_y, max_y
        
    except Exception as e:
        return 0.0, min_y, max_y

# 运行说明
print("DANTE训练循环已定义完成!")
print("\n要开始训练，请运行:")
print("results = run_dante_optimization()")
print("\n注意: 完整训练可能需要8-12小时，建议在P100 GPU环境下运行")


In [ ]:
# Cell 10: 环境验证与完整性检查
print("="*80)
print("DANTE-Kaggle 环境验证与完整性检查")
print("="*80)

# 1. 硬件环境验证
print("\n1. 硬件环境验证")
print("-" * 40)

# GPU检查
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU可用: {gpu_name}")
    print(f"   显存: {gpu_memory:.1f} GB")
    
    # 测试GPU内存分配
    try:
        test_tensor = torch.ones(1000, 1000).cuda()
        del test_tensor
        torch.cuda.empty_cache()
        print("✅ GPU内存分配测试通过")
    except Exception as e:
        print(f"❌ GPU内存分配失败: {e}")
else:
    print("❌ GPU不可用，将使用CPU")

# TensorFlow GPU检查
tf_gpus = tf.config.experimental.list_physical_devices('GPU')
if tf_gpus:
    print(f"✅ TensorFlow GPU可用: {len(tf_gpus)}个设备")
else:
    print("❌ TensorFlow未检测到GPU")

# 2. 数据验证
print("\n2. 数据完整性验证")
print("-" * 40)

data_status = {"train_x": False, "train_y": False, "test_x": False, "test_y": False}
data_shapes = {}

for key, filename in [("train_x", "Rosenbrock_x_train.npy"), 
                      ("train_y", "Rosenbrock_y_train.npy"),
                      ("test_x", "Rosenbrock_x_test.npy"), 
                      ("test_y", "Rosenbrock_y_test.npy")]:
    filepath = os.path.join(DATA_PATH, filename)
    if os.path.exists(filepath):
        try:
            data = np.load(filepath)
            data_status[key] = True
            data_shapes[key] = data.shape
            print(f"✅ {filename}: {data.shape}")
        except Exception as e:
            print(f"❌ {filename}: 加载失败 - {e}")
    else:
        print(f"❌ {filename}: 文件不存在")

# 验证数据维度一致性
if data_status["train_x"] and data_status["train_y"]:
    if data_shapes["train_x"][0] == data_shapes["train_y"][0]:
        print("✅ 训练数据X,Y样本数一致")
    else:
        print(f"❌ 训练数据维度不匹配: X={data_shapes['train_x']}, Y={data_shapes['train_y']}")

# 3. 核心功能验证
print("\n3. 核心功能模块验证")
print("-" * 40)

# 测试Rosenbrock函数
try:
    test_x = np.random.uniform(-2, 2, 20)
    result = rosenbrock_function(test_x)
    print(f"✅ Rosenbrock函数: 输入维度{test_x.shape} → 输出{result:.4e}")
    
    # 批量测试
    batch_x = np.random.uniform(-2, 2, (5, 20))
    batch_result = rosenbrock_function(batch_x)
    print(f"✅ Rosenbrock批量函数: 输入{batch_x.shape} → 输出{batch_result.shape}")
except Exception as e:
    print(f"❌ Rosenbrock函数测试失败: {e}")

# 测试safe_power10函数
try:
    test_log = np.array([1.0, 2.0, 3.0])
    result = safe_power10(test_log)
    print(f"✅ safe_power10函数: 输入{test_log} → 输出{result}")
except Exception as e:
    print(f"❌ safe_power10函数测试失败: {e}")

# 4. 内存状态检查
print("\n4. 系统资源检查")
print("-" * 40)

# 内存检查
try:
    memory_info = psutil.virtual_memory()
    available_gb = memory_info.available / 1e9
    total_gb = memory_info.total / 1e9
    print(f"✅ 系统内存: {available_gb:.1f}GB 可用 / {total_gb:.1f}GB 总计")
    
    if available_gb < 8:
        print("⚠️  可用内存不足8GB，可能影响训练性能")
    else:
        print("✅ 内存充足，可支持完整训练")
except Exception as e:
    print(f"❌ 内存检查失败: {e}")

# 5. 模块导入验证
print("\n5. 关键模块验证")
print("-" * 40)

modules_to_test = [
    ("numpy", np),
    ("pandas", pd), 
    ("torch", torch),
    ("tensorflow", tf),
    ("matplotlib", plt),
    ("scipy.stats", pearsonr),
]

for name, module in modules_to_test:
    try:
        # 简单功能测试
        if name == "numpy":
            test_array = module.array([1, 2, 3])
        elif name == "torch":
            test_tensor = module.tensor([1, 2, 3])
        elif name == "tensorflow":
            test_tf = module.constant([1, 2, 3])
        print(f"✅ {name}: 导入成功")
    except Exception as e:
        print(f"❌ {name}: 测试失败 - {e}")

# 6. 预期性能估算
print("\n6. 训练性能预估")
print("-" * 40)

ROSENBROCK_DIMENSION = 20
INITIAL_DATA_SIZE = 800
N_AL_ITERATIONS = 400
CNN_TRAINING_EPOCHS = 200

estimated_time_per_cnn = 30  # 秒
estimated_time_per_nte = 10  # 秒
total_estimated_time = N_AL_ITERATIONS * (estimated_time_per_cnn + estimated_time_per_nte)

print(f"配置参数:")
print(f"  - 维度: {ROSENBROCK_DIMENSION}D")
print(f"  - 初始样本: {INITIAL_DATA_SIZE}")
print(f"  - AL迭代: {N_AL_ITERATIONS}")
print(f"  - CNN轮数: {CNN_TRAINING_EPOCHS}")
print(f"预估训练时间: {total_estimated_time/3600:.1f} 小时")

if total_estimated_time > 43200:  # 12小时
    print("⚠️  预估时间超过12小时，建议在P100环境下运行")
else:
    print("✅ 预估时间合理，可在Kaggle环境完成")

print("\n" + "="*80)
print("环境验证完成！如果所有项目都显示✅，则可以开始训练。")
print("如果有❌项目，请检查对应的环境配置。")
print("="*80)


In [ ]:
# Cell 7: NTE Searcher 核心算法方法

def _calculate_ducb(self, node, parent_total_visits, c_rho, current_c0):
    """计算DUCB值"""
    if node.visit_count < 0:
        raise ValueError("Node visit count cannot be negative.")
    if parent_total_visits <= 0:
        parent_total_visits = 1

    exploitation_term = -node.value_pred
    n = node.visit_count
    N = parent_total_visits
    
    log_term = math.log(max(N, 1))
    exploration_term = current_c0 * c_rho * math.sqrt((2 * log_term) / (n + 1))

    return exploitation_term + exploration_term

def _calculate_ducb_with_global_awareness(self, node, parent_total_visits, 
                                        c_rho, current_c0,
                                        global_access_record, nte_id, 
                                        state_tuple):
    """计算考虑全局访问情况的DUCB值"""
    if node.visit_count < 0:
        raise ValueError("Node visit count cannot be negative.")
    if parent_total_visits <= 0:
        parent_total_visits = 1

    exploitation_term = -node.value_pred
    n = node.visit_count
    
    # 检查是否是重复访问的节点
    if (state_tuple in global_access_record and 
        len(global_access_record[state_tuple]['nte_visits']) > 1):
        # 这是一个被多个NTE访问过的节点
        # 使用增强的N值来提高探索性
        other_nte_visits = sum(
            visits for other_nte_id, visits in global_access_record[state_tuple]['nte_visits'].items()
            if other_nte_id != nte_id
        )
        enhanced_N = parent_total_visits + other_nte_visits
        
        logger.debug(f"重复访问节点检测: 原N={parent_total_visits}, 其他NTE访问={other_nte_visits}, 增强N={enhanced_N}")
    else:
        # 正常情况，使用当前NTE的总访问次数
        enhanced_N = parent_total_visits
    
    log_term = math.log(max(enhanced_N, 1))
    exploration_term = current_c0 * c_rho * math.sqrt((2 * log_term) / (n + 1))

    return exploitation_term + exploration_term

def _conditional_selection(self, nodes, parent_node):
    """条件选择机制，避免陷入低价值分支"""
    if not nodes:
        return None
        
    # 设置阈值：父节点预测值的一定比例
    value_threshold = parent_node.value_pred * 1.5  # 允许最多比父节点的预测值大50%
    
    # 筛选掉预测值远差于父节点的节点
    filtered_nodes = [node for node in nodes if node.value_pred <= value_threshold]
    
    # 如果过滤后没有节点，则使用原始节点列表
    if not filtered_nodes:
        filtered_nodes = nodes
        
    # 在过滤后的节点中选择DUCB最高的
    best_node = max(filtered_nodes, key=lambda n: n.visit_count)
    return best_node

def _select_candidates_mixed_strategy(self, visited_nodes):
    """使用混合策略从单个起始点的NTE搜索结果中选择候选点"""
    if not visited_nodes:
        return []

    nodes_list = list(visited_nodes.values())
    
    # 按预测值排序，选择最优的
    nodes_by_pred = sorted(nodes_list, key=lambda n: n.value_pred)
    best_pred_candidates = [np.array(node.state) for node in nodes_by_pred[:15]]
    
    # 按访问次数排序，选择最常访问的
    nodes_by_visits = sorted(nodes_list, key=lambda n: n.visit_count, reverse=True)
    most_visited_candidates = [np.array(node.state) for node in nodes_by_visits[:3]]
    
    # 随机选择2个
    remaining_nodes = [node for node in nodes_list 
                      if node not in nodes_by_pred[:15] and node not in nodes_by_visits[:3]]
    random_candidates = []
    if len(remaining_nodes) >= 2:
        random_selection = random.sample(remaining_nodes, 2)
        random_candidates = [np.array(node.state) for node in random_selection]
    elif remaining_nodes:
        random_candidates = [np.array(node.state) for node in remaining_nodes]
    
    # 合并所有候选点
    all_candidates = best_pred_candidates + most_visited_candidates + random_candidates
    
    # 去重（基于状态相似性）
    unique_candidates = []
    tolerance = 1e-8
    
    for candidate in all_candidates:
        is_duplicate = False
        for existing in unique_candidates:
            if np.allclose(candidate, existing, atol=tolerance, rtol=tolerance):
                is_duplicate = True
                break
        if not is_duplicate:
            unique_candidates.append(candidate)
    
    return unique_candidates[:self.validation_batch_size]

def _deduplicate_candidates_with_predictions(self, candidates_with_pred, tolerance=1e-8):
    """对带预测值的候选点列表进行去重，保留预测值更好的版本"""
    if not candidates_with_pred:
        return []
    
    unique_candidates = []
    
    for candidate, pred_value in candidates_with_pred:
        is_duplicate = False
        candidate_array = np.array(candidate)
        
        for i, (existing_candidate, existing_pred) in enumerate(unique_candidates):
            existing_array = np.array(existing_candidate)
            
            # 检查两个点是否在容差范围内相等
            if np.allclose(candidate_array, existing_array, atol=tolerance, rtol=tolerance):
                # 发现重复，保留预测值更好的那个
                if pred_value < existing_pred:  # 对于最小化问题，更小的值更好
                    unique_candidates[i] = (candidate, pred_value)
                is_duplicate = True
                break
        
        if not is_duplicate:
            unique_candidates.append((candidate, pred_value))
    
    return unique_candidates

# 将所有方法添加到NTESearcher类中
NTESearcher._calculate_ducb = _calculate_ducb
NTESearcher._calculate_ducb_with_global_awareness = _calculate_ducb_with_global_awareness
NTESearcher._conditional_selection = _conditional_selection
NTESearcher._select_candidates_mixed_strategy = _select_candidates_mixed_strategy
NTESearcher._deduplicate_candidates_with_predictions = _deduplicate_candidates_with_predictions

print("✓ NTE Searcher 核心算法方法已添加")


In [ ]:
# Cell 8: NTE Searcher 主搜索方法 - 完整实现

def search(self, surrogate_model, current_best_state, min_y_observed, 
           dynamic_exploration_factor=1.0, all_states=None, all_values=None,
           previous_iteration_samples=None, current_iteration=1, 
           return_ducb_trends=False):
    """
    NTE搜索主方法 - 完整实现，包含所有原始功能
    
    这是调试动态探索因子的核心方法：
    dynamic_exploration_factor = 0.8 + 1 * min(20, 2 ** no_improvement_streak)
    """
    
    logger.info(f"开始NTE搜索，动态探索因子: {dynamic_exploration_factor:.2f}")
    
    # 初始化数据结构
    visited_nodes = {}  # 状态tuple -> Node对象的映射
    all_candidates_with_predictions = []  # 所有候选点及其预测值
    ducb_trends_data = []  # DUCB趋势数据收集
    global_access_record = {}  # 全局访问记录
    
    # 计算自适应探索因子c(rho)
    c_rho = self.rho_scaling_factor_heuristic(min_y_observed)
    logger.info(f"自适应探索因子 c(rho): {c_rho:.4f}")
    
    # 预计算动态搜索边界以提高效率
    pre_computed_bounds = self._calculate_dynamic_search_space(all_states, all_values, current_iteration)
    dynamic_mins, dynamic_maxs = pre_computed_bounds
    logger.info(f"动态搜索区间: [{dynamic_mins[0]:.3f}, {dynamic_maxs[0]:.3f}] (示例第1维)")
    
    # 确定NTE搜索的起始点列表
    best_states_for_nte = []
    
    if current_best_state is not None:
        # 总是包含当前全局最优点
        best_states_for_nte.append(current_best_state.copy())
        
    # 如果有历史数据，选择额外的起始点
    if all_states is not None and all_values is not None and len(all_states) > 0:
        # 从最优的10%样本中选择额外起始点
        sorted_indices = np.argsort(all_values.flatten())
        top_candidates_count = max(1, min(3, len(sorted_indices) // 10))
        
        for i in range(min(top_candidates_count, len(sorted_indices))):
            candidate_state = all_states[sorted_indices[i]].copy()
            
            # 避免重复（如果候选状态与已有状态太接近）
            is_duplicate = False
            for existing_state in best_states_for_nte:
                if np.allclose(candidate_state, existing_state, atol=1e-6):
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                best_states_for_nte.append(candidate_state)
    
    # 如果仍然没有足够的起始点，添加随机起始点
    while len(best_states_for_nte) < 2:
        random_start = np.array([
            random.uniform(dynamic_mins[i], dynamic_maxs[i]) 
            for i in range(self.dimension)
        ])
        best_states_for_nte.append(random_start)
    
    logger.info(f"选择了 {len(best_states_for_nte)} 个NTE起始点进行搜索")
    
    # === 主要NTE搜索循环：对每个起始点执行搜索 ===
    for nte_id, start_state in enumerate(best_states_for_nte):
        # 初始化当前NTE的DUCB趋势收集器
        current_nte_ducb_trend = {
            'nte_id': nte_id,
            'ducb_values': [],
            'rollout_indices': [],
            'selected_states': [],
            'model_pred_values': [],
            'node_changes': []
        }
        
        # 初始化根节点
        root_state_tuple = tuple(start_state)
        try:
            root_pred_value = float(surrogate_model.predict(np.array([start_state]))[0])
        except Exception as e:
            logger.error(f"预测根节点失败: {e}")
            root_pred_value = float('inf')
        
        if root_state_tuple not in visited_nodes:
            visited_nodes[root_state_tuple] = Node(
                state=root_state_tuple, 
                value_pred=root_pred_value, 
                visit_count=0
            )
        
        current_root_state_tuple = root_state_tuple
        logger.debug(f"开始从起始点 {nte_id+1}/{len(best_states_for_nte)} 进行NTE搜索，预测值: {root_pred_value:.4e}")
        
        # 调整当前NTE的c0值
        current_c0 = self.base_c0 * dynamic_exploration_factor
        nte_total_visits = 0  # 当前NTE的总访问次数
        
        # === Rollout Phase ===
        for rollout_round in range(self.rollout_rounds):
            # 获取当前根节点对象
            current_root_node_obj = visited_nodes[current_root_state_tuple]
            current_root_state_vector = np.array(current_root_node_obj.state)
            
            # 递增当前根节点的访问计数
            current_root_node_obj.visit_count += 1
            nte_total_visits += 1
            
            # 更新全局访问记录
            if current_root_state_tuple not in global_access_record:
                global_access_record[current_root_state_tuple] = {
                    'total_visits': 0,
                    'nte_visits': {}
                }
            global_access_record[current_root_state_tuple]['total_visits'] += 1
            if nte_id not in global_access_record[current_root_state_tuple]['nte_visits']:
                global_access_record[current_root_state_tuple]['nte_visits'][nte_id] = 0
            global_access_record[current_root_state_tuple]['nte_visits'][nte_id] += 1
            
            # === 生成叶节点(膨胀阶段) ===
            leaf_states_vectors = self._mixed_expansion(
                current_root_state_vector, 
                dynamic_exploration_factor,
                all_states, 
                all_values, 
                current_iteration,
                pre_computed_bounds=pre_computed_bounds
            )
            
            if not leaf_states_vectors:
                # 如果混合扩展失败，生成随机状态
                rand_state = np.array([random.uniform(dynamic_mins[i], dynamic_maxs[i]) for i in range(self.dimension)])
                leaf_states_vectors = [rand_state]
            
            # 用替代模型预测叶节点值
            try:
                leaf_states_batch = np.array(leaf_states_vectors)
                leaf_pred_values = surrogate_model.predict(leaf_states_batch)
                if not isinstance(leaf_pred_values, np.ndarray):
                    leaf_pred_values = np.array(leaf_pred_values)
                
                leaf_pred_values = leaf_pred_values.flatten()

                # 处理 NaN/Inf 值
                if np.any(np.isnan(leaf_pred_values)) or np.any(np.isinf(leaf_pred_values)):
                    logger.warning(f"叶节点预测中存在NaN/Inf值，将替换它们。")
                    leaf_pred_values = np.nan_to_num(leaf_pred_values, nan=float('inf'), posinf=float('inf'), neginf=float('inf'))

            except Exception as e:
                logger.error(f"预测叶节点错误: {e}。使用无穷大。")
                leaf_pred_values = np.array([float('inf')] * len(leaf_states_vectors))
            
            # 创建或更新叶节点
            leaf_nodes = []
            
            for state_vec, pred_val in zip(leaf_states_vectors, leaf_pred_values):
                state_tuple = tuple(state_vec)
                node_value = float(pred_val)
                
                if state_tuple not in visited_nodes:
                    visited_nodes[state_tuple] = Node(state=state_tuple, value_pred=node_value, visit_count=0)
                    
                    # 初始化全局访问记录
                    if state_tuple not in global_access_record:
                        global_access_record[state_tuple] = {
                            'total_visits': 0,
                            'nte_visits': {}
                        }
                else:
                    # 更新预测值，保持访问计数
                    visited_nodes[state_tuple].value_pred = node_value
                
                leaf_nodes.append(visited_nodes[state_tuple])
            
            # 计算根节点和叶节点的DUCB
            root_ducb = self._calculate_ducb_with_global_awareness(
                current_root_node_obj, 
                current_root_node_obj.visit_count,
                c_rho, 
                current_c0,
                global_access_record,
                nte_id,
                current_root_state_tuple
            )
            
            # 使用条件选择
            best_leaf_node = self._conditional_selection(leaf_nodes, current_root_node_obj)
            if best_leaf_node is None:
                # 如果条件选择失败，回退到根节点
                next_root_state_tuple = current_root_state_tuple
                self.last_move_successful = False
                
                # 收集DUCB趋势数据
                if return_ducb_trends:
                    current_nte_ducb_trend['ducb_values'].append(root_ducb)
                    current_nte_ducb_trend['rollout_indices'].append(rollout_round)
                    current_nte_ducb_trend['selected_states'].append(np.array(current_root_state_tuple))
                    current_nte_ducb_trend['model_pred_values'].append(-current_root_node_obj.value_pred)
            else:
                # 对选择的最佳叶节点计算DUCB
                best_leaf_ducb = self._calculate_ducb_with_global_awareness(
                    best_leaf_node, 
                    nte_total_visits,
                    c_rho, 
                    current_c0,
                    global_access_record,
                    nte_id,
                    best_leaf_node.state
                )
                
                # 局部反向传播：根据DUCB比较决定下一步
                if root_ducb >= best_leaf_ducb:
                    next_root_state_tuple = current_root_state_tuple
                    self.last_move_successful = False
                    
                    # 收集DUCB趋势数据（根节点）
                    if return_ducb_trends:
                        current_nte_ducb_trend['ducb_values'].append(root_ducb)
                        current_nte_ducb_trend['rollout_indices'].append(rollout_round)
                        current_nte_ducb_trend['selected_states'].append(np.array(current_root_state_tuple))
                        current_nte_ducb_trend['model_pred_values'].append(-current_root_node_obj.value_pred)
                else:
                    next_root_state_tuple = best_leaf_node.state
                    # 判断是否找到了更好的点
                    self.last_move_successful = best_leaf_node.value_pred < current_root_node_obj.value_pred
                    if self.last_move_successful and self.last_move_deterministic:
                        logger.debug(f"确定性移动成功，找到了更优值: {current_root_node_obj.value_pred:.4e} -> {best_leaf_node.value_pred:.4e}")
                    
                    # 收集DUCB趋势数据（叶节点）
                    if return_ducb_trends:
                        current_nte_ducb_trend['ducb_values'].append(best_leaf_ducb)
                        current_nte_ducb_trend['rollout_indices'].append(rollout_round)
                        current_nte_ducb_trend['selected_states'].append(np.array(best_leaf_node.state))
                        current_nte_ducb_trend['model_pred_values'].append(-best_leaf_node.value_pred)
                        # 记录节点变更
                        if current_root_state_tuple != best_leaf_node.state:
                            current_nte_ducb_trend['node_changes'].append(rollout_round)
                    
                    # 根据NTE，增加选中叶节点的访问次数
                    best_leaf_node.visit_count += 1
                    nte_total_visits += 1
                    
                    # 更新全局访问记录
                    if next_root_state_tuple not in global_access_record:
                        global_access_record[next_root_state_tuple] = {
                            'total_visits': 0,
                            'nte_visits': {}
                        }
                    global_access_record[next_root_state_tuple]['total_visits'] += 1
                    if nte_id not in global_access_record[next_root_state_tuple]['nte_visits']:
                        global_access_record[next_root_state_tuple]['nte_visits'][nte_id] = 0
                    global_access_record[next_root_state_tuple]['nte_visits'][nte_id] += 1
            
            # 更新下一轮迭代的根节点
            current_root_state_tuple = next_root_state_tuple
        
        # 从这个起始点搜索得到的候选点添加到总列表中
        candidates_from_this_root = self._select_candidates_mixed_strategy(visited_nodes)
        for candidate in candidates_from_this_root:
            candidate_tuple = tuple(candidate)
            # 获取预测值
            if candidate_tuple in visited_nodes:
                pred_value = visited_nodes[candidate_tuple].value_pred
            else:
                try:
                    pred_value = float(surrogate_model.predict(np.array([candidate]))[0])
                except:
                    pred_value = float('inf')
            
            all_candidates_with_predictions.append((candidate, pred_value))
        
        # 将当前起始点的DUCB趋势数据添加到总的趋势数据中
        if return_ducb_trends:
            ducb_trends_data.append(current_nte_ducb_trend)
    
    # === 最终处理阶段：智能去重和最优选择 ===
    logger.info(f"NTE搜索完成，合并前共有 {len(all_candidates_with_predictions)} 个候选点")
    
    # 去重候选点（保留预测值更好的版本）
    unique_candidates_with_pred = self._deduplicate_candidates_with_predictions(
        all_candidates_with_predictions, 
        tolerance=1e-8
    )
    logger.info(f"去重后共有 {len(unique_candidates_with_pred)} 个唯一候选点，" +
               f"去重比例: {(len(all_candidates_with_predictions) - len(unique_candidates_with_pred))/len(all_candidates_with_predictions)*100:.1f}%")
    
    # 如果候选点不足，添加随机点
    if len(unique_candidates_with_pred) < self.validation_batch_size:
        num_random_to_add = self.validation_batch_size - len(unique_candidates_with_pred)
        logger.info(f"候选点不足，添加{num_random_to_add}个随机点")
        
        # 创建现有候选点的集合用于避免重复
        existing_candidates_set = set()
        for candidate, _ in unique_candidates_with_pred:
            rounded_candidate = tuple(np.round(candidate, 6))
            existing_candidates_set.add(rounded_candidate)
        
        added_random_count = 0
        max_attempts = num_random_to_add * 10
        attempts = 0
        
        while added_random_count < num_random_to_add and attempts < max_attempts:
            random_point = np.array([random.uniform(self.domain_mins[i], self.domain_maxs[i]) 
                                     for i in range(self.dimension)])
            rounded_random = tuple(np.round(random_point, 6))
            
            if rounded_random not in existing_candidates_set:
                try:
                    random_pred = float(surrogate_model.predict(np.array([random_point]))[0])
                except:
                    random_pred = float('inf')
                
                unique_candidates_with_pred.append((random_point, random_pred))
                existing_candidates_set.add(rounded_random)
                added_random_count += 1
            
            attempts += 1
        
        if added_random_count < num_random_to_add:
            logger.warning(f"只能添加 {added_random_count}/{num_random_to_add} 个不重复的随机点")
    
    # 按预测值排序（升序，因为Rosenbrock是最小化问题）
    unique_candidates_with_pred.sort(key=lambda x: x[1])
    
    # 选择最终的20个样本：前15个最优 + 后5个随机探索
    final_candidates = []
    
    # 前15个预测值最优的样本
    best_15_count = min(15, len(unique_candidates_with_pred))
    for i in range(best_15_count):
        final_candidates.append(unique_candidates_with_pred[i][0])
    
    # 剩余5个从其他候选点中随机选择
    remaining_candidates = unique_candidates_with_pred[best_15_count:]
    explore_count = min(self.validation_batch_size - best_15_count, len(remaining_candidates))
    
    if explore_count > 0:
        if len(remaining_candidates) > explore_count:
            # 随机选择剩余的探索样本
            import random as rand
            selected_indices = rand.sample(range(len(remaining_candidates)), explore_count)
            for idx in selected_indices:
                final_candidates.append(remaining_candidates[idx][0])
        else:
            # 全部添加
            for candidate, _ in remaining_candidates:
                final_candidates.append(candidate)
    
    logger.info(f"最终选择 {len(final_candidates)} 个样本：{best_15_count} 个最优样本 + {len(final_candidates) - best_15_count} 个探索样本")
    
    # 确保返回列表不超过所请求的候选点数量
    final_result = final_candidates[:self.validation_batch_size]
    
    if return_ducb_trends:
        return final_result, ducb_trends_data
    else:
        return final_result

# 将search方法添加到NTESearcher类中
NTESearcher.search = search

print("✓ NTE Searcher 完整实现已完成 - 这是调试动态探索因子的核心方法！")


In [ ]:
# Cell 9: 完整可视化工具 (plot_utils.py) - 恢复所有原始功能

import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import logging
import matplotlib
from scipy.stats import pearsonr

# 配置全局字体设置为默认英文字体
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']

# 使用Agg后端，以便在没有GUI的环境中也能保存图像
plt.switch_backend('Agg') 

def create_results_directory(base_results_path):
    """在指定的基础路径下创建一个以当前时间戳命名的子目录"""
    try:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        results_dir = os.path.join(base_results_path, timestamp)
        os.makedirs(results_dir, exist_ok=True)
        logger.info(f"Results directory created: {results_dir}")
        return results_dir
    except Exception as e:
        logger.error(f"Error creating results directory: {e}")
        return None

def plot_global_min_value_trend(iterations, min_values, output_dir, filename="global_min_value_trend.png"):
    """绘制并保存全局最小真值的变化趋势图"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return

    try:
        # 确保iterations和min_values长度相同
        if len(iterations) != len(min_values):
            logger.warning(f"Iterations和min_values数组长度不同: {len(iterations)} vs {len(min_values)}.")
            min_length = min(len(iterations), len(min_values))
            iterations = iterations[:min_length]
            min_values = min_values[:min_length]
            logger.warning(f"数组截断至相同长度: {min_length}")
            
        plt.figure(figsize=(10, 6))
        plt.plot(iterations, min_values, marker='o', linestyle='-', color='b')
        plt.title('Global Minimum Value Trend')
        plt.xlabel('Iteration')
        plt.ylabel('Current Minimum True Value Found')
        plt.grid(True)
        plt.xticks(iterations)
        
        # 只在找到更低全局最小值时添加标注
        min_values_array = np.array(min_values)
        global_min = float('inf')
        
        for i, (iter_num, current_min) in enumerate(zip(iterations, min_values_array)):
            if current_min < global_min:
                global_min = current_min
                if current_min < 1000:
                    label_txt = f"{current_min:.2f}"
                else:
                    label_txt = f"{current_min:.2e}"
                plt.annotate(label_txt, 
                            (iter_num, current_min),
                            textcoords="offset points", 
                            xytext=(0,10), 
                            ha='center',
                            fontsize=9,
                            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path)
        plt.close()
        logger.info(f"全局最小值趋势图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制全局最小值趋势图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())

def plot_loss_trend(data_points, output_dir, x_label="Epoch/Iteration", title="Model Loss Trend", 
                   filename="loss_trend.png", custom_ticks=None, pearson_coeffs=None):
    """绘制并保存损失函数的变化趋势图"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return
    
    try:
        plt.figure(figsize=(10, 6))
        
        # 绘制损失曲线
        if custom_ticks and len(custom_ticks) == len(data_points):
            plt.plot(custom_ticks, data_points, marker='.', linestyle='-', color='r', label='Loss')
            x_values = custom_ticks
        else:
            plt.plot(data_points, marker='.', linestyle='-', color='r', label='Loss')
            x_values = range(len(data_points))
        
        plt.title(title if title else 'Model Loss Trend')
        plt.xlabel(x_label if x_label else 'Training Epochs')
        plt.ylabel('Loss Value')
        plt.grid(True)
        
        # 如果提供了皮尔逊相关系数，添加第二个y轴并绘制
        if pearson_coeffs is not None and len(pearson_coeffs) == len(data_points):
            ax2 = plt.gca().twinx()
            ax2.plot(x_values, pearson_coeffs, marker='s', linestyle='--', color='g', label='Pearson Correlation')
            ax2.set_ylabel('Pearson Correlation Coefficient', color='g')
            ax2.tick_params(axis='y', labelcolor='g')
            ax2.set_ylim(-0.1, 1.1)
            
            # 在每个数据点上标注皮尔逊系数值
            for i, coef in enumerate(pearson_coeffs):
                ax2.annotate(f"{coef:.3f}",
                           (x_values[i], pearson_coeffs[i]),
                           textcoords="offset points", 
                           xytext=(0,10), 
                           ha='center',
                           fontsize=8,
                           color='g')
            
            # 修改图例以包含两个曲线
            lines1, labels1 = plt.gca().get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
        else:
            plt.legend(loc='upper right')
        
        # 使用自定义x轴刻度（如果提供）
        if custom_ticks:
            plt.xticks(custom_ticks)

        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path)
        plt.close()
        logger.info(f"Loss趋势图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制损失趋势图出错: {e}")

def plot_pearson_correlation_trend(iterations, pearson_coeffs, output_dir, 
                                  filename="pearson_correlation_trend.png", title=None):
    """绘制并保存皮尔逊相关系数的变化趋势图"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return

    try:
        plt.figure(figsize=(10, 6))
        plt.plot(iterations, pearson_coeffs, marker='o', linestyle='-', color='g')
        plt.title(title if title else 'Pearson Correlation Coefficient Trend (Model vs. Truth)')
        plt.xlabel('Iteration')
        plt.ylabel('Pearson Correlation Coefficient')
        plt.ylim(-1.1, 1.1)
        plt.grid(True)
        plt.xticks(iterations)
        
        # 添加水平参考线
        plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        plt.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='Moderate Correlation')
        plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='Strong Correlation')
        
        plt.legend()
        
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path)
        plt.close()
        logger.info(f"Pearson相关系数趋势图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制Pearson相关系数趋势图出错: {e}")

def plot_nte_performance(iterations, num_candidates, output_dir, filename="nte_candidates_trend.png"):
    """绘制并保存NTE搜索性能图，显示每次迭代找到的候选点数量"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return

    try:
        plt.figure(figsize=(10, 6))
        plt.plot(iterations, num_candidates, marker='s', linestyle='-', color='purple', linewidth=2)
        plt.title('NTE Search Performance: Number of Candidates Found')
        plt.xlabel('Iteration')
        plt.ylabel('Number of Candidate Points')
        plt.grid(True)
        plt.xticks(iterations)
        
        # 添加平均线
        avg_candidates = np.mean(num_candidates)
        plt.axhline(y=avg_candidates, color='red', linestyle='--', alpha=0.7, 
                   label=f'Average: {avg_candidates:.1f}')
        
        plt.legend()
        
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path)
        plt.close()
        logger.info(f"NTE性能图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制NTE性能图出错: {e}")

print("✓ 完整可视化工具 (第1部分) 已加载")


In [ ]:
# Cell 10: 完整可视化工具 (第2部分) - 高级可视化功能

def plot_prediction_vs_truth(predictions, ground_truth, output_dir, iteration=None, 
                            filename=None, title_suffix=None):
    """绘制预测值vs真实值散点图，并计算评估指标"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return None, None, None

    try:
        predictions = np.array(predictions).flatten()
        ground_truth = np.array(ground_truth).flatten()
        
        if len(predictions) != len(ground_truth):
            logger.error(f"预测值和真实值长度不匹配: {len(predictions)} vs {len(ground_truth)}")
            return None, None, None
        
        # 计算评估指标
        mse = np.mean((predictions - ground_truth) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(predictions - ground_truth))
        
        plt.figure(figsize=(10, 8))
        
        # 绘制散点图
        plt.scatter(ground_truth, predictions, alpha=0.6, s=20)
        
        # 绘制理想线 (y=x)
        min_val = min(np.min(ground_truth), np.min(predictions))
        max_val = max(np.max(ground_truth), np.max(predictions))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
        
        # 设置标题和标签
        title = f'Prediction vs Ground Truth'
        if title_suffix:
            title += f' ({title_suffix})'
        if iteration is not None:
            title += f' - Iteration {iteration}'
        
        plt.title(title)
        plt.xlabel('Ground Truth')
        plt.ylabel('Prediction')
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # 添加评估指标文本
        textstr = f'RMSE: {rmse:.4e}\\nMAE: {mae:.4e}\\nSamples: {len(predictions)}'
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
        plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
                verticalalignment='top', bbox=props)
        
        # 保存图形
        if filename is None:
            if iteration is not None:
                filename = f"prediction_vs_truth_iter_{iteration}.png"
            else:
                filename = "prediction_vs_truth.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"预测vs真实值图已保存: {save_path} (RMSE: {rmse:.4e})")
        return mse, rmse, mae
        
    except Exception as e:
        logger.error(f"绘制预测vs真实值图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None, None, None

def plot_residuals(predictions, ground_truth, output_dir, iteration=None, filename=None):
    """绘制残差分析图"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return

    try:
        predictions = np.array(predictions).flatten()
        ground_truth = np.array(ground_truth).flatten()
        
        if len(predictions) != len(ground_truth):
            logger.error(f"预测值和真实值长度不匹配: {len(predictions)} vs {len(ground_truth)}")
            return
        
        residuals = predictions - ground_truth
        
        plt.figure(figsize=(12, 8))
        
        # 子图1: 残差vs预测值
        plt.subplot(2, 2, 1)
        plt.scatter(predictions, residuals, alpha=0.6, s=20)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predictions')
        plt.ylabel('Residuals')
        plt.title('Residuals vs Predictions')
        plt.grid(True, alpha=0.3)
        
        # 子图2: 残差vs真实值
        plt.subplot(2, 2, 2)
        plt.scatter(ground_truth, residuals, alpha=0.6, s=20)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Ground Truth')
        plt.ylabel('Residuals')
        plt.title('Residuals vs Ground Truth')
        plt.grid(True, alpha=0.3)
        
        # 子图3: 残差直方图
        plt.subplot(2, 2, 3)
        plt.hist(residuals, bins=30, alpha=0.7, edgecolor='black')
        plt.xlabel('Residuals')
        plt.ylabel('Frequency')
        plt.title('Residuals Distribution')
        plt.grid(True, alpha=0.3)
        
        # 子图4: Q-Q图（简化版）
        plt.subplot(2, 2, 4)
        sorted_residuals = np.sort(residuals)
        theoretical_quantiles = np.linspace(-3, 3, len(sorted_residuals))
        plt.scatter(theoretical_quantiles, sorted_residuals, alpha=0.6, s=20)
        plt.plot(theoretical_quantiles, theoretical_quantiles * np.std(residuals), 'r--')
        plt.xlabel('Theoretical Quantiles')
        plt.ylabel('Sample Quantiles')
        plt.title('Q-Q Plot')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # 保存图形
        if filename is None:
            if iteration is not None:
                filename = f"residual_analysis_iter_{iteration}.png"
            else:
                filename = "residual_analysis.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"残差分析图已保存: {save_path}")
        
    except Exception as e:
        logger.error(f"绘制残差分析图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())

def plot_best_samples_pearson_trend(iterations_history, pearson_history, results_dir, 
                                   filename="best_samples_pearson_trend.png", 
                                   title="Pearson Correlation on Best Samples"):
    """绘制最佳样本皮尔逊相关系数趋势图"""
    if not os.path.exists(results_dir):
        logger.warning(f"Results directory {results_dir} does not exist. Skipping plot.")
        return

    try:
        plt.figure(figsize=(10, 6))
        plt.plot(iterations_history, pearson_history, marker='o', linestyle='-', 
                color='blue', linewidth=2, markersize=6)
        plt.title(title)
        plt.xlabel('Iteration')
        plt.ylabel('Pearson Correlation Coefficient')
        plt.grid(True, alpha=0.3)
        plt.ylim(-1.1, 1.1)
        
        # 添加水平参考线
        plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        plt.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='Moderate (0.5)')
        plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='Strong (0.8)')
        
        # 如果有足够的数据点，添加趋势线
        if len(iterations_history) > 2:
            z = np.polyfit(iterations_history, pearson_history, 1)
            p = np.poly1d(z)
            plt.plot(iterations_history, p(iterations_history), "g--", alpha=0.8, label='Trend')
        
        plt.legend()
        plt.xticks(iterations_history)
        
        save_path = os.path.join(results_dir, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"最佳样本Pearson趋势图已保存: {save_path}")
        
    except Exception as e:
        logger.error(f"绘制最佳样本Pearson趋势图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())

def plot_surrogate_vs_selected_samples(surrogate_model, selected_X, selected_y_true, 
                                      data_manager, output_dir, iteration, filename=None):
    """绘制替代模型预测vs新选择样本的对比图"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return None

    try:
        if len(selected_X) == 0:
            logger.warning("没有选择的样本，跳过对比图绘制")
            return None
        
        # 使用替代模型预测新选择的样本
        if hasattr(surrogate_model, 'predict'):
            predictions_log = surrogate_model.predict(selected_X)
            # 转换为原始尺度
            predictions_orig = safe_power10(predictions_log)
        else:
            logger.error("替代模型没有predict方法")
            return None
        
        # 计算评估指标
        if len(predictions_orig) >= 2:
            pearson_corr, _ = pearsonr(predictions_orig, selected_y_true.flatten())
            if np.isnan(pearson_corr):
                pearson_corr = 0.0
        else:
            pearson_corr = 0.0
        
        mae = np.mean(np.abs(predictions_orig - selected_y_true.flatten()))
        rmse = np.sqrt(np.mean((predictions_orig - selected_y_true.flatten()) ** 2))
        
        # 创建对比图
        plt.figure(figsize=(10, 8))
        
        plt.scatter(selected_y_true.flatten(), predictions_orig, alpha=0.7, s=50, 
                   c='blue', label=f'Selected Samples (n={len(selected_X)})')
        
        # 绘制理想线
        min_val = min(np.min(selected_y_true), np.min(predictions_orig))
        max_val = max(np.max(selected_y_true), np.max(predictions_orig))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
        
        plt.xlabel('True Values (Selected Samples)')
        plt.ylabel('Surrogate Model Predictions')
        plt.title(f'Surrogate Model vs Selected Samples (Iteration {iteration})')
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # 添加评估指标
        textstr = f'Pearson: {pearson_corr:.4f}\\nMAE: {mae:.4e}\\nRMSE: {rmse:.4e}'
        props = dict(boxstyle='round', facecolor='lightblue', alpha=0.8)
        plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
                verticalalignment='top', bbox=props)
        
        # 保存图形
        if filename is None:
            filename = f"surrogate_vs_selected_iter_{iteration}.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"替代模型vs选择样本对比图已保存: {save_path}")
        
        return {
            'pearson': pearson_corr,
            'mae': mae,
            'rmse': rmse,
            'n_samples': len(selected_X)
        }
        
    except Exception as e:
        logger.error(f"绘制替代模型vs选择样本对比图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None

print("✓ 完整可视化工具 (第2部分) 已加载")


In [ ]:
# Cell 11: 完整可视化工具 (第3部分) - 特殊可视化功能

def plot_top_samples_dimensions(data_manager, output_dir, iteration, n_top_samples=15, filename=None):
    """
    绘制最优样本在各个维度上的数值分布图
    
    Args:
        data_manager: 数据管理器
        output_dir: 输出目录
        iteration: 当前迭代次数
        n_top_samples: 要显示的最优样本数量
        filename: 可选的自定义文件名
    """
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return None

    try:
        # 获取所有样本数据
        all_X = data_manager.get_all_x_orig()
        all_y = data_manager.get_all_y_orig()
        
        if len(all_X) == 0:
            logger.warning("数据管理器中没有样本数据")
            return None
        
        # 按y值排序，获取最优的n_top_samples个样本
        sorted_indices = np.argsort(all_y.flatten())
        top_indices = sorted_indices[:min(n_top_samples, len(sorted_indices))]
        top_samples_X = all_X[top_indices]
        top_samples_y = all_y[top_indices]
        
        # 获取维度数量
        n_dimensions = top_samples_X.shape[1]
        dimensions = np.arange(1, n_dimensions + 1)
        
        # 创建图形
        plt.figure(figsize=(14, 8))
        
        # 为每个样本分配颜色和透明度
        colors = plt.cm.viridis(np.linspace(0, 1, len(top_samples_X)))
        
        # 绘制每个最优样本在各维度上的值
        for i, (sample_x, sample_y) in enumerate(zip(top_samples_X, top_samples_y)):
            alpha = 1.0 if i == 0 else 0.6  # 最优样本使用不透明，其他使用半透明
            linewidth = 3 if i == 0 else 1.5
            marker_size = 8 if i == 0 else 5
            
            if i == 0:
                label = f"Best (y={sample_y[0]:.4e})"
            elif i < 3:
                label = f"#{i+1} (y={sample_y[0]:.4e})"
            else:
                label = None
            
            plt.plot(dimensions, sample_x, marker='o', linestyle='-', 
                    color=colors[i], alpha=alpha, linewidth=linewidth, 
                    markersize=marker_size, label=label)
        
        # 添加参考线（平均值）
        if len(top_samples_X) > 1:
            mean_values = np.mean(top_samples_X, axis=0)
            plt.plot(dimensions, mean_values, marker='s', linestyle='--', 
                    color='red', alpha=0.8, linewidth=2, markersize=6,
                    label=f'Mean of Top {len(top_samples_X)}')
        
        # 设置图表属性
        plt.title(f'Top {n_top_samples} Samples Dimension Values (Iteration {iteration})', fontsize=14)
        plt.xlabel('Dimension', fontsize=12)
        plt.ylabel('Dimension Value', fontsize=12)
        plt.grid(True, alpha=0.3)
        
        # 设置x轴刻度
        plt.xticks(dimensions)
        
        # 设置图例，但只显示前几个最重要的样本以避免图例过于拥挤
        handles, labels = plt.gca().get_legend_handles_labels()
        # 显示最优样本、参考线以及其他几个样本
        important_indices = [0, -1] + list(range(1, min(4, len(handles)-1)))  # 最优、参考线、前几个其他样本
        important_handles = [handles[i] for i in important_indices if i < len(handles)]
        important_labels = [labels[i] for i in important_indices if i < len(labels)]
        plt.legend(important_handles, important_labels, 
                  bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        
        # 调整布局以适应图例
        plt.tight_layout()
        
        # 保存图形
        if filename is None:
            filename = f"top_samples_dimensions_iter_{iteration}.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"最优样本维度分布图已保存: {save_path}")
        return save_path
        
    except Exception as e:
        logger.error(f"绘制最优样本维度分布图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None

def plot_nte_ducb_trends(ducb_trends_data, output_dir, iteration, dynamic_exploration_factor, 
                        new_global_best_value=None, filename=None):
    """
    可视化NTE搜索过程中选择的父节点的DUCB值变化趋势
    
    Args:
        ducb_trends_data (list): 包含多个初始点的DUCB变化数据
        output_dir (str): 保存图表的目录路径
        iteration (int): 当前迭代次数
        dynamic_exploration_factor (float): 当前迭代使用的动态探索因子
        new_global_best_value (float, optional): 如果找到新的全局最小值，传入该值
        filename (str, optional): 自定义文件名
    """
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return

    try:
        # 创建图形
        plt.figure(figsize=(14, 10))
        
        # 定义颜色和节点变化标记
        colors = ['blue', 'red', 'green', 'orange', 'purple']
        node_change_markers = ['s', '^', 'D', 'v', 'p']  # 方块、三角、钻石、倒三角、五角形
        node_change_sizes = [80, 80, 80, 80, 80]  # 节点变化标记大小
        
        # 绘制每个初始点的DUCB变化趋势
        for i, trend_data in enumerate(ducb_trends_data):
            if not trend_data or 'ducb_values' not in trend_data:
                continue
                
            ducb_values = trend_data['ducb_values']
            rollout_indices = trend_data.get('rollout_indices', list(range(len(ducb_values))))
            node_changes = trend_data.get('node_changes', [])
            model_pred_values = trend_data.get('model_pred_values', [])
            
            if len(ducb_values) == 0:
                continue
            
            color_idx = i % len(colors)
            
            # 绘制DUCB值变化曲线
            plt.plot(rollout_indices, ducb_values, 
                    color=colors[color_idx], 
                    linewidth=2, alpha=0.8,
                    label=f'Initial Point {i+1} DUCB')
            
            # 绘制模型预测值的负值曲线（如果有数据）
            if len(model_pred_values) == len(ducb_values):
                plt.plot(rollout_indices, model_pred_values, 
                        color=colors[color_idx], 
                        linewidth=1.5, alpha=0.6, linestyle='--',
                        label=f'Initial Point {i+1} -v_ML')
            
            # 在节点变更位置添加特殊标记
            for change_idx in node_changes:
                if change_idx < len(rollout_indices) and change_idx < len(ducb_values):
                    plt.scatter(rollout_indices[change_idx], ducb_values[change_idx], 
                              marker=node_change_markers[color_idx],
                              s=node_change_sizes[color_idx],
                              color=colors[color_idx], 
                              edgecolors='white', linewidth=1.5,
                              alpha=0.9, zorder=5)
        
        # 设置图表属性
        title = f'NTE DUCB Values Trend (Iteration {iteration})'
        if dynamic_exploration_factor is not None:
            title += f'\\nExploration Factor: {dynamic_exploration_factor:.2f}'
        if new_global_best_value is not None:
            title += f' | New Global Best: {new_global_best_value:.6e}'
        
        plt.title(title, fontsize=14, fontweight='bold')
        plt.xlabel('NTE Rollout Round', fontsize=12)
        plt.ylabel('DUCB Value', fontsize=12)
        plt.grid(True, alpha=0.3)
        
        # 创建图例，包括DUCB曲线、模型预测值曲线和节点变化标记的说明
        legend_elements = []
        
        # 添加DUCB线条图例
        for i in range(min(len(ducb_trends_data), len(colors))):
            if (ducb_trends_data[i] and 'ducb_values' in ducb_trends_data[i] and 
                len(ducb_trends_data[i]['ducb_values']) > 0):
                legend_elements.append(plt.Line2D([0], [0], color=colors[i], 
                                                linewidth=2, label=f'Point {i+1} DUCB'))
        
        # 添加模型预测值线条图例
        for i in range(min(len(ducb_trends_data), len(colors))):
            if (ducb_trends_data[i] and 'model_pred_values' in ducb_trends_data[i] and 
                len(ducb_trends_data[i]['model_pred_values']) > 0):
                legend_elements.append(plt.Line2D([0], [0], color=colors[i], 
                                                linewidth=1.5, linestyle='--', alpha=0.6,
                                                label=f'Point {i+1} -v_ML'))
        
        # 添加节点变化标记的说明
        for i in range(min(3, len(ducb_trends_data))):  # 最多显示3个标记说明
            if i < len(colors) and i < len(node_change_markers):
                legend_elements.append(plt.Line2D([0], [0], marker=node_change_markers[i], 
                                                color=colors[i], linestyle='None',
                                                markersize=8, markeredgecolor='white', markeredgewidth=1.5,
                                                label=f'Point {i+1} Node Change'))
        
        plt.legend(handles=legend_elements, fontsize=9, loc='best')
        
        # 添加统计信息
        total_rollouts = max([len(trend['ducb_values']) for trend in ducb_trends_data 
                             if trend and 'ducb_values' in trend] + [0])
        total_node_changes = sum([len(trend.get('node_changes', [])) for trend in ducb_trends_data 
                                 if trend])
        
        stats_text = f'Total Rollouts: {total_rollouts}\\nNode Changes: {total_node_changes}'
        plt.text(0.02, 0.98, stats_text, 
                transform=plt.gca().transAxes, fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        # 调整布局
        plt.tight_layout()
        
        # 保存图形
        if filename is None:
            filename = f"nte_ducb_trends_iter_{iteration}.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"NTE DUCB趋势图已保存: {save_path}")
        return save_path
        
    except Exception as e:
        logger.error(f"绘制NTE DUCB趋势图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None

def plot_search_boundaries(iterations, boundaries_data, output_dir, filename_prefix="search_boundaries"):
    """绘制搜索边界变化图"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return

    try:
        if not boundaries_data:
            logger.warning("没有边界数据可以绘制")
            return
        
        # 假设我们只显示前几个维度的边界变化
        n_dims_to_show = min(5, len(boundaries_data[0][1]) if boundaries_data else 0)
        
        plt.figure(figsize=(12, 8))
        
        for dim in range(n_dims_to_show):
            mins = [boundary[1][dim] for boundary in boundaries_data]
            maxs = [boundary[2][dim] for boundary in boundaries_data]
            
            plt.subplot(2, 3, dim + 1)
            plt.plot(iterations, mins, 'b-', label=f'Min (Dim {dim+1})')
            plt.plot(iterations, maxs, 'r-', label=f'Max (Dim {dim+1})')
            plt.fill_between(iterations, mins, maxs, alpha=0.3)
            plt.title(f'Dimension {dim+1} Search Bounds')
            plt.xlabel('Iteration')
            plt.ylabel('Boundary Value')
            plt.legend()
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        save_path = os.path.join(output_dir, f"{filename_prefix}.png")
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info(f"搜索边界图已保存: {save_path}")
        
    except Exception as e:
        logger.error(f"绘制搜索边界图出错: {e}")
        import traceback
        logger.error(traceback.format_exc())

print("✓ 完整可视化工具 (第3部分) 已加载")
print("✓ 所有可视化功能已完全恢复，与原代码一致！")


In [ ]:
# Cell 12: 性能优化和进程管理模块
import os
import gc
import psutil
import time
import subprocess
import tensorflow as tf
from tensorflow.keras import mixed_precision
import logging

logger = logging.getLogger(__name__)

class PerformanceOptimizer:
    """性能优化器，负责优化TensorFlow训练性能"""
    
    def __init__(self):
        self.cpu_count = psutil.cpu_count(logical=True)
        self.physical_cpu_count = psutil.cpu_count(logical=False)
        self.memory_cache = {}
        self.is_configured = False
        logger.info(f"检测到CPU核心数: 物理={self.physical_cpu_count}, 逻辑={self.cpu_count}")
    
    def configure_tensorflow_performance(self):
        """配置TensorFlow性能设置 - 专门优化Kaggle P100显卡"""
        if self.is_configured:
            return
            
        logger.info("开始配置TensorFlow性能优化 (针对Kaggle P100)...")
        
        # P100专门的GPU内存配置
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                for gpu in gpus:
                    # P100显卡内存管理优化
                    tf.config.experimental.set_memory_growth(gpu, True)
                    # 设置虚拟GPU内存限制，避免OOM
                    tf.config.experimental.set_virtual_device_configuration(
                        gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=15000)]  # P100约16GB，留1GB余量
                    )
                logger.info(f"已配置P100 GPU内存管理，检测到{len(gpus)}个GPU")
            except RuntimeError as e:
                logger.warning(f"GPU内存配置失败: {e}")
        
        # 针对P100的并行度优化
        tf.config.threading.set_inter_op_parallelism_threads(4)  # P100最优配置
        tf.config.threading.set_intra_op_parallelism_threads(2)  # P100最优配置
        
        # P100混合精度优化
        if gpus:
            try:
                # P100支持混合精度但需要谨慎设置
                policy = mixed_precision.Policy('mixed_float16')
                mixed_precision.set_global_policy(policy)
                logger.info("已启用P100混合精度训练 (mixed_float16)")
            except Exception as e:
                logger.warning(f"P100混合精度配置失败: {e}")
        
        # P100专门的性能选项
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
        os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
        os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'  # P100最优线程模式
        os.environ['TF_GPU_THREAD_COUNT'] = '2'  # P100最优线程数
        os.environ['TF_USE_CUDNN_BATCHNORM_SPATIAL_PERSISTENT'] = '1'  # P100批归一化优化
        
        # XLA编译优化 (P100支持)
        tf.config.optimizer.set_jit(True)
        
        self.is_configured = True
        logger.info("P100 TensorFlow性能优化配置完成")
    
    def clear_memory_cache(self):
        """清理内存缓存"""
        self.memory_cache.clear()
        gc.collect()
        logger.info("已清理内存缓存")
    
    def monitor_performance(self, stage_name=""):
        """监控性能指标"""
        try:
            process = psutil.Process()
            memory_info = process.memory_info()
            memory_mb = memory_info.rss / 1024 / 1024
            cpu_percent = process.cpu_percent()
            logger.info(f"[{stage_name}] 内存: {memory_mb:.1f}MB, CPU: {cpu_percent:.1f}%")
            return memory_mb, cpu_percent
        except Exception as e:
            logger.warning(f"性能监控失败: {e}")
            return 0, 0

class ProcessOptimizer:
    """进程优化器，用于暂停非必要的进程以释放CPU资源"""
    
    def __init__(self):
        self.suspended_processes = []
        self.process_whitelist = [
            'python', 'python3', 'nvidia-smi', 'ssh', 'systemd', 'kernel', 
            'kthread', 'migration', 'rcu_', 'watchdog', 'bash', 'sh'
        ]
    
    def find_resource_heavy_processes(self, cpu_threshold=20.0, exclude_current=True):
        """找到占用CPU资源较多的非必要进程"""
        heavy_processes = []
        current_pid = os.getpid()
        
        try:
            for proc in psutil.process_iter(['pid', 'name', 'cpu_percent', 'username']):
                try:
                    proc_info = proc.info
                    
                    if exclude_current and proc_info['pid'] == current_pid:
                        continue
                    
                    if any(keyword in proc_info['name'].lower() for keyword in self.process_whitelist):
                        continue
                    
                    cpu_percent = proc_info['cpu_percent']
                    if cpu_percent is not None and cpu_percent > cpu_threshold:
                        heavy_processes.append({
                            'pid': proc_info['pid'],
                            'name': proc_info['name'],
                            'cpu_percent': cpu_percent,
                            'username': proc_info['username']
                        })
                        
                except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
                    continue
                    
        except Exception as e:
            logger.error(f"查找资源密集进程时出错: {e}")
        
        return heavy_processes
    
    def optimize_for_training(self, auto_suspend=False, cpu_threshold=25.0):
        """为训练优化系统进程 (Kaggle环境中禁用自动暂停)"""
        logger.info("检查系统进程状态...")
        
        if not auto_suspend:
            logger.info("在Kaggle环境中禁用进程优化功能")
            return []
        
        # Kaggle环境不执行实际的进程优化
        return []
    
    def restore_all_processes(self):
        """恢复所有被暂停的进程 (Kaggle环境中为空操作)"""
        if not self.suspended_processes:
            return
        
        logger.info("在Kaggle环境中无需恢复进程")

# 创建全局实例
performance_optimizer = PerformanceOptimizer()
process_optimizer = ProcessOptimizer()

logger.info("性能优化和进程管理模块加载完成")
print("性能优化器已初始化，适配Kaggle环境")


In [ ]:
# Cell 13: 完整的主执行逻辑 (main.py核心功能)
import atexit
import traceback
from scipy.stats import pearsonr

# 从主函数main.py移植的核心常量和工具函数
LOG_EPSILON = 1.0  # 对数变换常量

def rosenbrock_function(x):
    """计算给定向量x的Rosenbrock函数值"""
    return np.sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0, axis=0)

def rosenbrock_evaluator(x_vector):
    """外部评估器"""
    return rosenbrock_function(x_vector)

def calculate_pearson_correlation(model, data_manager):
    """计算模型预测与真值的皮尔逊相关系数"""
    if data_manager.num_samples < 2:
        return 0.0
    
    try:
        X_data = data_manager.get_all_x_orig()
        y_data_orig = data_manager.get_all_y_orig().flatten()
        
        predictions_log = model.predict(X_data)
        predictions_orig = safe_power10(predictions_log)
        
        if len(predictions_orig) < 2 or len(y_data_orig) < 2:
            logger.warning("数据点不足，无法计算皮尔逊相关系数")
            return 0.0
            
        correlation, _ = pearsonr(predictions_orig, y_data_orig)
        return correlation if not np.isnan(correlation) else 0.0
    
    except Exception as e:
        logger.error(f"计算皮尔逊相关系数时出错: {e}")
        return 0.0

def calculate_pearson_on_test_set(model, X_test, y_test):
    """计算测试集上的皮尔逊相关系数"""
    if X_test is None or y_test is None or X_test.shape[0] < 2:
        return 0.0
    
    try:
        predictions_log = model.predict(X_test)
        predictions_orig = safe_power10(predictions_log)
        y_test_flat = y_test.flatten()
        
        if len(predictions_orig) < 2 or len(y_test_flat) < 2:
            return 0.0
            
        correlation, _ = pearsonr(predictions_orig, y_test_flat)
        return correlation if not np.isnan(correlation) else 0.0
    except Exception as e:
        logger.error(f"计算测试集皮尔逊相关系数时出错: {e}")
        return 0.0

def evaluate_cnn_performance(model, data_manager, results_dir, iteration, X_test=None, y_test_orig=None, should_generate_plots=True):
    """评估CNN性能"""
    logger.info(f"评估CNN性能 (迭代 {iteration})...")
    metrics = {}
    
    # 训练集评估
    X_train = data_manager.get_all_x_orig()
    y_train_orig = data_manager.get_all_y_orig().flatten()
    
    train_predictions_log = model.predict(X_train)
    train_predictions_orig = safe_power10(train_predictions_log)
    
    if should_generate_plots:
        train_mse, train_rmse, train_mae = plot_prediction_vs_truth(
            train_predictions_orig, y_train_orig, results_dir, 
            iteration=iteration, 
            filename=f"train_prediction_vs_truth_iter_{iteration}.png"
        )
        plot_residuals(
            train_predictions_orig, y_train_orig, results_dir, 
            iteration=iteration,
            filename=f"train_residual_analysis_iter_{iteration}.png"
        )
    else:
        train_mse = np.mean((train_predictions_orig - y_train_orig) ** 2)
        train_rmse = np.sqrt(train_mse)
        train_mae = np.mean(np.abs(train_predictions_orig - y_train_orig))
        
    metrics.update({
        'train_mse': train_mse, 'train_rmse': train_rmse, 'train_mae': train_mae,
        'train_sample_count': len(y_train_orig)
    })
    
    # 测试集评估
    if X_test is not None and y_test_orig is not None and X_test.shape[0] > 0:
        test_predictions_log = model.predict(X_test)
        test_predictions_orig = safe_power10(test_predictions_log)
        test_ground_truth_orig = y_test_orig.flatten()
        
        if should_generate_plots:
            test_mse, test_rmse, test_mae = plot_prediction_vs_truth(
                test_predictions_orig, test_ground_truth_orig, results_dir, 
                iteration=iteration,
                filename=f"test_prediction_vs_truth_iter_{iteration}.png"
            )
            plot_residuals(
                test_predictions_orig, test_ground_truth_orig, results_dir, 
                iteration=iteration,
                filename=f"test_residual_analysis_iter_{iteration}.png"
            )
        else:
            test_mse = np.mean((test_predictions_orig - test_ground_truth_orig) ** 2)
            test_rmse = np.sqrt(test_mse)
            test_mae = np.mean(np.abs(test_predictions_orig - test_ground_truth_orig))
            
        if len(test_predictions_orig) >= 2 and len(test_ground_truth_orig) >= 2:
            test_pearson, _ = pearsonr(test_predictions_orig, test_ground_truth_orig)
            test_pearson = 0.0 if np.isnan(test_pearson) else test_pearson
        else:
            test_pearson = 0.0
        metrics.update({
            'test_mse': test_mse, 'test_rmse': test_rmse, 'test_mae': test_mae,
            'test_pearson': test_pearson, 'test_sample_count': len(test_ground_truth_orig)
        })
    
    logger.info(f"CNN性能 (迭代{iteration}): 训练样本: {metrics.get('train_sample_count')}, 训练RMSE: {metrics.get('train_rmse'):.4e}")
    if 'test_rmse' in metrics:
        logger.info(f"  测试样本: {metrics.get('test_sample_count')}, 测试RMSE: {metrics.get('test_rmse'):.4e}, 测试Pearson: {metrics.get('test_pearson'):.4f}")
    return metrics

def calculate_pearson_on_best_samples(model, data_manager, n_best=60):
    """计算最优样本上的皮尔逊相关系数"""
    all_X = data_manager.get_all_x_orig()
    all_y_orig = data_manager.get_all_y_orig()
    
    sorted_indices = np.argsort(all_y_orig.flatten())
    sorted_X = all_X[sorted_indices[:n_best]]
    sorted_y_orig = all_y_orig[sorted_indices[:n_best]]
    
    if sorted_X.shape[0] < 2:
        return 0.0, float('nan'), float('nan')

    min_y_orig_for_range = np.min(sorted_y_orig)
    max_y_orig_for_range = np.max(sorted_y_orig)

    try:
        predictions_log = model.predict(sorted_X)
        predictions_orig = safe_power10(predictions_log)
        
        if np.any(np.isinf(predictions_orig)) or np.any(np.isnan(predictions_orig)):
            logger.warning("预测值中存在无穷大或NaN值")
            return 0.0, min_y_orig_for_range, max_y_orig_for_range
            
    except Exception as e:
        logger.warning(f"计算预测值时出错: {e}")
        return 0.0, min_y_orig_for_range, max_y_orig_for_range
    
    true_values_orig = sorted_y_orig.flatten()
    
    if len(predictions_orig) < 2 or len(true_values_orig) < 2:
        return 0.0, min_y_orig_for_range, max_y_orig_for_range
    
    if np.var(predictions_orig) == 0 or np.var(true_values_orig) == 0:
        logger.warning("预测值或真实值没有变化，无法计算相关系数")
        return 0.0, min_y_orig_for_range, max_y_orig_for_range
        
    try:
        correlation, _ = pearsonr(predictions_orig, true_values_orig)
        if np.isnan(correlation) or np.isinf(correlation):
            return 0.0, min_y_orig_for_range, max_y_orig_for_range
    except Exception as e:
        logger.warning(f"计算Pearson相关系数时出错: {e}")
        return 0.0, min_y_orig_for_range, max_y_orig_for_range
    
    return (correlation if not np.isnan(correlation) else 0.0), min_y_orig_for_range, max_y_orig_for_range

class NTESurrogateWrapper:
    """NTE搜索器的代理模型包装器"""
    def __init__(self, actual_surrogate_model, data_manager_instance):
        self.surrogate_model = actual_surrogate_model
        self.dm = data_manager_instance
        self.logger = logging.getLogger(self.__class__.__name__)

    def predict(self, X_batch_orig: np.ndarray) -> np.ndarray:
        if X_batch_orig.shape[0] == 0:
            return np.array([])
        if X_batch_orig.ndim == 1:
             X_batch_orig = X_batch_orig.reshape(1, -1)

        try:
            pred_log = self.surrogate_model.predict(X_batch_orig)
            
            if X_batch_orig.shape[0] == 1:
                return pred_log[0] if isinstance(pred_log, np.ndarray) else pred_log
            else:
                return pred_log
        except Exception as e:
            self.logger.error(f"代理模型预测错误: {e}")
            
            # 出错时返回保守的高值
            if X_batch_orig.shape[0] == 1:
                return np.log10(1e6 + 1.0)
            else:
                return np.ones(X_batch_orig.shape[0]) * np.log10(1e6 + 1.0)

def save_iteration_samples_to_csv(X_batch, y_batch, results_dir, iteration):
    """保存迭代样本到CSV文件"""
    try:
        csv_filename = os.path.join(results_dir, f"iteration_{iteration}_samples.csv")
        
        data_dict = {}
        for i in range(X_batch.shape[1]):
            data_dict[f'x{i+1}'] = X_batch[:, i]
        data_dict['y_true'] = y_batch.flatten()
        
        df = pd.DataFrame(data_dict)
        df.to_csv(csv_filename, index=False)
        
        logger.info(f"迭代{iteration}的{len(y_batch)}个样本已保存到: {csv_filename}")
        
    except Exception as e:
        logger.error(f"保存样本到CSV失败: {e}")

logger.info("主执行逻辑核心功能加载完成")
print("完整的main.py核心功能已迁移完成")


In [ ]:
# Cell 4: CNN 1D 代理模型 (cnn_1d_surrogate.py)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

class CNN1DSurrogate:
    def __init__(self, input_dim, log_transform=True):
        """
        初始化CNN 1D替代模型
        参数:
            input_dim: 输入维度 (Rosenbrock函数为20维)
            log_transform: 是否使用对数变换预处理
        """
        self.input_dim = input_dim
        
        # 配置TensorFlow性能优化
        performance_optimizer.configure_tensorflow_performance()
        
        self.model = self._build_cnn_model()
        self.scaler = StandardScaler()
        self.log_transform = log_transform
        self.trained = False
        
        # 添加数据缓存
        self._data_cache_key = None
        self._cached_dataset = None
    
    def _build_cnn_model(self):
        """构建CNN 1D模型结构"""
        model = Sequential([
            # 输入层形状: (dims, 1)
            Conv1D(filters=128, kernel_size=3, strides=1, padding='same', 
                   activation='elu', input_shape=(self.input_dim, 1)),
            MaxPooling1D(pool_size=2),
            Dropout(0.2),
            
            Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='elu'),
            MaxPooling1D(pool_size=2),
            Dropout(0.2),
            
            Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='elu'),
            Conv1D(filters=16, kernel_size=3, strides=1, padding='same', activation='elu'),
            MaxPooling1D(pool_size=2),
            
            Conv1D(filters=8, kernel_size=3, strides=1, padding='same', activation='elu'),
            Conv1D(filters=4, kernel_size=3, strides=1, padding='same', activation='elu'),
            
            Flatten(),
            Dense(64, activation='elu'),
            Dense(1, activation='linear')
        ])
        
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return model
    
    def train(self, data_manager, epochs=200, batch_size=32, validation_split=0.2, patience=30, verbose=0, save_path=None):
        """
        使用DataManager中的数据训练CNN模型，集成性能优化。
        Args:
            data_manager (DataManager): 数据管理器对象，包含训练数据
            epochs (int): 训练轮数
            batch_size (int): 批次大小
            validation_split (float): 验证集比例
            patience (int): 早停耐心值
            verbose (int): 训练进度显示级别
            save_path (str, optional): 模型保存路径
        Returns:
            dict: 包含训练历史信息的字典
        """
        # 监控训练开始时的性能
        performance_optimizer.monitor_performance("训练开始")
        
        # 获取训练数据 - 使用log尺度的数据，避免重复变换
        X_train = data_manager.get_all_x_orig()
        y_train = data_manager.get_all_y_log()  # 使用已经log变换的数据
        
        # 如果数据是二维的，确保y_train是一维数组
        if y_train.ndim > 1:
            y_train = y_train.flatten()
        
        logger.info(f"训练数据形状: X={X_train.shape}, y={y_train.shape}")
        
        # 检查是否可以使用缓存的预处理数据
        data_cache_key = f"preprocessed_{hash(X_train.tobytes())}_{self.input_dim}"
        cached_data = performance_optimizer.get_cached_data(data_cache_key)
        
        if cached_data is not None:
            logger.info("使用缓存的预处理数据")
            X_train_scaled = cached_data
        else:
            # 标准化输入数据
            logger.info("进行数据标准化...")
            self.scaler.fit(X_train)
            X_train_scaled = self.scaler.transform(X_train)
            
            # 缓存预处理后的数据
            performance_optimizer.cache_data(data_cache_key, X_train_scaled)
            logger.info("已缓存预处理数据")
        
        # 调整输入形状为CNN 1D所需的 (samples, dims, 1)
        X_train_scaled = X_train_scaled.reshape(-1, self.input_dim, 1)
        
        # 使用优化的数据集
        dataset_cache_key = f"dataset_{data_cache_key}_{batch_size}_{validation_split}"
        
        if self._cached_dataset is not None and self._data_cache_key == dataset_cache_key:
            logger.info("使用缓存的数据集")
            train_dataset = self._cached_dataset
        else:
            logger.info("创建优化的训练数据集...")
            
            # 分割训练和验证数据
            n_samples = len(X_train_scaled)
            n_val = int(n_samples * validation_split)
            n_train = n_samples - n_val
            
            # 使用tf.data创建优化的数据集
            train_dataset = performance_optimizer.create_optimized_dataset(
                X_train_scaled[:n_train], 
                y_train[:n_train], 
                batch_size=batch_size, 
                shuffle=True, 
                cache=True
            )
            
            val_dataset = performance_optimizer.create_optimized_dataset(
                X_train_scaled[n_train:], 
                y_train[n_train:], 
                batch_size=batch_size, 
                shuffle=False, 
                cache=True
            )
            
            # 缓存数据集
            self._cached_dataset = (train_dataset, val_dataset)
            self._data_cache_key = dataset_cache_key
            logger.info("已缓存优化的数据集")
        
        if isinstance(self._cached_dataset, tuple):
            train_dataset, val_dataset = self._cached_dataset
        else:
            train_dataset = self._cached_dataset
            val_dataset = None
        
        # 设置早停回调
        early_stopping = EarlyStopping(monitor='val_loss', patience=patience, 
                                       restore_best_weights=True, verbose=1)
        
        # 监控训练前的性能
        performance_optimizer.monitor_performance("模型训练前")
        
        # 训练模型 - 使用优化的数据集
        if val_dataset is not None:
            history = self.model.fit(
                train_dataset,
                validation_data=val_dataset,
                epochs=epochs, 
                callbacks=[early_stopping], 
                verbose=verbose
            )
        else:
            # 回退到原始方法
            history = self.model.fit(
                X_train_scaled, y_train, 
                epochs=epochs, 
                batch_size=batch_size, 
                validation_split=validation_split, 
                callbacks=[early_stopping], 
                verbose=verbose
            )
        
        self.trained = True
        
        # 监控训练后的性能
        performance_optimizer.monitor_performance("模型训练后")
        
        # 保存模型（如果指定了路径）
        if save_path:
            self.model.save(save_path)
            
        # 清理不必要的内存
        performance_optimizer.optimize_memory_usage()
            
        return history
    
    def predict(self, states):
        """
        使用训练好的CNN 1D模型预测状态值
        参数:
            states: 需要预测的状态向量或状态向量列表
        返回:
            预测的函数值 (log尺度)
        """
        if not self.trained:
            raise RuntimeError("模型尚未训练，请先训练模型")
            
        # 确保输入是numpy数组
        states = np.array(states)
        
        # 输入可能是单个状态或多个状态
        single_input = False
        if states.ndim == 1:
            states = states.reshape(1, -1)
            single_input = True
            
        # 标准化输入
        states_scaled = self.scaler.transform(states)
        
        # 调整形状为CNN 1D所需的 (samples, dims, 1)
        states_scaled = states_scaled.reshape(-1, self.input_dim, 1)
        
        # 模型预测 - 返回log尺度的预测值
        predictions = self.model.predict(states_scaled, verbose=0)
        
        # 不进行逆变换，直接返回log尺度的预测值
        return predictions[0][0] if single_input else predictions.flatten()
    
    def visualize_training_history(self, history, save_path=None):
        """
        可视化训练历史
        参数:
            history: 训练返回的历史对象
            save_path: 图表保存路径
        """
        plt.figure(figsize=(12, 5))
        
        # 损失曲线
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Loss Curves')
        plt.xlabel('Epoch')
        plt.ylabel('Loss (MSE)')
        plt.legend()
        
        # MAE曲线
        plt.subplot(1, 2, 2)
        plt.plot(history.history['mae'], label='Training MAE')
        plt.plot(history.history['val_mae'], label='Validation MAE')
        plt.title('MAE Curves')
        plt.xlabel('Epoch')
        plt.ylabel('Mean Absolute Error')
        plt.legend()
        
        plt.tight_layout()
        
        if save_path:
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path)
            
        plt.show()

logger.info("CNN1DSurrogate模型类加载完成")
print("CNN 1D代理模型已准备就绪")


In [ ]:
# Cell 5: NTE搜索器 (nte_searcher.py) - 重点调试模块
import os
import sys
import numpy as np
import random
import logging
import math
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Any, Callable, Optional, Union

# Node数据类
@dataclass
class Node:
    state: Tuple[float, ...]  # 状态向量 (tuple for hashability)
    value_pred: float = float('inf')  # 替代模型预测 (v_ML)
    visit_count: int = 0  # 访问次数

class NTESearcher:
    """
    实现NTE (Neural-Surrogate-Guided Tree Exploration) 搜索策略
    重点用于调试动态探索因子参数
    """
    def __init__(self,
                 dimension: int,
                 domain: List[Tuple[float, float]],
                 c0: float = NTE_C0,
                 rollout_rounds: int = NTE_ROLLOUT_ROUNDS,
                 validation_batch_size: int = 20):
        """
        初始化NTE搜索器
        Args:
            dimension (int): 状态向量维度
            domain (List[Tuple[float, float]]): 每个维度的范围 [(min1, max1), ...]
            c0 (float): DUCB公式中的探索超参数
            rollout_rounds (int): 每次搜索调用的内部探索轮数
            validation_batch_size (int): 每次搜索后选择的候选数量
        """
        self.dimension = dimension
        self.domain = domain
        self.domain_mins = np.array([d[0] for d in domain])
        self.domain_maxs = np.array([d[1] for d in domain])
        self.base_c0 = c0  # 存储基础 c0 值
        self.rollout_rounds = rollout_rounds
        self.validation_batch_size = validation_batch_size
        
        # 初始测试范围
        self.initial_range_length = self.domain_maxs - self.domain_mins
        
        logger.info(f"NTESearcher initialized: dim={dimension}, c0={c0}, rollout_rounds={rollout_rounds}")

    def _calculate_dynamic_search_space(self, all_states: np.ndarray = None, all_values: np.ndarray = None, iteration: int = 1):
        """
        根据当前数据集中y值最小的20个样本计算动态搜索区间
        Args:
            all_states: 所有已知状态
            all_values: 所有已知状态对应的函数值
            iteration: 当前迭代次数，用于计算拓展区间
        Returns:
            Tuple[np.ndarray, np.ndarray]: 返回动态搜索区间的上下界
        """
        # 默认使用初始搜索区间
        dynamic_mins = self.domain_mins.copy()
        dynamic_maxs = self.domain_maxs.copy()
        
        # 如果没有提供状态和值，返回原始区间
        if all_states is None or all_values is None or all_states.shape[0] == 0:
            return dynamic_mins, dynamic_maxs
            
        try:
            # 选择y值最小的20个样本
            num_samples_to_use = min(20, all_values.shape[0])
            sorted_indices = np.argsort(all_values.flatten())[:num_samples_to_use]
            best_samples = all_states[sorted_indices]
            
            # 计算这些样本在各个维度上的最小值和最大值
            sample_mins = np.min(best_samples, axis=0)
            sample_maxs = np.max(best_samples, axis=0)
            
            # 计算基本区间长度l_i和区间中心a_i
            interval_lengths = sample_maxs - sample_mins
            interval_centers = (sample_maxs + sample_mins) / 2.0
            
            # 计算拓展区间大小d，随迭代次数递减
            extension = self.initial_range_length / (2 * max(1, iteration))
            
            # 确保最小区间长度
            min_interval_length = 3.0
            final_intervals = np.maximum(interval_lengths + 2 * extension, min_interval_length)
            
            # 计算动态搜索区间
            dynamic_mins = np.maximum(interval_centers - final_intervals / 2.0, self.domain_mins)
            dynamic_maxs = np.minimum(interval_centers + final_intervals / 2.0, self.domain_maxs)
            
            return dynamic_mins, dynamic_maxs
            
        except Exception as e:
            logger.error(f"计算动态搜索区间时出错: {str(e)}")
            return self.domain_mins.copy(), self.domain_maxs.copy()

    def _mixed_expansion(self, state_vector: np.ndarray, dynamic_factor: float = 1.0, 
                        all_states: np.ndarray = None, all_values: np.ndarray = None, 
                        iteration: int = 1, pre_computed_bounds: tuple = None) -> List[np.ndarray]:
        """
        混合膨胀策略：结合确定性和随机方法生成叶节点
        """
        if pre_computed_bounds is not None:
            dynamic_mins, dynamic_maxs = pre_computed_bounds
        else:
            dynamic_mins, dynamic_maxs = self._calculate_dynamic_search_space(all_states, all_values, iteration)
        
        # 基础步长：根据动态因子和搜索区间调整
        base_step_size = 0.1 * (dynamic_maxs - dynamic_mins) * dynamic_factor
        
        leaf_states = []
        
        # 1. 确定性移动 (4个方向，每个维度随机选择)
        num_deterministic = 4
        for _ in range(num_deterministic):
            # 随机选择一个维度
            dim_idx = random.randint(0, self.dimension - 1)
            direction = random.choice([-1, 1])
            
            new_state = state_vector.copy()
            step_size = base_step_size[dim_idx]
            new_state[dim_idx] += direction * step_size
            
            # 确保在域范围内
            new_state = np.clip(new_state, dynamic_mins, dynamic_maxs)
            leaf_states.append(new_state)
        
        # 2. 随机膨胀 (4个随机点)
        num_random = 4
        for _ in range(num_random):
            # 在当前状态附近生成随机点
            noise = np.random.normal(0, base_step_size * 0.5, self.dimension)
            new_state = state_vector + noise
            
            # 确保在域范围内
            new_state = np.clip(new_state, dynamic_mins, dynamic_maxs)
            leaf_states.append(new_state)
        
        return leaf_states

    def _calculate_ducb(self, node: Node, parent_total_visits: int, c_rho: float, current_c0: float) -> float:
        """
        计算DUCB (Discriminative Upper Confidence Bound) 值
        核心公式：DUCB = -pred_value + c0 * c(rho) * sqrt(ln(N)/n)
        """
        if node.visit_count == 0:
            return float('inf')  # 未访问的节点优先级最高
        
        # 探索项
        exploration_term = current_c0 * c_rho * math.sqrt(math.log(max(1, parent_total_visits)) / node.visit_count)
        
        # DUCB = 负预测值 + 探索项 (因为我们要最小化)
        ducb_value = -node.value_pred + exploration_term
        
        return ducb_value

    def _select_candidates_mixed_strategy(self, visited_nodes: Dict[Tuple[float, ...], Node]) -> List[np.ndarray]:
        """
        混合策略选择候选点：结合DUCB值和预测值
        """
        if not visited_nodes:
            return []
        
        nodes_list = list(visited_nodes.values())
        
        # 按预测值排序，选择前15个最优点
        nodes_by_pred = sorted(nodes_list, key=lambda n: n.value_pred)
        best_by_pred = nodes_by_pred[:min(15, len(nodes_by_pred))]
        
        # 计算总访问次数
        total_visits = sum(node.visit_count for node in nodes_list)
        
        # 按DUCB值排序，选择前5个高DUCB点
        c_rho = 1.0  # 简化的c_rho值
        current_c0 = self.base_c0
        
        nodes_with_ducb = []
        for node in nodes_list:
            ducb_val = self._calculate_ducb(node, total_visits, c_rho, current_c0)
            nodes_with_ducb.append((node, ducb_val))
        
        nodes_with_ducb.sort(key=lambda x: x[1], reverse=True)  # 按DUCB降序
        best_by_ducb = [item[0] for item in nodes_with_ducb[:5]]
        
        # 合并并去重
        selected_nodes = best_by_pred + best_by_ducb
        unique_states = []
        seen_states = set()
        
        for node in selected_nodes:
            if node.state not in seen_states:
                unique_states.append(np.array(node.state))
                seen_states.add(node.state)
        
        return unique_states[:self.validation_batch_size]

    def search(self,
               surrogate_model: Any, 
               current_best_state: np.ndarray,
               min_y_observed: float,
               dynamic_exploration_factor: float = 1.0,
               all_states: np.ndarray = None, 
               all_values: np.ndarray = None,
               previous_iteration_samples: np.ndarray = None,
               current_iteration: int = 1,
               return_ducb_trends: bool = False) -> Union[List[np.ndarray], Tuple[List[np.ndarray], List[Dict]]]:
        """
        改进版的NTE搜索算法
        
        重点调试参数：
        - dynamic_exploration_factor: 动态探索因子，影响搜索的探索强度
        
        Args:
            surrogate_model: 替代模型，必须有predict方法
            current_best_state: 当前已知最优状态
            min_y_observed: 当前已知最小值
            dynamic_exploration_factor: 动态探索因子 (重点调试参数)
            all_states: 所有已知状态点
            all_values: 所有已知状态值
            previous_iteration_samples: 上一次迭代选择的样本
            current_iteration: 当前迭代次数
            return_ducb_trends: 是否返回DUCB趋势数据
        """
        logger.info(f"NTE搜索开始: 迭代{current_iteration}, 动态探索因子={dynamic_exploration_factor:.2f}")
        
        # DUCB趋势数据收集
        ducb_trends_data = [] if return_ducb_trends else None
        
        # 动态调整c0
        current_c0 = self.base_c0 * dynamic_exploration_factor
        c_rho = 1.0  # 简化的rho调整因子
        
        logger.info(f"调整后的c0值: {current_c0:.2f} (基础c0={self.base_c0}, 动态因子={dynamic_exploration_factor:.2f})")
        
        # 选择起始点
        best_states_for_nte = []
        if all_states is not None and all_values is not None and all_states.shape[0] > 0:
            # 选择2个最优点
            num_best = min(2, all_states.shape[0])
            sorted_indices = np.argsort(all_values.flatten())
            for idx in sorted_indices[:num_best]:
                best_states_for_nte.append(all_states[idx])
            
            # 选择1个随机点
            if all_states.shape[0] > num_best:
                remaining_indices = sorted_indices[num_best:]
                random_idx = np.random.choice(remaining_indices, size=1)[0]
                best_states_for_nte.append(all_states[random_idx])
        else:
            # 使用当前最优状态或生成随机状态
            if current_best_state is not None:
                best_states_for_nte.append(current_best_state)
            else:
                random_state = np.array([random.uniform(self.domain_mins[d], self.domain_maxs[d]) 
                                       for d in range(self.dimension)])
                best_states_for_nte.append(random_state)
        
        logger.info(f"使用 {len(best_states_for_nte)} 个起始点进行NTE搜索")
        
        # 计算动态搜索区间
        try:
            dynamic_mins, dynamic_maxs = self._calculate_dynamic_search_space(all_states, all_values, current_iteration)
            pre_computed_bounds = (dynamic_mins, dynamic_maxs)
            
            original_range = np.mean(self.domain_maxs - self.domain_mins)
            dynamic_range = np.mean(dynamic_maxs - dynamic_mins)
            logger.info(f"搜索区间: 原始范围={original_range:.2f}, 动态范围={dynamic_range:.2f}")
        except Exception as e:
            logger.error(f"计算动态搜索区间失败: {e}")
            pre_computed_bounds = None
        
        # 对每个起始点进行NTE搜索
        all_candidates = []
        
        for nte_id, root_state_vector in enumerate(best_states_for_nte):
            visited_nodes: Dict[Tuple[float, ...], Node] = {}
            root_state_tuple = tuple(root_state_vector)
            
            # 初始化根节点
            try:
                root_pred_value = surrogate_model.predict(np.array([root_state_vector]))
                if isinstance(root_pred_value, (np.ndarray, list)):
                    root_pred_value = float(root_pred_value.item() if hasattr(root_pred_value, 'item') else root_pred_value[0])
                else:
                    root_pred_value = float(root_pred_value)
                
                if np.isnan(root_pred_value) or np.isinf(root_pred_value):
                    root_pred_value = float('inf')
            except Exception as e:
                logger.error(f"预测根节点值错误: {e}")
                root_pred_value = float('inf')
            
            root_node = Node(state=root_state_tuple, value_pred=root_pred_value, visit_count=0)
            visited_nodes[root_state_tuple] = root_node
            
            logger.debug(f"起始点 {nte_id+1}: 预测值={root_pred_value:.4e}")
            
            # Rollout阶段
            for rollout_round in range(self.rollout_rounds):
                current_root_node = visited_nodes[root_state_tuple]
                current_root_node.visit_count += 1
                current_root_state_vector = np.array(current_root_node.state)
                
                # 生成叶节点
                leaf_states = self._mixed_expansion(
                    current_root_state_vector, 
                    dynamic_exploration_factor,
                    all_states, 
                    all_values, 
                    current_iteration,
                    pre_computed_bounds=pre_computed_bounds
                )
                
                # 预测叶节点值
                try:
                    leaf_states_batch = np.array(leaf_states)
                    leaf_pred_values = surrogate_model.predict(leaf_states_batch)
                    if not isinstance(leaf_pred_values, np.ndarray):
                        leaf_pred_values = np.array(leaf_pred_values)
                    
                    # 处理预测值
                    leaf_pred_values = np.nan_to_num(leaf_pred_values, nan=float('inf'), 
                                                   posinf=float('inf'), neginf=float('inf'))
                except Exception as e:
                    logger.error(f"叶节点预测失败: {e}")
                    leaf_pred_values = np.full(len(leaf_states), float('inf'))
                
                # 添加叶节点到visited_nodes
                for leaf_state, pred_value in zip(leaf_states, leaf_pred_values):
                    leaf_tuple = tuple(leaf_state)
                    if leaf_tuple not in visited_nodes:
                        visited_nodes[leaf_tuple] = Node(state=leaf_tuple, value_pred=float(pred_value), visit_count=0)
                
                # 收集DUCB趋势数据
                if return_ducb_trends and rollout_round % 10 == 0:  # 每10轮记录一次
                    total_visits = sum(node.visit_count for node in visited_nodes.values())
                    ducb_values = []
                    for node in visited_nodes.values():
                        ducb_val = self._calculate_ducb(node, total_visits, c_rho, current_c0)
                        ducb_values.append(ducb_val)
                    
                    ducb_trends_data.append({
                        'nte_id': nte_id,
                        'rollout_round': rollout_round,
                        'max_ducb': max(ducb_values) if ducb_values else 0,
                        'min_ducb': min(ducb_values) if ducb_values else 0,
                        'avg_ducb': np.mean(ducb_values) if ducb_values else 0,
                        'num_nodes': len(visited_nodes),
                        'dynamic_factor': dynamic_exploration_factor,
                        'current_c0': current_c0
                    })
            
            # 选择候选点
            nte_candidates = self._select_candidates_mixed_strategy(visited_nodes)
            all_candidates.extend(nte_candidates)
            
            logger.info(f"NTE {nte_id+1} 完成: 访问了{len(visited_nodes)}个节点, 选择了{len(nte_candidates)}个候选点")
        
        # 去重并限制数量
        final_candidates = []
        seen_states = set()
        
        for candidate in all_candidates:
            candidate_tuple = tuple(candidate)
            if candidate_tuple not in seen_states:
                final_candidates.append(candidate)
                seen_states.add(candidate_tuple)
                
                if len(final_candidates) >= self.validation_batch_size:
                    break
        
        logger.info(f"NTE搜索完成: 总共返回{len(final_candidates)}个候选点")
        
        if return_ducb_trends:
            return final_candidates, ducb_trends_data
        else:
            return final_candidates

logger.info("NTESearcher类加载完成 - 重点调试模块已准备就绪")
print("NTE搜索器已准备就绪 - 动态探索因子可调试")


In [ ]:
# Cell 6: 可视化工具 (plot_utils.py) - 简化版
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import logging
import matplotlib
from scipy.stats import pearsonr

# 配置全局字体设置为默认英文字体 (避免Kaggle中文字体问题)
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']
plt.switch_backend('Agg')  # 使用Agg后端，适合无GUI环境

def create_results_directory(base_results_path):
    """创建带时间戳的结果目录"""
    try:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        results_dir = os.path.join(base_results_path, timestamp)
        os.makedirs(results_dir, exist_ok=True)
        logger.info(f"Results directory created: {results_dir}")
        return results_dir
    except Exception as e:
        logger.error(f"Error creating results directory: {e}")
        return None

def plot_global_min_value_trend(iterations, min_values, output_dir, filename="global_min_value_trend.png"):
    """绘制全局最小值趋势图"""
    if not os.path.exists(output_dir):
        logger.warning(f"Output directory {output_dir} does not exist. Skipping plot.")
        return

    try:
        # 确保数组长度相同
        if len(iterations) != len(min_values):
            min_length = min(len(iterations), len(min_values))
            iterations = iterations[:min_length]
            min_values = min_values[:min_length]
            
        plt.figure(figsize=(12, 6))
        plt.plot(iterations, min_values, marker='o', linestyle='-', color='b', linewidth=2)
        plt.title('Global Minimum Value Trend', fontsize=16)
        plt.xlabel('Iteration', fontsize=14)
        plt.ylabel('Current Minimum True Value Found', fontsize=14)
        plt.grid(True, alpha=0.3)
        
        # 只在找到更低全局最小值时添加标注
        min_values_array = np.array(min_values)
        global_min = float('inf')
        
        for i, (iter_num, current_min) in enumerate(zip(iterations, min_values_array)):
            if current_min < global_min:
                global_min = current_min
                label_txt = f"{current_min:.3e}" if current_min >= 1000 else f"{current_min:.4f}"
                plt.annotate(label_txt, 
                            (iter_num, current_min),
                            textcoords="offset points", 
                            xytext=(0,10), 
                            ha='center',
                            fontsize=10,
                            bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="orange", alpha=0.8))

        save_path = os.path.join(output_dir, filename)
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        logger.info(f"全局最小值趋势图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制全局最小值趋势图出错: {e}")

def plot_prediction_vs_truth(predictions, ground_truth, output_dir, iteration=None, filename=None, title_suffix=None):
    """绘制预测值vs真值对比图"""
    if not os.path.exists(output_dir):
        return None, None, None
    
    try:
        # 展平数组
        predictions_flat = np.array(predictions).flatten()
        ground_truth_flat = np.array(ground_truth).flatten()
        
        # 计算指标
        mse = np.mean((predictions_flat - ground_truth_flat) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(predictions_flat - ground_truth_flat))
        
        # 绘图
        plt.figure(figsize=(10, 8))
        plt.scatter(ground_truth_flat, predictions_flat, alpha=0.6, s=30)
        
        # 添加对角线 (理想预测线)
        min_val = min(np.min(ground_truth_flat), np.min(predictions_flat))
        max_val = max(np.max(ground_truth_flat), np.max(predictions_flat))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
        
        plt.xlabel('Ground Truth', fontsize=14)
        plt.ylabel('Predictions', fontsize=14)
        title = f'Prediction vs Truth Comparison'
        if title_suffix:
            title += f' - {title_suffix}'
        if iteration is not None:
            title += f' (Iteration {iteration})'
        plt.title(title, fontsize=16)
        
        # 添加指标文本
        plt.text(0.05, 0.95, f'RMSE: {rmse:.4e}\nMAE: {mae:.4e}\nMSE: {mse:.4e}', 
                transform=plt.gca().transAxes, fontsize=12, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        if filename is None:
            filename = f"prediction_vs_truth_iter_{iteration}.png" if iteration else "prediction_vs_truth.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        logger.info(f"预测vs真值图已保存: {save_path}")
        return mse, rmse, mae
    except Exception as e:
        logger.error(f"绘制预测vs真值图出错: {e}")
        return None, None, None

def plot_residuals(predictions, ground_truth, output_dir, iteration=None, filename=None):
    """绘制残差分析图"""
    if not os.path.exists(output_dir):
        return
    
    try:
        predictions_flat = np.array(predictions).flatten()
        ground_truth_flat = np.array(ground_truth).flatten()
        residuals = predictions_flat - ground_truth_flat
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # 残差散点图
        ax1.scatter(ground_truth_flat, residuals, alpha=0.6, s=30)
        ax1.axhline(y=0, color='r', linestyle='--', linewidth=2)
        ax1.set_xlabel('Ground Truth', fontsize=12)
        ax1.set_ylabel('Residuals (Predicted - Truth)', fontsize=12)
        ax1.set_title('Residuals vs Ground Truth', fontsize=14)
        ax1.grid(True, alpha=0.3)
        
        # 残差直方图
        ax2.hist(residuals, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        ax2.axvline(x=0, color='r', linestyle='--', linewidth=2)
        ax2.set_xlabel('Residuals', fontsize=12)
        ax2.set_ylabel('Frequency', fontsize=12)
        ax2.set_title('Residuals Distribution', fontsize=14)
        ax2.grid(True, alpha=0.3)
        
        # 添加统计信息
        residual_std = np.std(residuals)
        residual_mean = np.mean(residuals)
        ax2.text(0.05, 0.95, f'Mean: {residual_mean:.4e}\nStd: {residual_std:.4e}', 
                transform=ax2.transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        if filename is None:
            filename = f"residual_analysis_iter_{iteration}.png" if iteration else "residual_analysis.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        logger.info(f"残差分析图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制残差分析图出错: {e}")

def plot_pearson_correlation_trend(iterations, pearson_coeffs, output_dir, filename="pearson_correlation_trend.png", title=None):
    """绘制皮尔逊相关系数趋势图"""
    if not os.path.exists(output_dir):
        return

    try:
        plt.figure(figsize=(12, 6))
        plt.plot(iterations, pearson_coeffs, marker='o', linestyle='-', color='g', linewidth=2, markersize=6)
        plt.title(title if title else 'Pearson Correlation Coefficient Trend (Model vs. Truth)', fontsize=16)
        plt.xlabel('Iteration', fontsize=14)
        plt.ylabel('Pearson Correlation Coefficient', fontsize=14)
        plt.ylim(-1.1, 1.1)
        plt.grid(True, alpha=0.3)
        
        # 添加水平参考线
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
        plt.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='r=0.5')
        plt.axhline(y=0.8, color='blue', linestyle='--', alpha=0.5, label='r=0.8')
        
        # 在关键点添加数值标注
        for i in range(0, len(iterations), max(1, len(iterations)//10)):  # 每10%的点标注一次
            plt.annotate(f'{pearson_coeffs[i]:.3f}', 
                        (iterations[i], pearson_coeffs[i]),
                        textcoords="offset points", 
                        xytext=(0,10), 
                        ha='center',
                        fontsize=9)
        
        plt.legend()
        save_path = os.path.join(output_dir, filename)
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        logger.info(f"皮尔逊相关系数趋势图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制皮尔逊相关系数趋势图出错: {e}")

def plot_nte_ducb_trends(ducb_trends_data, output_dir, iteration, dynamic_exploration_factor, 
                        new_global_best_value=None, filename=None):
    """绘制NTE搜索的DUCB趋势图"""
    if not ducb_trends_data or not os.path.exists(output_dir):
        return
    
    try:
        plt.figure(figsize=(15, 10))
        
        # 提取数据
        rollout_rounds = [d['rollout_round'] for d in ducb_trends_data]
        max_ducb_values = [d['max_ducb'] for d in ducb_trends_data]
        avg_ducb_values = [d['avg_ducb'] for d in ducb_trends_data]
        num_nodes = [d['num_nodes'] for d in ducb_trends_data]
        
        # 创建子图
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. DUCB值趋势
        ax1.plot(rollout_rounds, max_ducb_values, 'r-', marker='o', label='Max DUCB', linewidth=2)
        ax1.plot(rollout_rounds, avg_ducb_values, 'b-', marker='s', label='Avg DUCB', linewidth=2)
        ax1.set_xlabel('Rollout Round')
        ax1.set_ylabel('DUCB Value')
        ax1.set_title(f'DUCB Values Trend (Factor: {dynamic_exploration_factor:.2f})')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. 节点数量增长
        ax2.plot(rollout_rounds, num_nodes, 'g-', marker='^', linewidth=2)
        ax2.set_xlabel('Rollout Round')
        ax2.set_ylabel('Number of Nodes Visited')
        ax2.set_title('Node Exploration Growth')
        ax2.grid(True, alpha=0.3)
        
        # 3. 动态因子影响分析
        current_c0_values = [d['current_c0'] for d in ducb_trends_data]
        ax3.plot(rollout_rounds, current_c0_values, 'm-', marker='d', linewidth=2)
        ax3.set_xlabel('Rollout Round')
        ax3.set_ylabel('Current C0 Value')
        ax3.set_title(f'Dynamic C0 Value (Base×Factor={dynamic_exploration_factor:.2f})')
        ax3.grid(True, alpha=0.3)
        
        # 4. 探索效率
        exploration_efficiency = [m/n if n > 0 else 0 for m, n in zip(max_ducb_values, num_nodes)]
        ax4.plot(rollout_rounds, exploration_efficiency, 'orange', marker='*', linewidth=2)
        ax4.set_xlabel('Rollout Round')
        ax4.set_ylabel('Max DUCB / Nodes Ratio')
        ax4.set_title('Exploration Efficiency')
        ax4.grid(True, alpha=0.3)
        
        # 添加全局信息
        if new_global_best_value is not None:
            fig.suptitle(f'NTE DUCB Analysis - Iteration {iteration} (New Best: {new_global_best_value:.4e})', 
                        fontsize=16, y=0.98)
        else:
            fig.suptitle(f'NTE DUCB Analysis - Iteration {iteration}', fontsize=16, y=0.98)
        
        if filename is None:
            filename = f"nte_ducb_trends_iter_{iteration}.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        logger.info(f"NTE DUCB趋势图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制NTE DUCB趋势图出错: {e}")

def plot_top_samples_dimensions(data_manager, output_dir, iteration, n_top_samples=15, filename=None):
    """绘制最优样本的维度分布图"""
    if not os.path.exists(output_dir):
        return
    
    try:
        all_X = data_manager.get_all_x_orig()
        all_y = data_manager.get_all_y_orig().flatten()
        
        # 选择最优的n个样本
        sorted_indices = np.argsort(all_y)[:n_top_samples]
        top_samples = all_X[sorted_indices]
        top_values = all_y[sorted_indices]
        
        # 计算每个维度的统计信息
        dim_means = np.mean(top_samples, axis=0)
        dim_stds = np.std(top_samples, axis=0)
        dim_mins = np.min(top_samples, axis=0)
        dim_maxs = np.max(top_samples, axis=0)
        
        dimensions = list(range(len(dim_means)))
        
        plt.figure(figsize=(15, 8))
        
        # 误差条图显示均值和标准差
        plt.errorbar(dimensions, dim_means, yerr=dim_stds, 
                    fmt='o', capsize=5, capthick=2, linewidth=2, markersize=8,
                    label=f'Mean ± Std (Top {n_top_samples} samples)')
        
        # 添加最小值最大值范围
        plt.fill_between(dimensions, dim_mins, dim_maxs, alpha=0.2, 
                        label=f'Min-Max Range')
        
        plt.xlabel('Dimension Index', fontsize=14)
        plt.ylabel('Dimension Value', fontsize=14)
        plt.title(f'Top {n_top_samples} Samples Dimension Distribution (Iteration {iteration})', fontsize=16)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 添加统计信息文本
        best_value = top_values[0]
        plt.text(0.02, 0.98, f'Best Value: {best_value:.4e}', 
                transform=plt.gca().transAxes, fontsize=12, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
        
        if filename is None:
            filename = f"top_samples_dimensions_iter_{iteration}.png"
        
        save_path = os.path.join(output_dir, filename)
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        logger.info(f"维度分布图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制维度分布图出错: {e}")

# 快速可视化函数 - 专为Kaggle环境优化
def quick_progress_plot(iterations, best_values, pearson_coeffs, output_dir, iteration):
    """快速生成进度概览图"""
    try:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # 最优值趋势
        ax1.plot(iterations, best_values, 'b-o', linewidth=2, markersize=6)
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Best Value Found')
        ax1.set_title('Optimization Progress')
        ax1.grid(True, alpha=0.3)
        ax1.set_yscale('log')  # 使用对数尺度
        
        # 相关系数趋势
        ax2.plot(iterations, pearson_coeffs, 'g-s', linewidth=2, markersize=6)
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Pearson Correlation')
        ax2.set_title('Model Quality')
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim(0, 1)
        
        plt.suptitle(f'DANTE Progress Overview - Iteration {iteration}', fontsize=16)
        plt.tight_layout()
        
        save_path = os.path.join(output_dir, f"progress_overview_iter_{iteration}.png")
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        logger.info(f"进度概览图已保存: {save_path}")
    except Exception as e:
        logger.error(f"绘制进度概览图出错: {e}")

logger.info("可视化工具模块加载完成")
print("可视化工具已准备就绪 - 优化后的Kaggle版本")


In [ ]:
# Cell 17: Kaggle环境适配和完整主逻辑函数

def setup_kaggle_environment():
    """设置Kaggle环境 - 检测和适配路径"""
    # 检测是否在Kaggle环境中
    if '/kaggle/' in os.getcwd():
        logger.info("检测到Kaggle环境")
        
        # Kaggle环境路径
        base_data_path = '/kaggle/input'  # 输入数据路径
        output_path = '/kaggle/working'   # 输出路径
        
        # 检查数据文件是否存在
        data_files = {
            'x_train': None,
            'y_train': None, 
            'x_test': None,
            'y_test': None
        }
        
        # 在Kaggle输入目录中查找数据文件
        if os.path.exists(base_data_path):
            for root, dirs, files in os.walk(base_data_path):
                for file in files:
                    if 'rosenbrock' in file.lower():
                        if 'x_train' in file.lower():
                            data_files['x_train'] = os.path.join(root, file)
                        elif 'y_train' in file.lower():
                            data_files['y_train'] = os.path.join(root, file)
                        elif 'x_test' in file.lower():
                            data_files['x_test'] = os.path.join(root, file)
                        elif 'y_test' in file.lower():
                            data_files['y_test'] = os.path.join(root, file)
        
        return output_path, data_files
    else:
        logger.info("本地环境")
        # 本地环境
        output_path = './dante_results'
        os.makedirs(output_path, exist_ok=True)
        
        data_files = {
            'x_train': 'rosenbrock/data_raw/Rosenbrock_x_train.npy',
            'y_train': 'rosenbrock/data_raw/Rosenbrock_y_train.npy',
            'x_test': 'rosenbrock/data_raw/Rosenbrock_x_test.npy',
            'y_test': 'rosenbrock/data_raw/Rosenbrock_y_test.npy'
        }
        
        return output_path, data_files

def load_initial_data(data_files, rosenbrock_dimension=20, initial_data_size=800, domain_bounds=(-2.048, 2.048)):
    """加载初始数据，支持Kaggle和本地环境"""
    logger.info("开始加载初始数据...")
    
    X_test, y_test = None, None
    
    try:
        # 尝试加载测试集
        if data_files['x_test'] and data_files['y_test']:
            if os.path.exists(data_files['x_test']) and os.path.exists(data_files['y_test']):
                X_test = np.load(data_files['x_test'])
                y_test = np.load(data_files['y_test'])
                if y_test.ndim == 1:
                    y_test = y_test.reshape(-1, 1)
                logger.info(f"成功加载测试数据: X shape {X_test.shape}, y shape {y_test.shape}")
        
        # 尝试加载训练集
        if data_files['x_train'] and data_files['y_train']:
            if os.path.exists(data_files['x_train']) and os.path.exists(data_files['y_train']):
                initial_X = np.load(data_files['x_train'])
                initial_y = np.load(data_files['y_train'])
                if initial_y.ndim == 1:
                    initial_y = initial_y.reshape(-1, 1)
                logger.info(f"成功加载训练数据: X shape {initial_X.shape}, y shape {initial_y.shape}")
                
                # 确保有足够的初始样本
                if initial_X.shape[0] < initial_data_size:
                    logger.warning(f"数据不足，需要生成额外样本")
                    additional_samples = initial_data_size - initial_X.shape[0]
                    additional_X = np.random.uniform(
                        low=domain_bounds[0], high=domain_bounds[1], 
                        size=(additional_samples, rosenbrock_dimension)
                    )
                    additional_y = np.array([rosenbrock_evaluator(x) for x in additional_X]).reshape(-1, 1)
                    initial_X = np.vstack((initial_X, additional_X))
                    initial_y = np.vstack((initial_y, additional_y))
                    logger.info(f"添加{additional_samples}个随机样本，总样本数: {initial_X.shape[0]}")
            else:
                raise FileNotFoundError("训练数据文件不存在")
        else:
            raise FileNotFoundError("未指定训练数据文件路径")
            
    except Exception as e:
        logger.warning(f"加载数据失败: {e}，生成随机数据")
        # 生成随机数据作为备用方案
        initial_X = np.random.uniform(
            low=domain_bounds[0], high=domain_bounds[1], 
            size=(initial_data_size, rosenbrock_dimension)
        )
        initial_y = np.array([rosenbrock_evaluator(x) for x in initial_X]).reshape(-1, 1)
        logger.info("生成随机初始数据")
        
        # 如果没有测试集，从随机数据中分离一部分作为测试集
        if X_test is None:
            test_size = min(200, initial_data_size // 4)
            indices = np.random.choice(initial_data_size, test_size, replace=False)
            X_test = initial_X[indices]
            y_test = initial_y[indices]
            
            # 从训练集中删除测试数据
            mask = np.ones(initial_data_size, dtype=bool)
            mask[indices] = False
            initial_X = initial_X[mask]
            initial_y = initial_y[mask]
            
            # 补充训练样本到原始大小
            if initial_X.shape[0] < initial_data_size:
                additional_samples = initial_data_size - initial_X.shape[0]
                additional_X = np.random.uniform(
                    low=domain_bounds[0], high=domain_bounds[1], 
                    size=(additional_samples, rosenbrock_dimension)
                )
                additional_y = np.array([rosenbrock_evaluator(x) for x in additional_X]).reshape(-1, 1)
                initial_X = np.vstack((initial_X, additional_X))
                initial_y = np.vstack((initial_y, additional_y))
            
            logger.info(f"从随机数据中创建测试集: {X_test.shape[0]}个样本")
    
    return initial_X, initial_y, X_test, y_test

logger.info("Kaggle环境适配模块加载完成")
print("环境适配完成，支持本地和Kaggle环境")


In [ ]:
# Cell 7: 主执行逻辑 (main.py核心) - DANTE优化算法
import time
import pandas as pd
from scipy.stats import pearsonr
import gc

# NTE搜索器包装类
class NTESurrogateWrapper:
    """替代模型包装类，适配NTE搜索器接口"""
    def __init__(self, actual_surrogate_model, data_manager_instance):
        self.surrogate_model = actual_surrogate_model
        self.dm = data_manager_instance
        self.logger = logging.getLogger(self.__class__.__name__)

    def predict(self, X_batch_orig: np.ndarray) -> np.ndarray:
        # 确保输入数据有效
        if X_batch_orig.shape[0] == 0:
            return np.array([])
        if X_batch_orig.ndim == 1:
             X_batch_orig = X_batch_orig.reshape(1, -1)

        try:
            # CNN1DSurrogate进行预测（直接返回log尺度的值）
            pred_log = self.surrogate_model.predict(X_batch_orig)
            
            # 返回与输入匹配的输出格式
            if X_batch_orig.shape[0] == 1:
                return pred_log[0] if isinstance(pred_log, np.ndarray) else pred_log
            else:
                return pred_log
        except Exception as e:
            self.logger.error(f"Prediction error in NTESurrogateWrapper: {e}")
            # 出错时返回保守的高值（log尺度）
            if X_batch_orig.shape[0] == 1:
                return np.log10(1e6 + 1.0)
            else:
                return np.ones(X_batch_orig.shape[0]) * np.log10(1e6 + 1.0)

def calculate_pearson_correlation(model, data_manager):
    """计算模型预测与真值的皮尔逊相关系数"""
    if data_manager.num_samples < 2:
        return 0.0
    
    try:
        X_data = data_manager.get_all_x_orig()
        y_data_orig = data_manager.get_all_y_orig().flatten()
        
        # CNN1DSurrogate返回log尺度预测值
        predictions_log = model.predict(X_data)
        predictions_orig = safe_power10(predictions_log)
        
        if len(predictions_orig) < 2 or len(y_data_orig) < 2:
            return 0.0
            
        correlation, _ = pearsonr(predictions_orig, y_data_orig)
        return correlation if not np.isnan(correlation) else 0.0
    
    except Exception as e:
        logger.error(f"计算皮尔逊相关系数时出错: {e}")
        return 0.0

def calculate_pearson_on_test_set(model, X_test, y_test_orig):
    """计算测试集上的皮尔逊相关系数"""
    if X_test.shape[0] < 2:
        return 0.0
    
    try:
        predictions_log = model.predict(X_test)
        predictions_orig = safe_power10(predictions_log)
        
        if len(predictions_orig) < 2 or len(y_test_orig) < 2:
            return 0.0
        correlation, _ = pearsonr(predictions_orig, y_test_orig.flatten())
        return correlation if not np.isnan(correlation) else 0.0
    except Exception as e:
        logger.error(f"计算测试集皮尔逊相关系数时出错: {e}")
        return 0.0

def evaluate_cnn_performance(model, data_manager, results_dir, iteration, X_test=None, y_test_orig=None, should_generate_plots=True):
    """评估CNN性能"""
    logger.info(f"Evaluating CNN performance (iteration {iteration})...")
    metrics = {}
    
    # 训练集评估
    X_train = data_manager.get_all_x_orig()
    y_train_orig = data_manager.get_all_y_orig().flatten()
    
    train_predictions_log = model.predict(X_train)
    train_predictions_orig = safe_power10(train_predictions_log)
    
    if should_generate_plots:
        train_mse, train_rmse, train_mae = plot_prediction_vs_truth(
            train_predictions_orig, y_train_orig, results_dir, 
            iteration=iteration, 
            filename=f"train_prediction_vs_truth_iter_{iteration}.png"
        )
        plot_residuals(
            train_predictions_orig, y_train_orig, results_dir, 
            iteration=iteration,
            filename=f"train_residual_analysis_iter_{iteration}.png"
        )
    else:
        train_mse = np.mean((train_predictions_orig - y_train_orig) ** 2)
        train_rmse = np.sqrt(train_mse)
        train_mae = np.mean(np.abs(train_predictions_orig - y_train_orig))
        
    metrics.update({
        'train_mse': train_mse, 'train_rmse': train_rmse, 'train_mae': train_mae,
        'train_sample_count': len(y_train_orig)
    })
    
    # 测试集评估
    if X_test is not None and y_test_orig is not None and X_test.shape[0] > 0:
        test_predictions_log = model.predict(X_test)
        test_predictions_orig = safe_power10(test_predictions_log)
        test_ground_truth_orig = y_test_orig.flatten()
        
        if should_generate_plots:
            test_mse, test_rmse, test_mae = plot_prediction_vs_truth(
                test_predictions_orig, test_ground_truth_orig, results_dir, 
                iteration=iteration,
                filename=f"test_prediction_vs_truth_iter_{iteration}.png"
            )
            plot_residuals(
                test_predictions_orig, test_ground_truth_orig, results_dir, 
                iteration=iteration,
                filename=f"test_residual_analysis_iter_{iteration}.png"
            )
        else:
            test_mse = np.mean((test_predictions_orig - test_ground_truth_orig) ** 2)
            test_rmse = np.sqrt(test_mse)
            test_mae = np.mean(np.abs(test_predictions_orig - test_ground_truth_orig))
            
        if len(test_predictions_orig) >= 2 and len(test_ground_truth_orig) >= 2:
            test_pearson, _ = pearsonr(test_predictions_orig, test_ground_truth_orig)
            test_pearson = 0.0 if np.isnan(test_pearson) else test_pearson
        else:
            test_pearson = 0.0
        metrics.update({
            'test_mse': test_mse, 'test_rmse': test_rmse, 'test_mae': test_mae,
            'test_pearson': test_pearson, 'test_sample_count': len(test_ground_truth_orig)
        })
    
    logger.info(f"CNN Performance (Iter {iteration}): Train RMSE: {metrics.get('train_rmse'):.4e}")
    if 'test_rmse' in metrics:
        logger.info(f"  Test RMSE: {metrics.get('test_rmse'):.4e}, Test Pearson: {metrics.get('test_pearson'):.4f}")
    return metrics

def run_dante_optimization(
    # 基本参数
    initial_data_size=800,
    n_al_iterations=400,  # 原项目完整迭代数
    rosenbrock_dimension=20,
    domain_range=(-2.048, 2.048),
    
    # CNN参数
    cnn_training_epochs=200,  # 原项目完整训练轮数
    cnn_batch_size=32,       # 原项目批次大小
    cnn_validation_split=0.2,
    cnn_early_stopping_patience=30,  # 原项目早停patience
    
    # NTE参数
    nte_c0=1.0,
    nte_rollout_rounds=100,
    validation_batch_size=20,
    
    # 动态探索因子参数 (重点调试)
    base_exploration_factor=0.2,  # 原项目验证的最佳基础因子
    exploration_exponent_base=2.0,
    exploration_max_limit=20,
    
    # 可视化控制
    viz_interval=10,
    detailed_viz_interval=50,  # 原项目设置
    
    # 数据路径
    data_input_path="/kaggle/input/rosenbrock-data-20d-800/rosenbrock_data_raw",
    results_output_path="/kaggle/working/results",
    
    # 实验标识
    experiment_name="default"
):
    """
    运行DANTE优化算法的主函数
    
    重点调试参数：
    - base_exploration_factor: 动态探索因子基础值 (默认0.8)
    - exploration_exponent_base: 指数基数 (默认2.0)
    - exploration_max_limit: 上限值 (默认20)
    
    动态公式: factor = base_exploration_factor + 1 * min(exploration_max_limit, exploration_exponent_base ** no_improvement_streak)
    """
    
    logger.info("="*60)
    logger.info(f"开始DANTE优化实验: {experiment_name}")
    logger.info(f"重点调试参数 - 基础因子: {base_exploration_factor}, 指数基数: {exploration_exponent_base}, 上限: {exploration_max_limit}")
    logger.info("="*60)
    
    # 创建结果目录
    experiment_results_dir = create_results_directory(results_output_path)
    if not experiment_results_dir:
        logger.error("无法创建结果目录，退出实验")
        return None
    
    # 记录实验参数
    experiment_params = {
        'experiment_name': experiment_name,
        'initial_data_size': initial_data_size,
        'n_al_iterations': n_al_iterations,
        'rosenbrock_dimension': rosenbrock_dimension,
        'domain_range': domain_range,
        'base_exploration_factor': base_exploration_factor,
        'exploration_exponent_base': exploration_exponent_base,
        'exploration_max_limit': exploration_max_limit,
        'nte_c0': nte_c0,
        'nte_rollout_rounds': nte_rollout_rounds,
        'cnn_training_epochs': cnn_training_epochs,
        'cnn_batch_size': cnn_batch_size,
        'timestamp': datetime.datetime.now().isoformat()
    }
    
    # 保存实验参数
    params_file = os.path.join(experiment_results_dir, 'experiment_parameters.json')
    with open(params_file, 'w') as f:
        import json
        json.dump(experiment_params, f, indent=2)
    logger.info(f"实验参数已保存: {params_file}")
    
    # 定义域
    domain_min, domain_max = domain_range
    rosenbrock_domain = [(domain_min, domain_max)] * rosenbrock_dimension
    
    # 加载初始数据
    try:
        x_train_path = os.path.join(data_input_path, "Rosenbrock_x_train.npy")
        y_train_path = os.path.join(data_input_path, "Rosenbrock_y_train.npy")
        x_test_path = os.path.join(data_input_path, "Rosenbrock_x_test.npy")
        y_test_path = os.path.join(data_input_path, "Rosenbrock_y_test.npy")
        
        # 加载测试集
        X_test = np.load(x_test_path)
        y_test = np.load(y_test_path)
        if y_test.ndim == 1:
            y_test = y_test.reshape(-1, 1)
        logger.info(f"测试集加载成功: X shape {X_test.shape}, y shape {y_test.shape}")
        
        # 加载训练集
        initial_X = np.load(x_train_path)
        initial_y = np.load(y_train_path)
        if initial_y.ndim == 1:
            initial_y = initial_y.reshape(-1, 1)
        logger.info(f"训练集加载成功: X shape {initial_X.shape}, y shape {initial_y.shape}")
        
        # 确保有足够的初始样本
        if initial_X.shape[0] < initial_data_size:
            additional_samples = initial_data_size - initial_X.shape[0]
            additional_X = np.random.uniform(low=domain_min, high=domain_max, size=(additional_samples, rosenbrock_dimension))
            additional_y = np.array([rosenbrock_function(x) for x in additional_X]).reshape(-1, 1)
            initial_X = np.vstack((initial_X, additional_X))
            initial_y = np.vstack((initial_y, additional_y))
            logger.info(f"补充了{additional_samples}个随机样本")
            
    except Exception as e:
        logger.error(f"加载数据失败: {e}")
        # 生成随机数据作为备用
        initial_X = np.random.uniform(low=domain_min, high=domain_max, size=(initial_data_size, rosenbrock_dimension))
        initial_y = np.array([rosenbrock_function(x) for x in initial_X]).reshape(-1, 1)
        
        # 生成测试集
        test_size = 200
        X_test = np.random.uniform(low=domain_min, high=domain_max, size=(test_size, rosenbrock_dimension))
        y_test = np.array([rosenbrock_function(x) for x in X_test]).reshape(-1, 1)
        logger.info("使用随机生成的数据")
    
    # 初始化数据管理器
    data_manager = DataManager(
        initial_X=initial_X, 
        initial_y=initial_y, 
        dimension=rosenbrock_dimension
    )
    logger.info("DataManager初始化完成")
    
    # 初始化CNN模型
    cnn_model = CNN1DSurrogate(input_dim=rosenbrock_dimension)
    logger.info("CNN1DSurrogate模型初始化完成")
    
    # 初始化NTE搜索器
    nte_searcher = NTESearcher(
        dimension=rosenbrock_dimension, 
        domain=rosenbrock_domain,
        c0=nte_c0,
        rollout_rounds=nte_rollout_rounds,
        validation_batch_size=validation_batch_size
    )
    logger.info("NTE搜索器初始化完成")
    
    # 历史记录变量
    history_iterations = []
    history_best_y = []
    history_pearson_all = []
    history_dynamic_factors = []  # 记录动态探索因子
    cnn_performance_history = []
    
    # 动态探索因子相关变量
    no_improvement_streak = 0
    previous_best_y = float('inf')
    
    # 开始主循环
    start_time = time.time()
    
    for al_iteration in range(n_al_iterations):
        iteration_start_time = time.time()
        current_iteration = al_iteration + 1
        history_iterations.append(current_iteration)
        
        # 计算动态探索因子 (重点调试公式)
        dynamic_exploration_factor = base_exploration_factor + 1 * min(exploration_max_limit, exploration_exponent_base ** no_improvement_streak)
        history_dynamic_factors.append(dynamic_exploration_factor)
        
        logger.info(f"\n--- 迭代 {current_iteration}/{n_al_iterations} ---")
        logger.info(f"动态探索因子: {dynamic_exploration_factor:.2f} (连续无改进次数: {no_improvement_streak})")
        
        # 决定是否生成可视化
        should_generate_plots = ((current_iteration % viz_interval == 0) or 
                               (current_iteration == n_al_iterations))
        should_generate_detailed_plots = ((current_iteration % detailed_viz_interval == 0) or 
                                        (current_iteration == n_al_iterations))
        
        # 训练CNN模型
        logger.info("开始训练CNN模型...")
        training_history = cnn_model.train(
            data_manager=data_manager,
            epochs=cnn_training_epochs,
            batch_size=cnn_batch_size,
            validation_split=cnn_validation_split,
            patience=cnn_early_stopping_patience,
            verbose=0
        )
        
        # 可视化训练历史
        if should_generate_plots:
            cnn_model.visualize_training_history(
                training_history,
                save_path=os.path.join(experiment_results_dir, f"cnn_training_history_iter_{current_iteration}.png")
            )
        
        # 评估模型性能
        perf_metrics = evaluate_cnn_performance(
            cnn_model, data_manager, experiment_results_dir, current_iteration, 
            X_test, y_test, should_generate_plots=should_generate_detailed_plots
        )
        cnn_performance_history.append(perf_metrics)
        
        # 计算皮尔逊相关系数
        current_pearson = calculate_pearson_on_test_set(cnn_model, X_test, y_test) if X_test is not None else calculate_pearson_correlation(cnn_model, data_manager)
        history_pearson_all.append(current_pearson)
        logger.info(f"模型皮尔逊相关系数: {current_pearson:.4f}")
        
        # NTE搜索新候选点
        logger.info("开始NTE搜索...")
        nte_model_wrapper = NTESurrogateWrapper(cnn_model, data_manager)
        
        current_best_x, current_best_y = data_manager.get_current_best()
        
        try:
            nte_search_result = nte_searcher.search(
                surrogate_model=nte_model_wrapper,
                current_best_state=current_best_x,
                min_y_observed=current_best_y,
                dynamic_exploration_factor=dynamic_exploration_factor,  # 传递动态探索因子
                all_states=data_manager.X_data,
                all_values=data_manager.get_all_y_orig(),
                current_iteration=current_iteration,
                return_ducb_trends=True
            )
            
            nte_candidates, ducb_trends_data = nte_search_result
            
            if len(nte_candidates) > 0:
                logger.info(f"NTE搜索找到{len(nte_candidates)}个候选点")
                
                # 评估候选点
                nte_y_true = np.array([rosenbrock_function(x) for x in nte_candidates]).reshape(-1, 1)
                
                # 添加到数据管理器
                data_manager.add_samples(nte_candidates, nte_y_true, re_normalize=True, shuffle=True)
                logger.info(f"已添加{len(nte_candidates)}个新样本，数据集总大小: {data_manager.num_samples}")
                
                # 更新最佳值
                current_best_x, current_best_y = data_manager.get_current_best()
                history_best_y.append(current_best_y)
                
                # 检查是否有改进
                if current_best_y < previous_best_y:
                    logger.info(f"发现更优解: {current_best_y:.6e} (改进: {previous_best_y - current_best_y:.6e})")
                    no_improvement_streak = 0
                    
                    # 发现新最优解时绘制维度分布图
                    if should_generate_plots:
                        plot_top_samples_dimensions(
                            data_manager=data_manager,
                            output_dir=experiment_results_dir,
                            iteration=current_iteration,
                            n_top_samples=15
                        )
                else:
                    no_improvement_streak += 1
                    logger.info(f"本轮无改进，连续无改进次数: {no_improvement_streak}")
                
                previous_best_y = current_best_y
                
                # 绘制DUCB趋势图
                if ducb_trends_data and should_generate_plots:
                    plot_nte_ducb_trends(
                        ducb_trends_data=ducb_trends_data,
                        output_dir=experiment_results_dir,
                        iteration=current_iteration,
                        dynamic_exploration_factor=dynamic_exploration_factor
                    )
            else:
                logger.info("NTE搜索未找到有效候选点")
                no_improvement_streak += 1
                if history_best_y:
                    history_best_y.append(history_best_y[-1])
                else:
                    history_best_y.append(current_best_y)
                    
        except Exception as e:
            logger.error(f"NTE搜索失败: {e}")
            no_improvement_streak += 1
            if history_best_y:
                history_best_y.append(history_best_y[-1])
            else:
                history_best_y.append(current_best_y)
        
        # 生成可视化图表
        if should_generate_plots:
            # 全局最优值趋势
            plot_global_min_value_trend(
                history_iterations[:len(history_best_y)], 
                history_best_y, 
                experiment_results_dir, 
                filename=f"global_min_iter_{current_iteration}.png"
            )
            
            # 皮尔逊相关系数趋势
            plot_pearson_correlation_trend(
                history_iterations, 
                history_pearson_all, 
                experiment_results_dir,
                filename=f"pearson_trend_iter_{current_iteration}.png"
            )
            
            # 快速进度概览
            quick_progress_plot(
                history_iterations[:len(history_best_y)], 
                history_best_y, 
                history_pearson_all, 
                experiment_results_dir, 
                current_iteration
            )
        
        # 内存管理
        if current_iteration % 10 == 0:
            data_manager.clear_cache()
            gc.collect()
            logger.info("已清理内存缓存")
        
        iteration_time = time.time() - iteration_start_time
        logger.info(f"迭代{current_iteration}完成，用时: {iteration_time:.1f}秒")
    
    # 保存最终结果
    total_time = time.time() - start_time
    logger.info(f"\n实验完成！总用时: {total_time:.1f}秒")
    
    # 保存性能历史
    perf_csv_path = os.path.join(experiment_results_dir, "performance_history.csv")
    pd.DataFrame(cnn_performance_history).to_csv(perf_csv_path, index=False)
    
    # 保存动态因子历史
    factor_history = pd.DataFrame({
        'iteration': history_iterations,
        'dynamic_exploration_factor': history_dynamic_factors,
        'no_improvement_streak': [factor_data for factor_data in range(len(history_dynamic_factors))],  # 简化记录
        'best_value': history_best_y[:len(history_iterations)],
        'pearson_correlation': history_pearson_all
    })
    factor_csv_path = os.path.join(experiment_results_dir, "dynamic_factor_history.csv")
    factor_history.to_csv(factor_csv_path, index=False)
    
    # 保存最终数据状态
    final_data_path = os.path.join(experiment_results_dir, "final_data_state.npz")
    data_manager.save_data(final_data_path)
    
    # 实验总结
    final_best_x, final_best_y = data_manager.get_current_best()
    experiment_summary = {
        'experiment_name': experiment_name,
        'total_iterations': n_al_iterations,
        'final_best_value': float(final_best_y),
        'final_pearson_correlation': float(history_pearson_all[-1]),
        'total_time_seconds': total_time,
        'average_time_per_iteration': total_time / n_al_iterations,
        'total_samples_evaluated': data_manager.num_samples,
        'max_dynamic_factor': float(max(history_dynamic_factors)),
        'final_dynamic_factor': float(history_dynamic_factors[-1])
    }
    
    summary_file = os.path.join(experiment_results_dir, 'experiment_summary.json')
    with open(summary_file, 'w') as f:
        json.dump(experiment_summary, f, indent=2)
    
    logger.info(f"实验总结:")
    logger.info(f"  最终最优值: {final_best_y:.6e}")
    logger.info(f"  最终相关系数: {history_pearson_all[-1]:.4f}")
    logger.info(f"  总样本数: {data_manager.num_samples}")
    logger.info(f"  最大动态因子: {max(history_dynamic_factors):.2f}")
    logger.info(f"  结果保存路径: {experiment_results_dir}")
    
    return {
        'results_dir': experiment_results_dir,
        'final_best_value': final_best_y,
        'final_pearson': history_pearson_all[-1],
        'experiment_summary': experiment_summary,
        'performance_history': cnn_performance_history,
        'factor_history': factor_history
    }

logger.info("主执行逻辑模块加载完成")
print("DANTE主执行逻辑已准备就绪 - 重点调试动态探索因子")


In [ ]:
# Cell 19: 实际运行代码和测试
def run_quick_demo():
    """运行一个快速的DANTE演示，适合Kaggle环境测试"""
    logger.info("开始运行DANTE快速演示...")
    
    try:
        # 快速演示参数
        demo_result = run_dante_experiment(
            experiment_name="kaggle_demo",
            n_al_iterations=15,  # 快速演示仅15次迭代
            cnn_training_epochs=30,  # 减少训练轮数
            initial_data_size=200,   # 减少初始数据
            nte_rollout_rounds=20,   # 减少NTE轮数
            viz_interval=5,          # 每5次迭代可视化
            detailed_viz_interval=10,
            # 使用默认的动态探索因子参数
            base_exploration_factor=0.8,
            exploration_exponent_base=2,
            exploration_max_limit=20
        )
        
        if demo_result:
            logger.info("快速演示完成!")
            logger.info(f"最终最优值: {demo_result['final_best_value']:.6e}")
            logger.info(f"结果目录: {demo_result['results_dir']}")
            return demo_result
        else:
            logger.error("快速演示失败")
            return None
            
    except Exception as e:
        logger.error(f"快速演示出现异常: {e}")
        import traceback
        traceback.print_exc()
        return None

def run_parameter_test():
    """运行参数测试 - 简化版参数扫描"""
    logger.info("开始运行参数测试...")
    
    # 简化的参数组合，适合Kaggle环境
    test_configs = [
        {'base': 0.8, 'exp': 2.0, 'max': 15, 'name': 'conservative'},
        {'base': 1.0, 'exp': 2.5, 'max': 20, 'name': 'aggressive'}
    ]
    
    results = []
    
    for i, config in enumerate(test_configs):
        logger.info(f"测试配置 {i+1}/{len(test_configs)}: {config['name']}")
        
        try:
            result = run_dante_experiment(
                experiment_name=f"param_test_{config['name']}",
                n_al_iterations=10,  # 快速测试
                cnn_training_epochs=20,
                initial_data_size=150,
                nte_rollout_rounds=15,
                viz_interval=5,
                base_exploration_factor=config['base'],
                exploration_exponent_base=config['exp'],
                exploration_max_limit=config['max']
            )
            
            if result:
                results.append({
                    'config_name': config['name'],
                    'parameters': config,
                    'final_best_value': result['final_best_value'],
                    'final_pearson': result['final_pearson']
                })
                logger.info(f"配置 {config['name']} 完成: {result['final_best_value']:.6e}")
            
        except Exception as e:
            logger.error(f"配置 {config['name']} 失败: {e}")
    
    if results:
        # 比较结果
        logger.info("\n参数测试结果比较:")
        for r in results:
            logger.info(f"{r['config_name']}: 最优值={r['final_best_value']:.6e}, Pearson={r['final_pearson']:.4f}")
        
        # 找到最佳配置
        best = min(results, key=lambda x: x['final_best_value'])
        logger.info(f"\n最佳配置: {best['config_name']}")
        logger.info(f"参数: {best['parameters']}")
        
        return results
    else:
        logger.error("所有参数测试都失败了")
        return None

# 准备就绪提示
logger.info("="*80)
logger.info("DANTE Kaggle Notebook 已完全加载!")
logger.info("="*80)
logger.info("可用的运行函数:")
logger.info("1. run_quick_demo() - 运行快速演示 (推荐先运行)")
logger.info("2. run_parameter_test() - 运行参数对比测试")
logger.info("3. run_dante_experiment(name, **params) - 运行自定义实验")
logger.info("="*80)

print("✅ DANTE Notebook 完全加载完成!")
print("🚀 现在可以运行: run_quick_demo() 开始体验")


In [ ]:
# Cell 21: 最终完整性检查和运行测试
def check_notebook_completeness():
    """检查notebook的完整性，确保所有组件都正确加载"""
    logger.info("开始检查notebook完整性...")
    
    checks = {
        'DataManager': DataManager,
        'CNN1DSurrogate': CNN1DSurrogate, 
        'NTESearcher': NTESearcher,
        'PerformanceOptimizer': PerformanceOptimizer,
        'ProcessOptimizer': ProcessOptimizer,
        'NTESurrogateWrapper': NTESurrogateWrapper,
        'safe_power10': safe_power10,
        'rosenbrock_function': rosenbrock_function,
        'plot_global_min_value_trend': plot_global_min_value_trend,
        'plot_prediction_vs_truth': plot_prediction_vs_truth,
        'plot_nte_ducb_trends': plot_nte_ducb_trends,
        'run_dante_experiment': run_dante_experiment,
        'setup_kaggle_environment': setup_kaggle_environment,
        'load_initial_data': load_initial_data
    }
    
    missing = []
    working = []
    
    for name, obj in checks.items():
        try:
            if callable(obj):
                working.append(name)
            else:
                working.append(name)
        except NameError:
            missing.append(name)
    
    # 检查核心依赖
    try:
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        import tensorflow as tf
        import torch
        import psutil
        import scipy
        deps_ok = True
    except ImportError as e:
        deps_ok = False
        logger.error(f"依赖检查失败: {e}")
    
    # 检查GPU可用性
    gpu_available = len(tf.config.experimental.list_physical_devices('GPU')) > 0
    torch_gpu = torch.cuda.is_available()
    
    # 报告结果
    logger.info(f"✅ 组件检查: {len(working)}/{len(checks)} 个组件正常")
    logger.info(f"✅ 依赖检查: {'通过' if deps_ok else '失败'}")
    logger.info(f"🔧 TensorFlow GPU: {'可用' if gpu_available else '不可用'}")
    logger.info(f"🔧 PyTorch GPU: {'可用' if torch_gpu else '不可用'}")
    
    if missing:
        logger.warning(f"❌ 缺失组件: {missing}")
        return False
    
    if not deps_ok:
        logger.error("❌ 依赖检查失败")
        return False
    
    logger.info("🎉 Notebook完整性检查通过!")
    return True

def quick_system_test():
    """快速系统测试，验证核心功能"""
    logger.info("开始快速系统测试...")
    
    try:
        # 测试数据管理器
        test_X = np.random.randn(50, 20)
        test_y = np.random.randn(50, 1)
        dm = DataManager(test_X, test_y, 20)
        logger.info("✅ 数据管理器测试通过")
        
        # 测试CNN模型
        cnn = CNN1DSurrogate(input_dim=20)
        logger.info("✅ CNN模型初始化通过")
        
        # 测试NTE搜索器
        domain = [(-2, 2)] * 20
        nte = NTESearcher(dimension=20, domain=domain, c0=1, rollout_rounds=5, validation_batch_size=5)
        logger.info("✅ NTE搜索器初始化通过")
        
        # 测试环境设置
        output_path, data_files = setup_kaggle_environment()
        logger.info(f"✅ 环境设置通过: {output_path}")
        
        # 测试Rosenbrock函数
        test_vec = np.array([1.0] * 20)
        result = rosenbrock_function(test_vec)
        logger.info(f"✅ Rosenbrock函数测试通过: f(1,...,1) = {result}")
        
        logger.info("🎉 所有系统测试通过!")
        return True
        
    except Exception as e:
        logger.error(f"❌ 系统测试失败: {e}")
        import traceback
        traceback.print_exc()
        return False

# 运行检查
completeness_ok = check_notebook_completeness()
system_ok = quick_system_test()

if completeness_ok and system_ok:
    logger.info("="*80)
    logger.info("🚀 DANTE Notebook已完全准备就绪!")
    logger.info("💡 建议运行: run_quick_demo() 开始体验")
    logger.info("="*80)
    print("✅ Notebook验证完成 - 可以安全运行!")
else:
    logger.error("❌ Notebook验证失败，请检查错误信息")
    print("❌ 验证失败，请检查日志")


In [ ]:
# Cell 8: 参数调试实验框架 - 动态探索因子优化
import itertools
import json
from datetime import datetime

def run_parameter_sweep_experiment():
    """
    运行参数扫描实验，调试动态探索因子的最优组合
    
    重点调试的动态公式:
    dynamic_exploration_factor = base_exploration_factor + 1 * min(exploration_max_limit, exploration_exponent_base ** no_improvement_streak)
    """
    
    logger.info("="*80)
    logger.info("开始动态探索因子参数扫描实验")
    logger.info("="*80)
    
    # 参数网格定义
    parameter_grid = {
        'base_exploration_factor': [0.5, 0.8, 1.0, 1.2],  # 基础因子
        'exploration_exponent_base': [1.5, 2.0, 2.5],     # 指数基数
        'exploration_max_limit': [10, 15, 20, 25]         # 上限值
    }
    
    # 快速测试参数
    quick_test_params = {
        'n_al_iterations': 30,  # 快速测试用较少迭代
        'cnn_training_epochs': 50,
        'initial_data_size': 400,  # 减少初始数据量
        'viz_interval': 15,  # 较少可视化频率
        'detailed_viz_interval': 30
    }
    
    # 生成所有参数组合
    param_combinations = list(itertools.product(
        parameter_grid['base_exploration_factor'],
        parameter_grid['exploration_exponent_base'], 
        parameter_grid['exploration_max_limit']
    ))
    
    logger.info(f"总共将测试 {len(param_combinations)} 种参数组合")
    
    # 存储实验结果
    sweep_results = []
    best_result = None
    best_score = float('inf')
    
    # 创建扫描实验总目录
    sweep_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    sweep_base_dir = f"/kaggle/working/parameter_sweep_{sweep_timestamp}"
    os.makedirs(sweep_base_dir, exist_ok=True)
    
    for i, (base_factor, exp_base, max_limit) in enumerate(param_combinations):
        experiment_name = f"sweep_{i+1:02d}_base{base_factor}_exp{exp_base}_max{max_limit}"
        
        logger.info(f"\n开始实验 {i+1}/{len(param_combinations)}: {experiment_name}")
        logger.info(f"参数: base_factor={base_factor}, exp_base={exp_base}, max_limit={max_limit}")
        
        try:
            # 运行单次实验
            result = run_dante_optimization(
                # 动态探索因子参数 (重点调试)
                base_exploration_factor=base_factor,
                exploration_exponent_base=exp_base,
                exploration_max_limit=max_limit,
                
                # 快速测试参数
                **quick_test_params,
                
                # 实验标识
                experiment_name=experiment_name,
                results_output_path=sweep_base_dir
            )
            
            if result is not None:
                # 记录关键指标
                experiment_record = {
                    'experiment_id': i + 1,
                    'experiment_name': experiment_name,
                    'base_exploration_factor': base_factor,
                    'exploration_exponent_base': exp_base,
                    'exploration_max_limit': max_limit,
                    'final_best_value': float(result['final_best_value']),
                    'final_pearson': float(result['final_pearson']),
                    'max_dynamic_factor': float(result['experiment_summary']['max_dynamic_factor']),
                    'total_time': float(result['experiment_summary']['total_time_seconds']),
                    'total_samples': int(result['experiment_summary']['total_samples_evaluated']),
                    'results_dir': result['results_dir']
                }
                
                sweep_results.append(experiment_record)
                
                # 更新最佳结果 (以最终最优值为主要指标)
                if result['final_best_value'] < best_score:
                    best_score = result['final_best_value']
                    best_result = experiment_record.copy()
                
                logger.info(f"实验完成: 最优值={result['final_best_value']:.4e}, 相关系数={result['final_pearson']:.4f}")
                
            else:
                logger.error(f"实验 {experiment_name} 失败")
                
        except Exception as e:
            logger.error(f"实验 {experiment_name} 出现异常: {e}")
            continue
    
    # 保存扫描结果
    sweep_results_file = os.path.join(sweep_base_dir, 'parameter_sweep_results.json')
    with open(sweep_results_file, 'w') as f:
        json.dump({
            'sweep_timestamp': sweep_timestamp,
            'total_experiments': len(param_combinations),
            'successful_experiments': len(sweep_results),
            'parameter_grid': parameter_grid,
            'quick_test_params': quick_test_params,
            'best_result': best_result,
            'all_results': sweep_results
        }, f, indent=2)
    
    # 创建结果分析
    if sweep_results:
        results_df = pd.DataFrame(sweep_results)
        
        # 保存CSV格式结果
        results_csv = os.path.join(sweep_base_dir, 'parameter_sweep_results.csv')
        results_df.to_csv(results_csv, index=False)
        
        # 生成分析报告
        analysis_report = generate_parameter_analysis_report(results_df, sweep_base_dir)
        
        logger.info(f"\n" + "="*80)
        logger.info("参数扫描实验完成！")
        logger.info(f"成功完成 {len(sweep_results)}/{len(param_combinations)} 个实验")
        
        if best_result:
            logger.info(f"\n最佳参数组合:")
            logger.info(f"  基础因子: {best_result['base_exploration_factor']}")
            logger.info(f"  指数基数: {best_result['exploration_exponent_base']}")
            logger.info(f"  上限值: {best_result['exploration_max_limit']}")
            logger.info(f"  最优值: {best_result['final_best_value']:.6e}")
            logger.info(f"  相关系数: {best_result['final_pearson']:.4f}")
        
        logger.info(f"\n结果保存位置: {sweep_base_dir}")
        logger.info(f"详细结果: {sweep_results_file}")
        logger.info(f"CSV格式: {results_csv}")
        logger.info("="*80)
        
        return {
            'sweep_dir': sweep_base_dir,
            'best_result': best_result,
            'all_results': sweep_results,
            'results_df': results_df,
            'analysis_report': analysis_report
        }
    else:
        logger.error("参数扫描实验失败：没有成功的实验")
        return None

def generate_parameter_analysis_report(results_df, output_dir):
    """生成参数分析报告和可视化"""
    
    logger.info("生成参数分析报告...")
    
    try:
        # 基本统计分析
        report = {
            'summary_stats': {
                'best_value_mean': float(results_df['final_best_value'].mean()),
                'best_value_std': float(results_df['final_best_value'].std()),
                'best_value_min': float(results_df['final_best_value'].min()),
                'best_value_max': float(results_df['final_best_value'].max()),
                'pearson_mean': float(results_df['final_pearson'].mean()),
                'pearson_std': float(results_df['final_pearson'].std())
            }
        }
        
        # 按参数分组分析
        base_factor_analysis = results_df.groupby('base_exploration_factor').agg({
            'final_best_value': ['mean', 'std', 'min'],
            'final_pearson': ['mean', 'std']
        }).round(6)
        
        exp_base_analysis = results_df.groupby('exploration_exponent_base').agg({
            'final_best_value': ['mean', 'std', 'min'],
            'final_pearson': ['mean', 'std']
        }).round(6)
        
        max_limit_analysis = results_df.groupby('exploration_max_limit').agg({
            'final_best_value': ['mean', 'std', 'min'],
            'final_pearson': ['mean', 'std']
        }).round(6)
        
        # 创建可视化图表
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # 1. 基础因子影响
        base_means = results_df.groupby('base_exploration_factor')['final_best_value'].mean()
        axes[0,0].bar(base_means.index.astype(str), base_means.values)
        axes[0,0].set_title('Base Exploration Factor vs Best Value')
        axes[0,0].set_xlabel('Base Factor')
        axes[0,0].set_ylabel('Mean Best Value')
        axes[0,0].set_yscale('log')
        
        # 2. 指数基数影响
        exp_means = results_df.groupby('exploration_exponent_base')['final_best_value'].mean()
        axes[0,1].bar(exp_means.index.astype(str), exp_means.values)
        axes[0,1].set_title('Exponent Base vs Best Value')
        axes[0,1].set_xlabel('Exponent Base')
        axes[0,1].set_ylabel('Mean Best Value')
        axes[0,1].set_yscale('log')
        
        # 3. 上限值影响
        limit_means = results_df.groupby('exploration_max_limit')['final_best_value'].mean()
        axes[0,2].bar(limit_means.index.astype(str), limit_means.values)
        axes[0,2].set_title('Max Limit vs Best Value')
        axes[0,2].set_xlabel('Max Limit')
        axes[0,2].set_ylabel('Mean Best Value')
        axes[0,2].set_yscale('log')
        
        # 4. 皮尔逊相关系数分析
        base_pearson = results_df.groupby('base_exploration_factor')['final_pearson'].mean()
        axes[1,0].bar(base_pearson.index.astype(str), base_pearson.values)
        axes[1,0].set_title('Base Factor vs Pearson Correlation')
        axes[1,0].set_xlabel('Base Factor')
        axes[1,0].set_ylabel('Mean Pearson Correlation')
        
        exp_pearson = results_df.groupby('exploration_exponent_base')['final_pearson'].mean()
        axes[1,1].bar(exp_pearson.index.astype(str), exp_pearson.values)
        axes[1,1].set_title('Exponent Base vs Pearson Correlation')
        axes[1,1].set_xlabel('Exponent Base')
        axes[1,1].set_ylabel('Mean Pearson Correlation')
        
        # 5. 散点图：最优值 vs 相关系数
        scatter = axes[1,2].scatter(results_df['final_best_value'], results_df['final_pearson'], 
                                   c=results_df['base_exploration_factor'], cmap='viridis', alpha=0.7)
        axes[1,2].set_xlabel('Final Best Value')
        axes[1,2].set_ylabel('Final Pearson Correlation')
        axes[1,2].set_title('Best Value vs Pearson Correlation')
        axes[1,2].set_xscale('log')
        plt.colorbar(scatter, ax=axes[1,2], label='Base Factor')
        
        plt.tight_layout()
        
        # 保存图表
        analysis_plot_path = os.path.join(output_dir, 'parameter_analysis_report.png')
        plt.savefig(analysis_plot_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        # 保存文本报告
        report_text = f"""
参数扫描实验分析报告
===================

实验概要:
- 总实验数: {len(results_df)}
- 最优值范围: {report['summary_stats']['best_value_min']:.4e} - {report['summary_stats']['best_value_max']:.4e}
- 平均最优值: {report['summary_stats']['best_value_mean']:.4e} ± {report['summary_stats']['best_value_std']:.4e}
- 平均相关系数: {report['summary_stats']['pearson_mean']:.4f} ± {report['summary_stats']['pearson_std']:.4f}

按基础探索因子分析:
{base_factor_analysis}

按指数基数分析:
{exp_base_analysis}

按上限值分析:
{max_limit_analysis}

最佳实验:
{results_df.loc[results_df['final_best_value'].idxmin()].to_dict()}
        """
        
        report_text_path = os.path.join(output_dir, 'parameter_analysis_report.txt')
        with open(report_text_path, 'w') as f:
            f.write(report_text)
        
        report['analysis_plot'] = analysis_plot_path
        report['report_text'] = report_text_path
        
        logger.info(f"分析报告已保存: {report_text_path}")
        logger.info(f"分析图表已保存: {analysis_plot_path}")
        
        return report
        
    except Exception as e:
        logger.error(f"生成分析报告时出错: {e}")
        return None

# 快速单次实验函数
def run_quick_experiment(
    base_exploration_factor=0.8,
    exploration_exponent_base=2.0,
    exploration_max_limit=20,
    n_iterations=20,
    experiment_name="quick_test"
):
    """运行快速单次实验，用于验证特定参数组合"""
    
    logger.info(f"运行快速实验: {experiment_name}")
    logger.info(f"动态探索因子参数: base={base_exploration_factor}, exp_base={exploration_exponent_base}, max_limit={exploration_max_limit}")
    
    result = run_dante_optimization(
        # 动态探索因子参数
        base_exploration_factor=base_exploration_factor,
        exploration_exponent_base=exploration_exponent_base,
        exploration_max_limit=exploration_max_limit,
        
        # 快速测试参数
        n_al_iterations=n_iterations,
        cnn_training_epochs=30,
        initial_data_size=200,
        viz_interval=10,
        detailed_viz_interval=20,
        
        # 实验标识
        experiment_name=experiment_name
    )
    
    return result

logger.info("参数调试实验框架加载完成")
print("🎯 参数调试实验框架已准备就绪")
print("📊 可使用 run_parameter_sweep_experiment() 进行参数网格搜索")
print("⚡ 可使用 run_quick_experiment() 进行快速单次测试")


In [ ]:
# Cell 9: 测试和使用指南
print("🚀 DANTE Rosenbrock 优化系统 - Kaggle版本")
print("="*60)
print("所有模块已成功加载完成！")
print()

print("📋 可用的主要功能:")
print("1. run_quick_experiment() - 快速单次实验 (推荐先运行)")
print("2. run_dante_optimization() - 完整DANTE优化算法")
print("3. run_parameter_sweep_experiment() - 参数网格搜索")
print()

print("🎯 重点调试参数 (动态探索因子公式):")
print("dynamic_exploration_factor = base_exploration_factor + 1 * min(exploration_max_limit, exploration_exponent_base ** no_improvement_streak)")
print()
print("  • base_exploration_factor: 基础因子 (默认0.8)")
print("  • exploration_exponent_base: 指数基数 (默认2.0)")  
print("  • exploration_max_limit: 上限值 (默认20)")
print()

print("💻 快速开始示例:")
print()
print("# 1. 运行快速测试 (20次迭代)")
print("result = run_quick_experiment()")
print()
print("# 2. 调试特定参数组合")
print("result = run_quick_experiment(")
print("    base_exploration_factor=1.0,")
print("    exploration_exponent_base=2.5,")
print("    exploration_max_limit=15,")
print("    n_iterations=30")
print(")")
print()
print("# 3. 运行完整实验 (50次迭代)")
print("result = run_dante_optimization(")
print("    base_exploration_factor=0.8,")
print("    exploration_exponent_base=2.0,")
print("    exploration_max_limit=20,")
print("    n_al_iterations=50")
print(")")
print()
print("# 4. 参数网格搜索 (48种组合)")
print("sweep_result = run_parameter_sweep_experiment()")
print()

print("📊 输出结果说明:")
print("  • 最优值趋势图: 优化进展可视化")
print("  • 皮尔逊相关系数: 模型质量评估")
print("  • DUCB趋势图: NTE搜索分析")
print("  • 维度分布图: 最优解特征分析")
print("  • 实验参数和总结: JSON格式保存")
print()

print("🔧 系统状态检查:")
print(f"  • GPU可用: {'✅' if gpu_available else '❌'}")
print(f"  • TensorFlow版本: {tf.__version__}")
print(f"  • 数据路径: /kaggle/input/rosenbrock-data-20d-800/rosenbrock_data_raw")
print(f"  • 结果路径: /kaggle/working/results")
print()

print("⚠️  重要提醒:")
print("  • 确保已上传Rosenbrock数据集到Kaggle")
print("  • P100 GPU推荐，16GB内存")
print("  • 完整400次迭代需要8-12小时")
print("  • 先运行快速测试验证环境")
print()

print("🎓 实验建议:")
print("  1. 先运行 run_quick_experiment() 验证环境")
print("  2. 尝试不同的动态探索因子参数")
print("  3. 观察DUCB趋势图分析搜索行为")
print("  4. 使用参数扫描找到最优组合")
print("  5. 最后运行完整400次迭代实验")
print()

print("准备开始优化实验！🚀")

# 运行环境验证
try:
    # 验证所有核心组件
    test_rosenbrock_val = rosenbrock_function(np.ones(20))
    test_log_transform = safe_power10(np.array([1.0, 2.0]))
    print(f"✅ 环境验证通过 - Rosenbrock(1^20) = {test_rosenbrock_val:.2f}")
except Exception as e:
    print(f"❌ 环境验证失败: {e}")
